<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [2]:
import json
import random
import datetime
import os
import logging
import pickle
import re
import uuid
import threading
import asyncio
from typing import *
from tqdm import tqdm
import time
import yaml
import sys
import asyncio
import json
import random
import time
from tqdm.asyncio import tqdm_asyncio
from graphs.common.utils import collect_vocabulary
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from libs.LLMs import *
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [3]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

#### load Models

In [4]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_chinese): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- Kanji reading (kanji_reading): 8 questions
- Write Chinese characters (write_chinese): 6 questions
- Word Meaning Selection (word_meaning): 11 questions
- Synonyms substitution (synonym_substitution): 5 questions
- word usage (word_usage): 5 questions
- Grammar fill in the blank (sentence_grammar): 13 questions
- Sentence sorting (sentence_sort): 5 questions
- Grammar structure selection (sentence_structure): 4-5 questions

Section 2: Reading Comprehension
- Short passages (short_passage_read): 4 articles
- Mid-size passages (midsize_passage_read): 2 articles
- Long passages (long_passage_read): 1 articles
- Information retrieval (info_retrieval): 1 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 6 questions
- Key understanding (keypoint_understanding): 6 questions
- Summary understanding (summary_understanding): 3 questions
- Active expression (active_expression): 4 questions
- Immediate acknowledgment (immediate_ack): 9 questions
"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. the complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First,  randomly pick words in 'Vocabulary' for questions in Vocabulary and Grammar. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [5]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [6]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [7]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-05-16 13:47:06,296 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [8]:
from IPython.display import display, Markdown, Latex
display(Markdown(initial_outline.as_str))

# 日本語能力試験N3模擬試験

## 第1部：語彙・文法

### 漢字読み (kanji_reading)

8問：漢字の読みをひらがなで答える問題です。

- **旗**
- **炎**
- **袖**
- **城**
- **景気**
- **資格**
- **頂上**
- **貯金**

### 漢字書き取り (write_chinese)

6問：ひらがなを漢字に直す問題です。

- **ちゅうもく（注目）**
- **けっか（結果）**
- **しゅじゅつ（手術）**
- **きんぞく（金属）**
- **ちょきん（貯金）**
- **はっぴょう（発表）**

### 語彙意味選択 (word_meaning)

11問：文中に使われている語彙の意味として最も適切なものを選ぶ問題です。

- **妙**
- **意外**
- **就職**
- **資格**
- **誤解**
- **意志**
- **農家**
- **耐える**
- **態度**
- **支払う**
- **拍手**

### 同意語置き換え (synonym_substitution)

5問：文中の語句を同じ意味の別の語句に置き換える問題です。

- **改善**
- **交換**
- **参加**
- **整理**
- **終了**

### 語彙用法 (word_usage)

5問：語彙の正しい使い方を選ぶ問題です。

- **支払う**
- **就職**
- **判断**
- **届ける**
- **並ぶ**

### 文法穴埋め (sentence_grammar)

13問：文中の空欄に最も適切な語句や文法形式を選ぶ問題です。

- **ますます**
- **ところが**
- **そのうち**
- **しかも**
- **それとも**
- **だけど**
- **～観**
- **違いない**
- **いけない**
- **かもしれない**
- **すなわち**
- **つまり**
- **たとえ**

### 文の並び替え (sentence_sort)

5問：バラバラになった文を正しい順番に並び替える問題です。

- **発表**
- **就職**
- **判断**
- **改善**
- **資格**

### 文法構造選択 (sentence_structure)

4問：与えられた文の構造や接続詞の使い方について最も適切なものを選ぶ問題です。

- **しかも**
- **すなわち**
- **だけど**
- **たとえ**

## 第2部：読解

### 短文読解 (short_passage_read)

4題：短い文章を読んで、内容に関する質問に答える問題です。

- **店で価格を尋ねる**
- **レストランで食べ物を注文する**
- **道を尋ねる**
- **天気の状況について話す**

### 中文読解 (midsize_passage_read)

2題：中程度の長さの文章を読んで、理解を問う問題です。

- **趣味について話す**
- **旅行の計画について話す**

### 長文読解 (long_passage_read)

1題：長文を読んで、全体の内容や筆者の意図を問う問題です。

- **家族について話す**

### 情報検索 (info_retrieval)

1題：表や案内などから必要な情報を探す問題です。

- **電車の切符を買う**

## 第3部：聴解

### 話題理解 (topic_understanding)

6問：会話や短い話を聞いて、主な話題を理解する問題です。

- **購入したい商品の説明**
- **支払い方法について話す**
- **交通手段について話す**
- **週末の予定について話す**
- **健康とフィットネスについて話す**
- **友人との日常会話**

### 要点理解 (keypoint_understanding)

6問：会話や説明を聞いて、要点や重要な情報を理解する問題です。

- **割引交渉**
- **おすすめを尋ねる**
- **領収書を求める**
- **仕事のプロジェクトについて話す**
- **支払い方法について話す**
- **交通状況について話す**

### 概要理解 (summary_understanding)

3問：少し長めの話を聞いて、全体の内容を要約する問題です。

- **健康診断や医者への訪問について話す**
- **最近の映画について話す**
- **家事の分担について話す**

### 積極的表現 (active_expression)

4問：聞いた内容に対して自分の意思や考えを表現する問題です。

- **友人へのプレゼント選びについて話す**
- **特別な日の計画について話す**
- **言語学習のコツについて話す**
- **新しいスキルを学ぶ計画について話す**

### 即時応答 (immediate_ack)

9問：会話の中で適切に返事や反応をする問題です。

- **バスの時刻表を尋ねる**
- **ショッピング体験を説明する**
- **お気に入りのレストランについて話す**
- **支払い方法について話す**
- **趣味について話す**
- **公共施設の利用方法について話す**
- **異文化交流について話す**
- **家事の分担について話す**
- **学校や職場での一日の流れについて話す**

In [9]:
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = subsection['subsection_title'].split(' (')[1].rstrip(')')
        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing 第1部：語彙・文法:   0%|          | 0/8 [00:00<?, ?it/s]

---WEB SEARCH---
---WEB SEARCH---
---Generator----


2025-05-16 13:47:18,531 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


---REVISOR---


2025-05-16 13:47:26,015 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


---Generator----


Processing 第1部：語彙・文法:   0%|          | 0/8 [00:25<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# Display as formatted YAML
display(Markdown(yaml_str))

sections:
- section_title: 第1部：語彙・文法
  subsections:
  - subsection_title: 漢字の読み (kanji_reading)
    description: 8問：漢字の読み方をひらがなで答える問題。
    question_topics:
    - topic: 悲劇
      result:
        question: 悲劇 の漢字の読み方を選びなさい。
        choices:
        - ひげき
        - かんげき
        - ひげつ
        - かなしいげき
        correct_answer: 1
        explanation: 「悲劇」はN3レベルの語彙として重要です。「悲」は「ひ」と音読みし、「劇」は「げき」と音読みします。選択肢の「かんげき」は「感激」という別の語の読みであり、「ひげつ」は実際には「劇」の読みではありません（「げつ」は「月」の音読みです）。また、「かなしいげき」は訓読みと音読みを混ぜた不正確な読み方です。音読みの区別が重要ですので、正しい読みを覚えましょう。
    - topic: 収める
      result:
        question: 書類を棚に**収めた**。
        choices:
        - おさめた
        - いれた
        - ながめた
        - おさえた
        correct_answer: 1
        explanation: 「収める」は「中にきちんと入れる」「収納する」という意味で使われ、読み方は「おさめる」です。例文の「書類を棚に収めた」は、「書類を棚の中にきちんと入れた」という意味になり、最も自然な使い方です。他の選択肢について：-
          2「いれた」は意味は近いですが、漢字「収めた」の読みではありません。- 3「ながめた」は「眺めた」（見る）で意味が異なります。- 4「おさえた」は「押さえた」（押す・抑える）で意味も読みも違います。使い分けポイント：-
          収める（おさめる）：物を中に入れる、成果を得る（例：成果を収める）- 納める（おさめる）：税金や会費などを支払う、物を納入する（例：税金を納める）-
          治める（おさめる）：国や集団を支配・統治する（例：国を治める）- 修める（おさめる）：学問や技術を身につける（例：学業を修める）このように、同じ「おさめる」でも使う漢字によって意味が異なるため、文脈をよく読み取ることが大切です。
    - topic: 間隔
      result:
        question: 次の文の（　）に入る漢字の正しい読み方を選びなさい。この列車は10分の（　）で出発します。
        choices:
        - かんけつ
        - かんかく
        - けんかく
        - かんがく
        correct_answer: 2
        explanation: 「間隔」は「かんかく」と読みます。似た語に「感覚（かんかく）」がありますが、意味も使い方も異なります。また、「間欠（かんけつ）」や「間隙（かんげき）」など、同じ「間」の漢字を使う語も多いので、正しい読みをしっかり覚えておきましょう。「間隔」は物や時間のあいだ、距離を表すときによく使われます。
    - topic: 女優
      result:
        question: 彼女は有名な **女優** です。 1 じょゆう 2 じょうゆ 3 にょゆう 4 じょうゆう
        correct_answer: 1
        explanation: 「女優」は「じょゆう」と読みます。N3レベルでは「女（じょ）」と「優（ゆう）」の音読みを組み合わせて読む熟語が多く出題されますが、「優」を「ゆう」と読むことに慣れていない学習者も多いです。また、「女」を「じょ」と読む熟語は「女性」「女神」など他にもあるので、音読みのパターンをしっかり覚えておく必要があります。他の選択肢は実際には使われない読み方なので、消去法も有効です。
        choices:
        - じょゆう
        - じょうゆ
        - にょゆう
        - じょうゆう
    - topic: 頼る
      result:
        question: 今の私は友人に **頼る** しかありません。  1 たよる  2 たのる  3 たるる  4 たゆる
        correct_answer: 1
        explanation: 「頼る」はN3レベルで頻出の動詞で、「他の人や物に助けを求める」「依存する」という意味です。読み方は「たよる」です。「たのる」と間違えやすいですが、「頼む（たのむ）」とは別の動詞なので注意が必要です。「たゆる」は「弛む」と書き意味も異なります。漢字の読みと意味の違いを正確に区別できるかがポイントです。
        choices:
        - たよる
        - たのる
        - たるる
        - たゆる
    - topic: 休憩
      result:
        question: 工場では、昼の**休憩**時間にみんなが食堂に集まります。
        choices:
        - きゅうけい
        - きゅうけ
        - きゅうけん
        - きゅうけいん
        correct_answer: 1
        explanation: 「休憩」はN3レベルの重要語彙で、正しい読みは「きゅうけい」です。選択肢2「きゅうけ」や3「きゅうけん」、4「きゅうけいん」は実際には使われない読み方ですが、漢字の訓読みや音読みの混同、また「憩」の字に慣れていない受験者が間違えやすいポイントです。特に「休」は「やすむ」とも読むため、訓読みと音読みの区別が問われます。
    - topic: 記事
      result:
        question: 次の漢字語の読みとして最もよいものを、1・2・3・4から一つ選びなさい。今朝の新聞に、サッカー日本代表についての記事が載っていました。
        choices:
        - きじ
        - きし
        - ぎじ
        - ぎし
        correct_answer: 1
        explanation: 「記事（きじ）」は、新聞や雑誌、ウェブサイトなどでニュースや出来事について書かれた文章を指します。JLPT N3レベルでは、読解問題や語彙問題で頻繁に登場する重要な単語です。「記」は「き」と読むことが多く、「記事」は「きじ」と読むのが正解です。似たような読み方の「議事（ぎじ）」や「技師（ぎし）」などと混同しやすいので注意しましょう。また、「記事」は実生活でもよく使われる表現であり、ニュースや出来事を伝える際に欠かせない語彙です。意味や使い方も合わせて覚えると、読解力や会話力の向上に役立ちます。
    - topic: 趣味について話す
      result:
        question: 友人と一緒に旅行へ行き、たくさんの写真を撮りました。次の中で「写真」の正しい読み方はどれですか？
        choices:
        - しゃしん
        - しんしゃ
        - しゃじん
        - しんじゃ
        correct_answer: 1
        explanation: 「写真」はJLPT N3公式出題範囲（新完全マスターN3漢字・N3語彙など）にも含まれており、日常会話や趣味の話題で頻出する重要語彙です。この問題では「旅行」「友人」「撮る」などの文脈から、自然に「写真＝しゃしん」と読めるかどうかを確認します。他の選択肢は実際の日本語では使われない読みや、意味が異なる語（例：「しんじゃ」は「信者」）を含め、注意深く選ぶ力を養います。また、写真撮影は日本でも人気の趣味であり、面接や自己紹介の話題にもよく登場します。普段から例文を音読したり、写真に関する語彙（撮影・写す・思い出など）もあわせて覚えると、語彙力がさらに強化されます。
  - subsection_title: 漢字の書き取り (write_chinese)
    description: 6問：ひらがなで書かれている言葉を漢字で書く問題。
    question_topics:
    - topic: 天気の状況について話す
      result:
        question: 今日は朝から空が（　）ので、気持ちがいいです。
        choices:
        - 晴れる
        - 清める
        - 静まる
        - 濡れる
        correct_answer: 1
        explanation: 「はれる」は天気がよくなる、雲がなくなって青空になることを表し、正しい漢字は「晴れる」です。2.「清める（きよめる）」は「きれいにする」という意味で、天気には使いません。3.「静まる（しずまる）」は「うるさいものがおさまる」時などに使う言葉です。4.「濡れる（ぬれる）」は「水や液体でぬれる」ことを表し、天気が良くなる意味ではありません。似た読みや意味に惑わされず、文脈と意味から正しい漢字を選ぶことが大切です。
    - topic: ぼうけん（冒険）
      result:
        question: 次の文の（　）に入る「ぼうけん」の正しい漢字表記を1～4から一つ選びなさい。子どもたちは森で（　）を始めることにしました。
        choices:
        - 冒険
        - 冒険者
        - 冒険心
        - 冒険談
        correct_answer: 1
        explanation: この問題のポイントは、「ぼうけん」という言葉が様々な熟語に使われることですが、文脈上「森で冒険を始める」は「冒険（adventure）」が最も適切です。2の「冒険者」は「冒険する人」、3の「冒険心」は「冒険したい気持ち」、4の「冒険談」は「冒険についての話」を意味します。選択肢の違いを理解し、正しい語彙を文脈に合わせて選ぶ力が求められます。
    - topic: きじ（記事）
      result:
        question: この きじ はとてもおもしろかったです。
        choices:
        - 生地
        - 記事
        - 木地
        - 基地
        correct_answer: 2
        explanation: 「きじ」には複数の漢字表記があり、意味によって使い分けられます。生地（きじ）は布地や素材、木地（きじ）は木の表面、基地（きち）は軍事や活動の拠点を指します。一方、新聞や雑誌で書かれた文章やニュースを意味する「きじ」は「記事」と書きます。N3レベルでは同音異義語の区別が重要なポイントであり、文脈から正しい漢字を選ぶ力が問われます。
    - topic: たよる（頼る）
      result:
        question: （　　　）に入る最も適切な漢字を、1～4の中から一つ選びなさい。困ったときは友人に（　　　）ことが多いです。
        choices:
        - 頼る
        - 頼む
        - 頼み
        - 頼れ
        correct_answer: 1
        explanation: この問題は、動詞「たよる」の正しい漢字表記とその活用形を問うものです。「頼る」は「他人や何かに助けを求めて依存する」という意味で、文脈に最も合っています。2の「頼む」は「お願いする」という意味で、文法的にも意味的にも合いません。3の「頼み」は名詞形（例：頼みがある）で、動詞としては使えません。4の「頼れ」は可能形（例：頼れる人）ですが、「ことが多い」という表現には使えません。N3学習者には、動詞の原形と活用形、意味の違いを正確に理解することが重要なポイントとなります。
    - topic: じょゆう（女優）
      result:
        question: 彼女は有名な じょゆう です。
        choices:
        - 女性
        - 女優
        - 女役
        - 女星
        correct_answer: 2
        explanation: この問題のポイントは、「じょゆう（女優）」という言葉の正しい漢字表記を選ぶことです。「女優」は「女性の俳優」という意味で、映画やドラマなどで演技をする女性を指します。1の「女性」は「じょせい」と読み、女性一般を指します。3の「女役」は「おんなやく」と読み、役の種類を表しますが、職業名としては使いません。4の「女星」は中国語圏でよく使われる表現で、日本語では一般的ではありません。N3レベルでは、似たような意味や漢字が並ぶことで混乱しやすいですが、「女優」という正しい熟語を覚えておくことが大切です。
    - topic: きょうちょう（協調）
      result:
        question: チームで仕事をするときは、みんなで きょうちょう することが大切です。
        choices:
        - 協定
        - 協調
        - 協力
        - 強調
        correct_answer: 2
        explanation: この問題の主な難しさは、似た意味や読み方を持つ漢字語が選択肢に並んでいる点です。「協調（きょうちょう）」は、意見や立場の異なる人が協力し合い、調和を図ることを意味します。1の「協定」は「きょうてい」と読み、取り決めや合意の意味です。3の「協力（きょうりょく）」は一緒に力を合わせる意味で、「協調」と似ていますが、文脈的に「協調」が最も適切です。4の「強調（きょうちょう）」は「力を入れて目立たせる」という意味で、読みは同じですが意味が異なります。このように、読みや意味が似ている語の区別がN3学習者にとってのポイントです。
  - subsection_title: 語彙の意味選択 (word_meaning)
    description: 11問：与えられた単語の意味に最も近いものを選ぶ。
    question_topics:
    - topic: 協調
      result:
        question: 新しいプロジェクトを始めるには、みんなの「きょうちょう」が必要です。
        choices:
        - 協調
        - 協調性
        - 協定
        - 協力
        correct_answer: 1
        explanation: 「きょうちょう」は協調と書き、意見や立場の違う人たちが調和をはかりながら協力することを意味します。選択肢2「協調性」は、性格や能力を表す語で、この文脈では不適切です。選択肢3「協定」は、国や団体などが結ぶ約束事です。選択肢4「協力」は、単に力を合わせることですが、調和というニュアンスが足りません。N3レベルでは「協調」と「協力」の違いが特に混同されやすいので注意しましょう。
    - topic: 睡眠
      result:
        question: 最近、夜遅くまで仕事をしていたので、すいみんが不足しています。
        choices:
        - 睡眠
        - 水面
        - 推進
        - 随分
        correct_answer: 1
        explanation: この問題のポイントは、「すいみん」という読みの正しい漢字表記を選ぶことです。・「睡眠（すいみん）」は「ねむること、ねむり」の意味で、N3レベルの重要語彙です。・「水面（すいめん）」は「みずのうえ」という意味で、読みは似ていますが意味が異なります。・「推進（すいしん）」は「物事をすすめること」という意味で、漢字や音が似ているため混同しやすいですが、意味が異なります。・「随分（ずいぶん）」は「かなり、非常に」という意味で、読みが似ているためのひっかけです。このように、選択肢には実際に使われる熟語を入れ、受験者が漢字の意味・読み・使い方を総合的に判断できるようにしています。JLPT
          N3レベルでは、似た音や漢字の語彙を区別する力が求められます。
    - topic: 子供の教育について話す
      result:
        question: 子供の教育について話すとき、「きょういく」という言葉がよく使われます。次の文の（　）に入る正しい漢字を選びなさい。最近、親同士で子供の（きょういく）について話す機会が増えました。
        choices:
        - 教育
        - 教域
        - 教欲
        - 教浴
        correct_answer: 1
        explanation: 「きょういく」の正しい漢字は「教育」です。N3レベルではよく出る重要な語彙ですが、似た音の漢字（域＝いき、欲＝よく、浴＝あびる）と混同しやすいため、正確に覚える必要があります。特に「育」は「育てる」と関連づけて覚えると良いでしょう。
    - topic: 体温
      result:
        question: 風邪をひいたので、今朝は（たいおん）をはかりました。
        choices:
        - 体音
        - 体温
        - 耐温
        - 体恩
        correct_answer: 2
        explanation: この問題の難しさは、「たいおん」という読みの漢字表記を正しく選ぶことにあります。「体音」や「耐温」など、見た目が似ている漢字を使った選択肢が紛らわしいですが、「たいおん（体温）」は「体（からだ）」と「温（あたたかい）」の組み合わせで、「身体の温度」という意味になります。他の選択肢は意味や読みが異なるので注意が必要です。N3レベルでは、日常生活や健康に関する基本的な語彙として「体温」は押さえておきたい単語です。
    - topic: 販売
      result:
        question: 新しいスマートフォンの（はんばい）が、全国の家電量販店で始まりました。
        choices:
        - 販売
        - 発売
        - 販売
        - 飯売
        correct_answer: 1
        explanation: 「はんばい」は「販売」と書き、商品やサービスを売ることを意味します。N3レベルでは「販売」と「発売」の違いも問われやすいです。2の「発売」は「売り出しを始めること」で、「販売」は「実際に売る活動全般」を指します。3の「販売」は「販」の後ろに「賣（旧字体）」を使っており、現代日本語では使いません。4の「飯売」は意味が異なります（「飯」は「ご飯」）。特に「発売」と「販売」は混同しやすいので、文脈から「売り始める」ではなく「売ること全般」を指す「販売」が正解となります。
    - topic: 様子
      result:
        question: 最近、田中さんのようすが少し変だと感じます。
        choices:
        - 様子
        - 洋子
        - 様式
        - 状子
        correct_answer: 1
        explanation: 「ようす」は「様子」と書きます。「様子」は人や物事の状態・ありさまを表すN3レベルの重要語彙です。選択肢2「洋子」は人名でよく使われますが、意味が異なります。3「様式」は「スタイル」「形式」を表し、「ようしき」と読みます。4「状子」は実際には使われない組み合わせで、誤答としてよく出されるパターンです。この問題の主な難しさは、「様」「子」「式」「状」など、形や意味が似ている漢字を区別できるかを問う点です。漢字の正しい組み合わせを覚えているかどうかが重要です。
    - topic: 改善
      result:
        question: 最近、会社では仕事の流れをもっとよくするために（　　　）活動が行われている。
        choices:
        - 改善
        - 改選
        - 改造
        - 改築
        correct_answer: 1
        explanation: 「改善（かいぜん）」は「悪いところを直して、より良くすること」という意味で、ビジネスシーンや日常生活でもよく使われる言葉です。例：「生活を改善する」「業務改善」。他の選択肢はすべて「改」の字を含みますが、意味が異なります。-
          「改選（かいせん）」は、役員などを選び直すこと。- 「改造（かいぞう）」は、物や建物などを作り変えること。- 「改築（かいちく）」は、建物を建て直すこと。読みや漢字が似ているため混同しやすいですが、「善」は「よい、よくする」という意味を持つので、「改善」で「より良くする」という意味になります。使用例：・生活習慣を改善したいです。・会社の業務を改善するために会議を開きました。教師の視点からのポイント：N3レベルでは、同じ「改」を使った熟語の意味や使い分けが課題となります。選択肢の漢字はどれもよく使われるため、正しい意味と使い方をしっかり覚えることが大切です。
    - topic: 材料
      result:
        question: 次の文の（　）に入る正しい漢字を、1～4の中から一つ選びなさい。新しい機械を作るために、いろいろな（　）を集めています。
        choices:
        - 材料
        - 材質
        - 素材
        - 原料
        correct_answer: 1
        explanation: 「材料（ざいりょう）」は、物を作るためのもとになるものを表します。例えば、家具を作る時の木や金属、料理を作る時の肉や野菜などが「材料」です。-
          「材質（ざいしつ）」は、そのものの性質や品質を指します。例：「この机の材質は木です。」- 「素材（そざい）」は、加工されていない自然のままのものを指します。例：「この服の素材はコットンです。」-
          「原料（げんりょう）」は、主に工業や化学、食品などで最初に使うもとの物質を指します。例：「チョコレートの原料はカカオ豆です。」このように、似ている言葉ですが使い分けが必要です。文脈によって最もふさわしい漢字を選びましょう。ポイントの強調例：-
          材料：物を作るためのもとになるもの（正解）- 材質：ものの性質・品質- 素材：加工前のもとのもの- 原料：製品のもとになる物質（主に工業・食品など）このように、指示文の追加や解説の視覚的な強調を加えることで、受験者がより理解しやすくなります。今後もこの形式を参考に、複数の語彙で問題を作成することをおすすめします。
    - topic: 議会
      result:
        question: 新しい法律を決めるために、国の（ぎかい）が開かれました。
        choices:
        - 議会
        - 義会
        - 宜会
        - 儀会
        correct_answer: 1
        explanation: この問題のポイントは「ぎかい」という読みの正しい漢字表記を選ぶことです。N3レベルの学習者にとって、「議」（議論・議事などでよく出る漢字）と「会」（会社・会議などでよく出る漢字）の組み合わせで「議会」となります。他の選択肢は音や意味が似ている漢字を使っていますが、意味が異なります。「義」は「正義」など、「宜」は「適宜」など、「儀」は「儀式」などで使われるため、混同しやすいですが、正しい組み合わせを選ぶ必要があります。
    - topic: 天気の状況について話す
      result:
        question: 次の文章の（　）に入る正しい漢字を、1・2・3・4の中から一つ選びなさい。昨日は一日中「くもり」だったので、洗濯物がなかなか乾きませんでした。（　）
        choices:
        - 曇
        - 雲
        - 霧
        - 雨
        correct_answer: 1
        explanation: 「くもり」は天気の状態を表す言葉で、正しい漢字は「曇」です。選択肢2の「雲」は空に浮かぶ「くも」、3の「霧」は「きり」、4の「雨」は「あめ」と、それぞれ異なる天気や現象を表します。JLPT
          N3レベルでは、「くもり（曇）」と「くも（雲）」の区別が混同しやすいので注意が必要です。また、例文の「乾きませんでした」のように、天気と日常生活（洗濯物が乾かないこと）を結びつける表現は、日本語の実用的な運用力を測る良い出題となります。教師からのポイント：この問題の主な難しさは、「曇」と「雲」の漢字の意味の違いを理解し、文脈から適切な語を選ぶ点です。天気に関する表現は会話やニュースなどでよく使われるため、正しい漢字表記を身につけておくことが大切です。
    - topic: 流れ
      result:
        question: 次の文の（　）に入る正しい漢字を選びなさい。川の（ながれ）が速くて、泳ぐのは危険です。
        choices:
        - 流木（りゅうぼく）
        - 流星（りゅうせい）
        - 流れ（ながれ）
        - 流行（りゅうこう）
        correct_answer: 3
        explanation: この問題は「ながれ」の正しい漢字表記を問うものです。選択肢にはすべて「流」の漢字が含まれており、どれも実在する日本語の単語です。しかし、「川の流れが速い」という文脈に合うのは「流れ」だけです。他の選択肢は意味が異なります（流木：川などに流れている木、流星：流れ星、流行：はやり）。N3レベルでは、意味と漢字表記の正しい組み合わせを理解しているかが問われます。似ている言葉が並んでいるため、意味の違いをしっかり区別できるかがポイントです。
  - subsection_title: 同意語置き換え (synonym_substitution)
    description: 5問：文中の語を同じ意味の語に置き換える問題。
    question_topics:
    - topic: 意外
      result:
        question: 昨日のニュースで、彼が会社を辞めると聞いて、このニュースはとても意外（いがい）でした。
        choices:
        - 普通（ふつう）
        - 予想外（よそうがい）
        - 安全（あんぜん）
        - 重要（じゅうよう）
        correct_answer: 2
        explanation: ・「意外（いがい）」は、「思っていたことと違っていること」や「予期しなかったこと」という意味です。・「予想外（よそうがい）」は、「予想していなかったこと」「思いがけないこと」という意味で、「意外」と最も近い意味を持っています。・他の選択肢について：　-
          「普通（ふつう）」は「一般的」「特別ではない」という意味です。　- 「安全（あんぜん）」は「危険がないこと」を表します。　- 「重要（じゅうよう）」は「大切なこと」を意味します。・この問題のポイントは、「意外」と「予想外」の意味の近さを理解し、文脈から正しい語を選べるかどうかです。・「意外」と「予想外」はどちらも「unexpected」という意味ですが、日常会話では「意外」の方が使われやすいです。・選択肢の意味をしっかり区別し、文全体の意味を正確に理解する力が求められます。
    - topic: 占める
      result:
        question: この会社の商品は、国内市場の50％を占める。適切な意味の言葉を選びなさい。
        choices:
        - 占有する
        - 示す
        - 埋める
        - 支配する
        correct_answer: 1
        explanation: 「占める」は「ある範囲や割合、地位などを自分のものとする」「全体の中で一定の割合を持つ」という意味です。選択肢1の「占有する」は「場所や地位などを自分のものとして持つ」という意味で、「占める」と最も近い意味になります。選択肢2の「示す」は「見せる、表す」という意味で、「占める」とは異なります。選択肢3の「埋める」は「空いているところをふさぐ」という意味で、割合や地位を取る意味では使いません。選択肢4の「支配する」は「コントロールする、管理する」という意味で、直接的に割合や位置を取る意味ではありません。この問題の難しさは、「占める」の持つ「割合を取る」という意味を正しく理解し、それに最も近い語を選ぶ必要がある点です。選択肢の中には意味が似ている語もあるため、細かい意味の違いを理解しているかどうかが問われます。
    - topic: 問い
      result:
        question: 次の文の「問い」と最も近い意味の言葉を選びなさい。先生は授業の終わりに新しい問いを出しました。
        choices:
        - 問題
        - 答え
        - 説明
        - 注意
        correct_answer: 1
        explanation: 「問い」は「質問」「設問」「問題」などと近い意味を持ちますが、文脈によっては「質問」とも似ています。しかし、授業で「出す」のは「問題」や「設問」であることが多く、答えや説明、注意とは意味が異なります。N3レベルでは「問い」と「問題」の使い分けが難しく、特に「質問」との違いに迷う受験生もいるため、設問の意図を読み取る力が問われます。
    - topic: 泊める
      result:
        question: 次の文の意味に最も近いものを、1～4の中から一つ選びなさい。友人が遠くから来たので、私の家に泊めることにした。
        choices:
        - 受け入れる
        - 止める
        - 集める
        - 送る
        correct_answer: 1
        explanation: 「泊める」は「他人を自分の家などに宿泊させる」という意味です。選択肢1の「受け入れる」は、人を迎えて家の中に入れてあげる、世話をするという意味があり、「泊める」と近い意味になります。一方、「止める」は動いているものを止める、「集める」はたくさんのものを一か所に集める、「送る」はどこかへ行かせる、という意味であり、文脈に合いません。また、「とめる」と読む漢字には「止める（動きをやめる）」「留める（位置を保つ・残す）」「停める（車などを止める）」などがあり、意味が異なります。「泊める」は「宿泊させる」という意味で使われるので、漢字の違いにも注意しましょう。この問題のポイントは、似た動詞の意味の違いをしっかり理解しているかどうかです。誤答選択肢も日常でよく使う動詞なので、文脈から最も適切な語を選ぶ力が求められます。
    - topic: 海外旅行の体験を話す
      result:
        question: 海外旅行の体験を話すときは、できるだけ具体的に説明すると、聞いている人に伝わりやすくなります。この文の「説明する」と意味が近い言葉を選びなさい。
        choices:
        - 伝える
        - 遊ぶ
        - 笑う
        - 集める
        correct_answer: 1
        explanation: 「説明する」は「物事を分かりやすく話す、伝える」という意味です。選択肢2「遊ぶ」、3「笑う」、4「集める」は意味が異なります。1「伝える」が最も近い意味を持つ同義語です。N3レベルでは「説明」と「伝える」の違いを文脈で判断する力が求められます。「説明」は詳細に話すこと、「伝える」は情報や気持ちを相手に届けることですが、ここでは「体験を話す」＝「伝える」と自然に置き換えられるため、紛らわしさがあります。distractor（誤答肢）も日常的な動詞なので注意が必要です。
  - subsection_title: 語彙の使い方 (word_usage)
    description: 5問：語彙の使い方が正しい文を選ぶ。
    question_topics:
    - topic: 手術
      result:
        question: 次の文の中で、「手術」の使い方として最も適切なものを一つ選びなさい。
        choices:
        - 祖母は病気が進んでしまったので、来週病院で手術を受けることになりました。
        - 彼は新しいパソコンを買ったので、古いパソコンの手術をしてデータを移しました。
        - 昨日、料理中に手を切ってしまい、少し手術をしました。
        - 旅行の前に部屋の手術をして、きれいに片付けました。
        correct_answer: 0
        explanation: 「手術」とは、医師がメスや特別な道具を使って、病気やけがを治すために体を切ったり治療したりする医療行為です。Aは医療の場面で正しい使い方です。BやDのように、パソコンや部屋など人や体以外のものには使いません。Cはけがの手当てをした場合、「手術」ではなく「消毒」や「ばんそうこうを貼った」などと言います。この問題のポイントは、「手術」が医療の専門的な行為であり、日常の作業や簡単なけがの手当てには使わないことを理解できるかどうかです。N3レベルの学習者にとって、語彙の意味と使い方の正確な区別が重要です。
    - topic: 哀れ
      result:
        question: 次の文の中で、「哀れ」の使い方として最も適切なものを一つ選びなさい。
        choices:
        - この映画の主人公は、家族を失って一人で生きる哀れな少年です。
        - 長い間手入れされず、哀れな景色が広がっていた。
        - 昨日の天気は哀れで、みんな外に出られませんでした。
        - この料理は哀れな味がして、あまりおいしくありません。
        correct_answer: 1
        explanation: 「哀れ」は「かわいそう」「気の毒」といった意味で、主に人や動物など、感情や同情の対象となるものに使います。1番の文は、家族を失った少年について「哀れな」と表現しており、正しい使い方です。2番の「哀れな景色」は、景色に対して「哀れ」を使うのはやや不自然で、通常は「寂しい」「荒れた」などを使います。3番の「天気が哀れ」、4番の「哀れな味」も誤用です。天気や味など、物や抽象的なものには「哀れ」は使いません。類義語には「かわいそう」「気の毒」「不憫」などがあり、対義語には「立派」「堂々」などがあります。「哀れ」は人や動物など、同情や悲しみを感じる対象に使うことを意識しましょう。
    - topic: 動詞
      result:
        question: 次の文の（　　）に入る最も適切な動詞を1つ選びなさい。Aさんは毎朝6時に（　　　）、コーヒーを飲んでから会社に行きます。
        choices:
        - 起きます
        - 泳ぎます
        - 住みます
        - 使います
        correct_answer: 1
        explanation: この問題は、日常生活の流れと動詞の意味・使い方を理解しているかどうかを問うものです。「毎朝6時に（　　　）、コーヒーを飲んでから会社に行きます。」という文脈では、朝の行動として最も自然なのは「起きます（おきます）」＝「to
          get up」です。他の選択肢「泳ぎます（およぎます）」＝「to swim」、「住みます（すみます）」＝「to live」、「使います（つかいます）」＝「to
          use」は、この文脈では不自然です。動詞の基本的な意味と日常的な使い方、そして文脈に合うかどうかを判断する力が問われる点が学習者にとっての主なチャレンジです。
    - topic: 好み
      result:
        question: 次の文の中から、最も自然で適切な文を一つ選びなさい。
        choices:
        - 彼は私の好みのタイプです。
        - この店は、好みの服がたくさん売っています。
        - 好みを言えば、もう少し甘いほうが好きです。
        - 好みの問題だから、どちらを選んでもいいですよ。
        correct_answer: 2
        explanation: 「好み」は「好きなものの傾向」や「自分が良いと思うもの」という意味で使われます。特に「好みの＋名詞」の形（例：好みの服）は、N3レベルでよく出題される典型的なコロケーションです。2番の文は「自分の好きなタイプの服」という意味になり、最も自然な使い方です。他の選択肢について、1番も自然な表現ですが、2番の方が「好みの＋名詞」としてより基本的です。3番の「好みを言えば」は「もし自分の好みを伝えるなら」という意味で使われ、やや抽象的な表現です。4番の「好みの問題」は「個人の好き嫌いによる問題」という慣用的表現ですが、2番と比べると、N3レベルの基本的な使い方としてはやや難易度が高いです。参考例文：「好みの問題と言われればそれまでですが、私はこのレストランの内装があまり好きではありません。」「好みを言えば、もう少し辛いほうがいいです。」主なポイント・難しさ：「好み」には複数の意味や使い方があり、文脈によって使い分ける必要があります。「好みの＋名詞」や「好みを言えば」など、決まったコロケーション・表現があるため、正しく使えるかどうかがポイントです。
    - topic: キャリア目標について話す
      result:
        question: '次の語句の使い方として最も適切な文を一つ選びなさい。内容: 1. 修理のため、エアコンの内容を一度取り出します。 2. 鍋の中にカレーの内容を入れて、1時間くらい煮てください。
          3. 古い財布から新しい財布へ内容を移しました。 4. この手紙の内容は、ほかの人には秘密にしてください。'
        choices:
        - 修理のため、エアコンの内容を一度取り出します。
        - 鍋の中にカレーの内容を入れて、1時間くらい煮てください。
        - 古い財布から新しい財布へ内容を移しました。
        - この手紙の内容は、ほかの人には秘密にしてください。
        correct_answer: 4
        explanation: この問題は、語彙の意味だけでなく、自然な使い方（コロケーション）を問うものです。 「内容」は「手紙の内容」など、情報や中身を表すときに使います。
  - subsection_title: 文法穴埋め (sentence_grammar)
    description: 13問：文中の空欄に当てはまる文法表現を選ぶ。
    question_topics:
    - topic: 年齢
      result:
        question: （自己紹介の場面）A：「はじめまして。田中と申します。Bさんはおいくつですか。」B：「私は今年、二十歳（　　　）なります。」
        choices:
        - が
        - を
        - に
        - で
        correct_answer: 3
        explanation: 「〜に なる」は、年齢や状態が変化するときに使われる表現です。たとえば、「学生になります」「先生になります」「二十歳になります」などのように、変化の到達点を表すときは「に」を使います。他の助詞「が」「を」「で」はこの文脈では使えません。N3レベルの学習者は母語の影響で誤った助詞を選びやすいため、特に注意が必要です。
    - topic: 高まる
      result:
        question: 次の文の（　　　）に入る最も適切なものを、1～4の中から一つ選びなさい。最近、世界的に環境問題への関心が（　　　）、多くの国が協力して対策を進めています。
        choices:
        - 高まっている
        - 高めている
        - 高められている
        - 高まらせている
        correct_answer: 1
        explanation: 「関心が高まる」は「関心の程度が強くなる」という意味で、自動詞「高まる」を使います。「高まっている」は、その状態が続いていることを表します。他の選択肢は他動詞や受け身・使役形であり、この文には合いません。例文：環境問題に対する関心は年々高まっています。
    - topic: 仲間
      result:
        question: （サークル活動にて）A：「来月のイベントの準備、大変だね。」B：「うん。でも、（　）と一緒なら頑張れる気がするよ。」
        choices:
        - 仲間
        - 友達
        - 家族
        - 先生
        correct_answer: 1
        explanation: この問題のポイントは、「一緒に何かをする人」や「共通の目的に向かって協力する人」を指す最も適切な語彙を選ぶことです。「仲間」はN3レベルでよく使われる語彙であり、サークル活動やイベントの準備など、共通の目的を持つ集団にぴったりの言葉です。「友達」も親しい人を指しますが、目的を共有して協力するニュアンスが弱くなります。「家族」「先生」は文脈に合いません。学習者は「仲間」と「友達」の違いを文脈で判断するのが難しい場合が多いので、意味の違いに注意しましょう。
    - topic: 冷ます
      result:
        question: （食堂にて）A：「このスープ、まだ熱いね。」B：「うん、少し（　　　　）から飲もう。」※（　　　　）に入る最も適切な動詞のて形を答えなさい。
        correct_answer: 0
        choices:
        - 冷まして
        explanation: 「冷ます」は他動詞で、「熱いものの温度を下げる」という意味です。この場面では、スープを飲む前に「冷まして」から飲む、という流れになります。「冷める」は自動詞で、自然に温度が下がる場合に使いますが、ここでは人が意図的に行う動作なので「冷まして」が正解です。N3レベルでは自動詞と他動詞の区別がよく問われるため、注意が必要です。
    - topic: 節約
      result:
        question: （友人との会話） A：「最近、生活費が高くなってきたね。何か（　）方法を考えている？」 B：「うん、家計簿をつけたり、外食を減らしたりしているよ。」
          （　）に入る最も適切な語句を選びなさい。
        explanation: 「方法」という名詞の前には、その方法がどのようなものかを説明する修飾語が入ります。「節約する方法」も文法的には正しいですが、一般的には「節約できる方法」と言うことで、「節約することができる方法」、つまり“可能性”や“実現性”を強調する自然な表現になります。「節約する方法」は、単に「節約するための方法」という意味ですが、「節約できる方法」は「実際に自分が節約することが可能な方法」というニュアンスを持ち、より実用的で会話でもよく使われます。このように、「～できる＋名詞」はJLPT
          N3レベルでよく問われる文法ポイントで、可能性や能力を表現する際に使われます。【追加例文】・最近は、簡単に作れる料理のレシピや、安く買い物ができるアプリなど、いろいろな（　）サービスが増えています。→　節約できる・新しいアパートに引っ越したいけど、家賃を（　）場所を探しています。→　節約できる
        correct_answer: 4
        choices:
        - 節約する
        - 節約した
        - 節約している
        - 節約できる
    - topic: 身長
      result:
        question: ご丁寧なフィードバックをいただき、ありがとうございます。内容の正確性・問題の質・形式・文化的適切性など、多角的にご評価いただき感謝いたします。ご指摘いただいた改善点をふまえ、より良い問題例を以下にご提案いたします。---【健康診断の場面での会話】先生：「はい、次の方、こちらに来てください。まず、（　　　）を測りますので、靴を脱いでください。体重は後で測ります。」1.
          体重 2. 年齢 3. 身長 4. 気分---【正解】3. 身長【解説】- 健康診断の場面で「まず、（　　　）を測ります」という指示は、身長測定が一般的です。-
          「体重は後で測ります」と加えることで、「体重」と「身長」の語彙の使い分けがより明確になります。- 「身長」「体重」「年齢」「気分」はN3レベルの基本語彙です。-
          「年齢」「気分」は測定の対象として不自然であるため、文脈理解力が問われます。---ご指摘いただいた点を反映し、今後もより分かりやすく、実用的な問題作成を心がけます。貴重なご意見、ありがとうございました。
        choices:
        - 体重
        - 年齢
        - 身長
        - 気分
        correct_answer: 3
        explanation: 健康診断の場面で「まず、（　　　）を測ります」という指示は、身長測定が一般的です。「体重は後で測ります」と加えることで、「体重」と「身長」の語彙の使い分けがより明確になります。「身長」「体重」「年齢」「気分」はN3レベルの基本語彙です。「年齢」「気分」は測定の対象として不自然であるため、文脈理解力が問われます。
    - topic: 適切
      result:
        question: この問題に最も（　　　）な答えを選びなさい。
        choices:
        - 適切
        - 適当
        - 適性
        - 適格
        correct_answer: 1
        explanation: この文は「この問題に最も（　　　）な答えを選びなさい。」という指示文です。「適切」は、状況や目的にぴったり合っている、ふさわしいという意味で使われます。「適当」も似た意味ですが、「いいかげん」という意味もあり、試験や正式な場面では「適切」の方が正確で望ましい表現となります。「適性」は「向いている性質」、「適格」は「資格・条件に合っている」という意味なので文脈に合いません。教師からみた主な難しさ：「適切」と「適当」はN3レベルでも混同しやすい語彙です。どちらも「ふさわしい」という意味があるため、文脈によって正しい語を選ぶ必要があります。特に「適切」はフォーマルな場面や試験など、正確さが求められる状況で使われることが多いので、その使い分けを理解しているかがポイントとなります。
    - topic: 契機
      result:
        question: 部長：「今回の新しいシステム導入を（　）、業務の効率化を図りたいと考えています。」
        choices:
        - 契機に
        - 契機で
        - 契機を
        - 契機から
        correct_answer: 1
        explanation: この文では「～を契機に」という表現を使います。「契機に」は「何かをきっかけとして」「何かを機会として」という意味で、物事の変化や新しいことを始める直接的な原因やきっかけを表します。他の選択肢は文法的に自然ではありません。「契機に」はN3レベルでもよく使われる表現ですが、漢語的でややフォーマルな響きがあるため、「きっかけに」との使い分けが難しい点が主な課題です。特に「契機で」「契機を」「契機から」など、助詞の誤用が多く見られるため、正しいコロケーションを覚えることが重要です。
    - topic: 技術について話す
      result:
        question: 次の会話を読んで、（　）に入る最も適切な語を選びなさい。（会議室にて）A：「今回のプロジェクトでは、新しい技術（　）話す必要があります。」B：「はい、分かりました。分かりやすく説明します。」
        choices:
        - を
        - について
        - で
        - が
        correct_answer: 2
        explanation: 「〜について話す」は「〜の内容を話す、〜に関して話す」という意味で使われるN3レベルの重要表現です。「技術を話す」は日本語では不自然な表現になります。この問題の主な難しさは、「を話す」と「について話す」の違いを理解することです。「について」を使うことで、「新しい技術に関して話す」という正しい意味になります。N3レベルの学習者にとって、「について」の使い方は頻出で大切なポイントですので、しっかり身につけておきましょう。
    - topic: 正午
      result:
        question: 次の会話を読んで、空欄に入る最も適切な語句を1つ選びなさい。（昼の時間帯、時計を見ながら）A：「今、何時ですか。」B：「もうすぐ（　　　）ですよ。」
        choices:
        - 午前
        - 午後
        - 正午
        - 深夜
        correct_answer: 3
        explanation: この問題は、日常会話でよく使われる時間に関する語彙の理解を問うものです。「もうすぐ（　　　）ですよ。」という発言は、昼の12時が近い状況を想定しています。「正午」は「昼の12時」を意味し、時間帯を表す「午前（夜中～昼前）」「午後（昼～夜）」「深夜（夜中）」とは異なります。N3レベルでは「正午」と「午後」の区別が難しいことが多いですが、「正午」は特定の時刻（12時ちょうど）を指し、「午後」は12時以降の時間帯全体を指します。したがって、ここでは「正午」が最も適切です。主なチャレンジポイントは、「正午」と「午後」「午前」の意味の違いを正確に理解できているかどうか、会話の状況（昼の時間帯、12時が近いこと）を文脈から推測できるかどうかです。
    - topic: 信用
      result:
        question: （会社の会議にて）部長：「新しい取引先と契約する前に、その会社の（　）をしっかり調べておく必要があります。」（　）に入る最も適切な語を1〜4の中から選びなさい。
        choices:
        - 信用
        - 信号
        - 信号機
        - 信念
        correct_answer: 1
        explanation: この文では、「その会社の（　）をしっかり調べておく必要があります。」とあります。「信用（しんよう）」は、「他人や会社などを信じて受け入れること、または信頼できると判断すること」という意味です。取引や契約の前に、相手の信頼性や過去の実績などを調べる場合によく使います。「信念（しんねん）」は「自分が正しいと信じている考えや価値観」のことで、会社の信頼性を調べる文脈では使いません。「信号（しんごう）」や「信号機（しんごうき）」は、どちらも交通に関する言葉であり、この文脈には合いません。N3レベルでは、「信用」と「信念」など、意味が異なるが漢字や音が似ている語彙の区別が難しいです。また、「信号」など全く違う意味の語も選択肢にあるため、文脈をしっかり読む力が必要です。「信用」はビジネスや日常生活でもよく使う重要語彙なので、使い方を正確に理解しましょう。
    - topic: 個人的な成果について話す
      result:
        question: （会社の面談にて）上司：「最近のプロジェクトで、何か個人的に（　）ことはありましたか。」
        choices:
        - 成し遂げた
        - 成し遂げる
        - 成し遂げて
        - 成し遂げます
        correct_answer: 1
        explanation: この文は、過去の経験や成果について尋ねる表現です。「成し遂げたことはありましたか」は「過去に何かをやり遂げた経験がありましたか」という意味になります。N3レベルでは「〜たことがある」「〜たことはありますか」など経験を表す文型がよく出題されますが、「〜たことはありましたか」はやや丁寧な言い方で、過去の特定の期間や状況に焦点を当てる場合に使われます。「成し遂げた（過去形）」以外の活用形（辞書形、て形、ます形）はこの文脈では不自然です。主なチャレンジポイントは、「〜たことはありましたか」と「〜たことがありますか」のニュアンスの違い（前者は過去のある期間や状況に焦点、後者は一般的な経験）、適切な動詞の活用（過去形）が使えるかどうか、ビジネスシーンでよく使われる語彙や表現に慣れているか、です。
    - topic: 音楽について話す
      result:
        question: （　）に入る適切な言葉を選びなさい。友人との会話 A：「最近、どんな音楽をよく（　）いますか。」 B：「最近はジャズをよく聴いています。」
        choices:
        - 聞いて
        - 聞き
        - 聞いてる
        - 聞かれて
        correct_answer: 1
        explanation: この文では「～ていますか」という現在進行形・現在習慣の形を作る必要があります。「聞いていますか」が正しい形です。選択肢2「聞き」は未然形で文法的に不正確です。選択肢3「聞いてる」は話し言葉の縮約形で、日常会話ではよく使われますが、書き言葉や正式な場面では「聞いています」が望ましいです。選択肢4「聞かれて」は受け身形で意味が変わってしまいます。N3レベルでは「～ています」の使い方、動詞のて形＋いますの活用に注意しましょう。
  - subsection_title: 文の並び替え (sentence_sort)
    description: 5問：バラバラになった文を正しい順序に並び替える。
    question_topics:
    - topic: 異なる
      result:
        question: 次の文の__★_ に入る最も適切な語順を、下の1～4の中から一つ選びなさい。文化が__★_ 生活習慣も違います。1.　国によって
          2.　異なるため 3.　人々の 4.　それぞれ
        choices:
        - 4 → 1 → 2 → 3
        - 1 → 4 → 2 → 3
        - 1 → 2 → 4 → 3
        - 4 → 2 → 1 → 3
        correct_answer: 3
        explanation: この問題の主な難しさは、「異なるため」の使い方と、その前後に来る語句の関係を正しく理解することです。「国によって異なるため、それぞれ人々の生活習慣も違います。」という流れが自然ですが、「それぞれ」と「人々の」の位置関係も間違えやすいポイントです。「異なる」はN3レベルでよく出る語で、理由を表す「ため」と接続する形が頻出です。語順を間違えると意味が通じなくなるため、正しい語順を選ぶ力が問われます。
    - topic: 大会
      result:
        question: 山川大学では、__★__ 新入生にアンケート調査を行っている。次の語句を正しい順序に並べ替えなさい。1. 大学生活 2. に対して
          3. 持っている 4. イメージ
        choices:
        - 1→2→3→4
        - 1→4→2→3
        - 2→1→3→4
        - 1→3→2→4
        correct_answer: 0
        explanation: この並べ替え問題の主なポイントは、「に対して」の接続先、「持っている」の修飾関係、「イメージ」の位置です。「大学生活に対して持っているイメージ」で一つの名詞句を作り、それを調査する、という流れになります。N3レベルでは「に対して」の使い方や、名詞修飾節の語順が難しく感じられることが多いので、正しい語順を身につける良い練習になります。また、「新入生にアンケート調査を行っている」という表現で、調査の対象が明確になっています。
    - topic: 要素
      result:
        question: 次の文の__★_ に入る最も適切な語順を、下の1〜4の中から一つ選びなさい。プロジェクトを成功させるためには、いくつかの__★_
          が必要です。1. 要素 2. 重要な 3. が 4. あります 1. 2 → 1 → 3 → 4 2. 1 → 2 → 3 → 4 3. 2 →
          1 → 4 → 3 4. 1 → 3 → 2 → 4
        explanation: この問題では、「重要な要素がある（あります）」という自然な日本語の語順を作る力が問われます。「重要な」は「要素」を修飾し、「重要な要素」で「大切な部分・成分」という意味の名詞句になります。その名詞句が「が」によって主語となり、「あります（存在する）」で文が完結します。N3レベルの学習者にとって、「形容詞＋名詞」の修飾関係と、「が」を使った主語の作り方、そして「あります」の使い方を総合的に理解することがポイントです。また、「要素」は「物事を成り立たせる基本的な部分」という意味で、プロジェクトや計画の説明によく使われます。
        correct_answer: 1
        choices:
        - 2 → 1 → 3 → 4
        - 1 → 2 → 3 → 4
        - 2 → 1 → 4 → 3
        - 1 → 3 → 2 → 4
    - topic: 捜査
      result:
        question: 次の文の（　★　）に入るように、下の1～4の文を正しい順番に並べたものを一つ選びなさい。警察は、事件の真相を明らかにするために__★__行った。1.
          証拠を 2. 集める 3. 捜査を 4. ために
        choices:
        - 3 → 1 → 2 → 4
        - 1 → 2 → 3 → 4
        - 3 → 2 → 1 → 4
        - 1 → 3 → 2 → 4
        correct_answer: 1
        explanation: この問題の難しさは、動詞「捜査を行う」の目的語となる「証拠を集めるために」という副詞句を正しく並べることです。まず「捜査を」が来て、その目的「証拠を集める」が続き、最後に「ために」で理由や目的を表現します。N3レベルの語彙と文法（目的の「ために」）を理解しているかが問われます。
    - topic: 問う
      result:
        question: 山川大学では、__★_ アンケート調査を行っている。次の語句を正しい順番に並べ替えて、自然な文を完成させなさい。 【並べ替え語句】
          1. 大学生活 2. についての 3. イメージ 4. 新入生が
        choices:
        - 4 → 1 → 2 → 3
        - 4 → 3 → 1 → 2
        - 4 → 2 → 1 → 3
        - 4 → 3 → 2 → 1
        correct_answer: 1
        explanation: この問題の主なポイントは、「についての」という連語と「イメージ」のつながりを正しく理解し、「大学生活についてのイメージ」という自然な名詞句を作ることです。JLPT
          N3レベルでは、「についての」の使い方や、名詞修飾の順序（どの名詞がどの名詞を修飾するか）がよく問われます。また、「新入生が」という主語を先頭に置き、「大学生活についてのイメージ」というまとまりを作った上で、「アンケート調査を行う」という動詞句につなげる必要があります。語句の区切りや助詞の扱いを意識しないと、「イメージ大学生活について」など不自然な語順になりやすいので、文全体の意味のつながりを考えながら並べ替えることが大切です。
  - subsection_title: 文法構造選択 (sentence_structure)
    description: 5問：文法的に正しい文を選ぶ。
    question_topics:
    - topic: 大戦
      result:
        html_article: '# 大戦について 20世紀には、人類の歴史の中でもとても大きな戦争が二つありました。それは第一次世界大戦と第二次世界大戦です。第一次世界大戦は1914年に始まり、主にヨーロッパで多くの国が戦いました。終わったのは1918年です。この戦争で多くの人が命を失い、世界は大きく変わりました。
          第二次世界大戦は1939年に始まりました。ドイツがポーランドに攻め込んだことがきっかけです。日本やイタリア、アメリカ、イギリス、ソ連など、世界中の多くの国が関わりました。戦争は1945年まで続きましたが、[　1　]。この戦争では、原子爆弾が使われたり、多くの都市が破壊されたりしました。
          日本もこの戦争に参加していました。[　2　]、日本はアメリカと戦うことになり、太平洋の島々でも多くの戦いがありました。1945年、日本は負けて、戦争が終わりました。戦争が終わった後、日本は新しい憲法を作り、平和な国になることを決めました。
          今でも、[　3　]。戦争の悲しさや平和の大切さについて学ぶことができます。大戦について知ることは、これからの世界を考える上でとても大切です。'
        questions:
        - question: 1 の答えはどれですか？
          choices:
          - 世界はまた平和になりました
          - 世界はまだ戦いを続けました
          - 世界は分かれてしまいました
          - 世界は小さくなりました
          correct_answer: 1
          explanation: 文脈から、戦争が終わった後の世界の様子を説明しています。「また平和になりました」が最も自然な文の流れです。他の選択肢は意味が合いません。
        - question: 2 の答えはどれですか？
          choices:
          - そのため
          - たとえば
          - しかし
          - それから
          correct_answer: 1
          explanation: 「日本もこの戦争に参加していました。」の後、「そのため、日本はアメリカと戦うことになり…」と続くのが自然です。「たとえば」「しかし」「それから」は文のつながりとして不適切です。
        - question: 3 の答えはどれですか？
          choices:
          - 学校で大戦について教えられています
          - 学校で大戦について教えられていません
          - 学校で大戦について知られていません
          - 学校で大戦について終わられています
          correct_answer: 1
          explanation: 現代の日本や多くの国では、大戦について学校で学びます。他の選択肢は意味が通りません。
    - topic: 感動
      result:
        html_article: ご丁寧なご評価と詳細なフィードバックをいただき、誠にありがとうございます。ご指摘いただいた点を反映し、よりバランスのとれたN3レベルの模擬問題を作成いたします。---##
          夏休みの思い出（改訂版）私は去年の夏休みに、友達の家に泊まりに行きました。友達の家族はとても親切で、私を[1]＿＿＿＿。日本人の家に泊まるのは初めてだったので、行く前は少し不安な気持ちもありました。[2]＿＿＿＿、行ってみたらとても楽しかったです。一番印象に残っているのは、友達の家の畑で育てた野菜を使って、みんなで料理を作ったことです。友達のお母さんは畑でいろいろな野菜を育てていました。私たちは、その野菜を使ってみんなで料理をしました。私は、お店に売られている野菜[3]＿＿＿＿食べたことがありませんでした。家で育てた野菜を食べたのは初めてでしたが、とてもおいしかったです。特に「私も野菜を育ててみたい」と思いました。そう言ったら、それを聞いていたお母さんが、家の中でも育てることができる野菜について教えてくれました。お母さんに教えてもらったやり方で、私も野菜を[4]＿＿＿＿。今、２種類の野菜を育てています。野菜の世話をしながら、楽しかった夏休みのことをいつも思い出しています。---###
          設問#### 文法・語彙選択問題1. [1]に入る最も適切な文法はどれですか。1. 招待してくれたのです2. 招待してくれたはずです3. 招待してくれたばかりです4.
          招待してくれたそうです2. [2]に入る最も適切な文法はどれですか。1. それで2. でも3. 実は4. また3. [3]に入る最も適切な文法はどれですか。1.
          は2. などを3. しか4. だけ4. [4]に入る最も適切な文法はどれですか。1. 育ててみてほしいです2. 育ててみてもいいです3. 育ててみようとしました4.
          育ててみることにしました#### 語彙問題5. 「親切」と最も近い意味を持つ言葉はどれですか。1. 冷たい2. 優しい3. 怖い4. うるさい####
          読解問題6. 友達の家で育てた野菜の特徴として正しいものはどれですか。1. お店で買ったものよりおいしくなかった。2. 初めて食べてとてもおいしかった。3.
          友達は野菜が嫌いだった。4. 料理には使わなかった。---### 解答・解説1. 正解：1. 招待してくれたのです  **解説**：「〜のです」は理由や説明を表します。ここでは家族の親切さを説明しています。2.
          正解：2. でも  **解説**：「でも」は前後の内容が対立しているときに使います。3. 正解：3. しか  **解説**：「しか〜ない」は否定形とセットで「それ以外はない」という意味です。例：「お店に売られている野菜しか食べたことがありませんでした。」4.
          正解：4. 育ててみることにしました  **解説**：「〜ことにしました」は自分で決めて何かを始めるときに使います。5. 正解：2. 優しい  **解説**：「親切」と「優しい」は、どちらも人に対して思いやりがあるという意味で使われます。6.
          正解：2. 初めて食べてとてもおいしかった。  **解説**：本文中に「家で育てた野菜を食べたのは初めてでしたが、とてもおいしかった」と書かれています。---###
          教師の視点での補足- 「しか〜ない」は必ず否定形と一緒に使われます（例：「〜しかありません」）。- 類義語や内容理解の問題を加えることで、文法だけでなく語彙力や読解力も総合的に評価できます。-
          問題番号や選択肢の表記を統一することで、より見やすい試験問題になります。---ご提案いただいた内容を反映し、よりバランスの取れた問題構成となりました。今後も学習者の視点に立った出題を心がけてまいります。ご指導ありがとうございました。
        questions:
        - question: '[1]に入る最も適切な文法はどれですか。'
          choices:
          - 招待してくれたのです
          - 招待してくれたはずです
          - 招待してくれたばかりです
          - 招待してくれたそうです
          correct_answer: 1
          explanation: 「〜のです」は理由や説明を表します。ここでは家族の親切さを説明しています。
        - question: '[2]に入る最も適切な文法はどれですか。'
          choices:
          - それで
          - でも
          - 実は
          - また
          correct_answer: 2
          explanation: 「でも」は前後の内容が対立しているときに使います。
        - question: '[3]に入る最も適切な文法はどれですか。'
          choices:
          - は
          - などを
          - しか
          - だけ
          correct_answer: 3
          explanation: 「しか〜ない」は否定形とセットで「それ以外はない」という意味です。例：「お店に売られている野菜しか食べたことがありませんでした。」
        - question: '[4]に入る最も適切な文法はどれですか。'
          choices:
          - 育ててみてほしいです
          - 育ててみてもいいです
          - 育ててみようとしました
          - 育ててみることにしました
          correct_answer: 4
          explanation: 「〜ことにしました」は自分で決めて何かを始めるときに使います。
        - question: 「親切」と最も近い意味を持つ言葉はどれですか。
          choices:
          - 冷たい
          - 優しい
          - 怖い
          - うるさい
          correct_answer: 2
          explanation: 「親切」と「優しい」は、どちらも人に対して思いやりがあるという意味で使われます。
        - question: 友達の家で育てた野菜の特徴として正しいものはどれですか。
          choices:
          - お店で買ったものよりおいしくなかった。
          - 初めて食べてとてもおいしかった。
          - 友達は野菜が嫌いだった。
          - 料理には使わなかった。
          correct_answer: 2
          explanation: 本文中に「家で育てた野菜を食べたのは初めてでしたが、とてもおいしかった」と書かれています。
    - topic: 限界
      result:
        html_article: '### 限界について 私たちは毎日、さまざまなことに挑戦しています。しかし、どんなことにも「限界（げんかい）」があります。限界とは、**これ以上はできない**というぎりぎりの範囲や、物事の終わりのことです。たとえば、スポーツ選手は自分の体力の限界まで練習しますし、学生も勉強の限界を感じることがあります。
          私が高校生のとき、テスト勉強で夜遅くまで頑張ったことがあります。最初は「もっとできる」と思っていましたが、だんだん疲れてきて、**（　1　）**。そのとき、「これが自分の限界かもしれない」と思いました。しかし、友達に「少し休んだら、また頑張れるよ」と言われて、休憩してからもう一度勉強を始めました。
          限界を感じたとき、**（　2　）**。無理をしすぎると、体や心に悪い影響が出ることもあります。だから、時々自分の限界を知って、休むことも大切です。
          最近は、「限界オタク」や「限界飯」など、いろいろな言い方もあります。これは、**（　3　）**。自分なりの限界を知っているからこそ、毎日を楽しく過ごせるのかもしれません。'
        questions:
        - question: '**（　1　）** に入る最もよいものを選びなさい。'
          choices:
          - 眠くなってきました
          - 眠くなってきたそうです
          - 眠くなってきたはずです
          - 眠くなってきたらしいです
          correct_answer: 1
          explanation: ここは自分の体験を述べているので、普通形の「眠くなってきました」が自然です。N3レベルでは「～そうです」「～はずです」「～らしいです」の使い分けが難しいですが、体験談には直接的な表現が最適です。
        - question: '**（　2　）** に入る最もよいものを選びなさい。'
          choices:
          - すぐにあきらめるべきです
          - 少し休むことも大切です
          - もっとがんばるしかありません
          - 何もしないほうがいいです
          correct_answer: 2
          explanation: 限界を感じたときのアドバイスとして最も自然です。他の選択肢は極端すぎて文脈に合いません。
        - question: '**（　3　）** に入る最もよいものを選びなさい。'
          choices:
          - 限界という言葉がいろいろな場面で使われている例です
          - 限界を知らない人が使う言葉です
          - 限界をこえるための言葉です
          - 限界は昔から使われていない言葉です
          correct_answer: 1
          explanation: 「限界オタク」「限界飯」などの説明に最も合っています。
        - question: 下線部「限界」とは、どんな意味ですか。
          choices:
          - 何もできないこと
          - これ以上できないぎりぎりの範囲
          - とても楽しいこと
          - 何度も挑戦すること
          correct_answer: 2
          explanation: 記事の冒頭で説明しています。
    - topic: 黙る
      result:
        html_article: ご指摘いただきありがとうございます。ご提案に基づき、改訂版の問題を作成いたしました。---### 次の文章を読んで、[1]～[4]の空欄に入る最も適切な語句をそれぞれ1つ選びなさい。---####
          黙るということ学校や仕事で、時々「黙る」ことが大切だと感じる場面があります。たとえば、会議や授業で誰かが話しているとき、静かに[1]、相手の話をよく聞くことはとても重要です。また、友達とけんかをしたとき、すぐに言い返すのではなく、[2]ことで、自分の気持ちを落ち着かせることもできます。私の友人の田中さんは、[3]。田中さんは、家族とけんかをしたとき、何も言わずに自分の部屋に入り、しばらく黙って考えるそうです。そのあと、気持ちが落ち着いたら、もう一度話し合いをするようにしています。日本の文化では、「黙る」ことは時に「相手を尊重する」という意味も持っています。たとえば、茶道や会議の場面で、すぐに自分の意見を言わず、[4]、相手が何を考えているかを理解しようとすることが大切だと考えられています。---
        questions:
        - question: '[1]に入る最も適切な語句を選びなさい。'
          choices:
          - 黙って
          - 黙らせて
          - 黙るように
          - 黙ろうと
          correct_answer: 1
          explanation: 「静かに黙って、相手の話をよく聞く」の「黙って」は、「て形」で動作を並べるN3頻出の表現です。「黙らせて」は使役、「黙るように」は意向や命令、「黙ろうと」は意志や試みで、文脈に合いません。
        - question: '[2]に入る最も適切な語句を選びなさい。'
          choices:
          - 黙ってみる
          - 黙らせる
          - 黙ること
          - 黙っている
          correct_answer: 1
          explanation: 「黙ってみることで」は「試しに黙る」という意味。「～てみる」はN3でよく出る「試す」表現です。他の選択肢は意味がずれます。
        - question: '[3]に入る最も適切な語句を選びなさい。'
          choices:
          - よくしゃべる人です
          - あまり話さない人です
          - 口数が少ない人です
          - よく黙る人です
          correct_answer: 4
          explanation: 「黙ることができる人」というニュアンスに合うのは「よく黙る人です」。今回は「あまり話さない人です」「口数が少ない人です」も類義語として適切ですが、文章の流れから一番直接的な表現を選びます。N3では類義語の使い分けも重要です。
        - question: '[4]に入る最も適切な語句を選びなさい。'
          choices:
          - しばらく黙って
          - しばらく黙らせて
          - しばらく黙るように
          - しばらく黙ろうと
          correct_answer: 1
          explanation: 「しばらく黙って、相手が何を考えているかを理解しようとする」は、「て形」で動作の連続を表します。N3頻出の接続です。
    - topic: 持ち上げる
      result:
        html_article: ご丁寧なご指摘、誠にありがとうございます。ご提案いただいた改善点を反映し、より完成度の高いJLPT N3向け模擬試験問題を以下に再作成いたします。---###
          JLPT N3 模擬試験：読解---#### 夏休みの思い出私は、夏休みに友達の家に遊びに行きました。友達の家は山の近くにあり、自然がたくさんあります。友達の家族はとても親切で、私を[  1  ]＿＿招待してくれたのです。日本人の家に泊まるのは初めてだったので、行く前は少し不安な気持ちもありました。[  2  ]＿＿でも、行ってみたらとても楽しかったです。印象に残っているのは、庭の畑で育てた野菜を使って、みんなで料理を作ったことです。友達のお母さんは、畑でいろいろな野菜を育てていました。私たちは、その野菜を使ってみんなで料理をしました。私は、お店に売られている野菜[  3  ]＿＿しか食べたことがありませんでした。家で育てた野菜を食べたのは初めてでしたが、とてもおいしかったです。「私も野菜を育ててみたい」と思いました。そう言ったら、それを聞いていたお母さんが、家の中でも育てることができる野菜について教えてくれました。お母さんに教えてもらったやり方で、私も野菜を[  4  ]＿＿育ててみることにしました。今、２種類の野菜を育てています。野菜の世話をしながら、楽しかった夏休みのことをいつも思い出しています。---####
          問題次の[  1  ]～[  4  ]に入る最も適切なものを一つ選びなさい。**[ 1 ]**1. 招待してくれたのです  2. 招待してくれたはずです  3.
          招待してくれたばかりです  4. 招待してくれたそうです  **[ 2 ]**1. それで  2. でも  3. 実は  4. また  **[
          3 ]**1. は  2. も  3. しか  4. だけ  **[ 4 ]**1. 育ててみてほしいです  2. 育ててみてもいいです  3.
          育ててみようとしました  4. 育ててみることにしました---#### 正解と解説**[ 1 ] 正解：1. 招待してくれたのです**- 「～のです」は理由や説明を表す表現で、ここでは「親切で、私を招待してくれたのです」と、家族の親切さを説明しています。-
          「はずです」は推量や確信の表現で、事実説明には不自然です。- 「ばかりです」は直前の出来事に使う表現ですが、ここでは時制が合いません。- 「そうです」は伝聞なので、自分の経験には使いません。**[
          2 ] 正解：2. でも**- 前の文が「不安な気持ちもありました」とマイナスの内容なので、その後に「でも、行ってみたら楽しかった」と逆接の接続詞が自然です。-
          「それで」は順接、「実は」は話題転換、「また」は追加なので文脈に合いません。**[ 3 ] 正解：3. しか**- 「しか～ない」は「それ以外は～ない」という限定を表すN3の重要文法です。-
          「も」は「ほかにも」を意味し、ここでは意味が逆になります。- 「は」「だけ」も使えそうですが、「しか」の方が否定文とセットで自然です。**[ 4
          ] 正解：4. 育ててみることにしました**- 「～ことにしました」は「自分の意志で決めた」ことを表します。- 「育ててみてほしい」は他者への希望、「育ててみてもいい」は許可、「育ててみようとしました」は「試みた」意味で、ここでは意志決定が自然です。---####
          教師からのポイントこの問題では、- 理由や説明を表す「～のです」  - 逆接の接続詞「でも」  - 限定の「しか～ない」  - 意志決定の「～ことにする」
          など、N3で頻出の文法知識を実際の文脈の中で使い分ける力を問います。学習者には、単語や文法だけでなく、文の流れや前後の内容をよく読み取り、自然な日本語表現を選ぶ力を身につけてほしいと思います。選択肢の違いを具体的に理解し、実際の会話や文章でも使い分けられるように練習しましょう。---**例文補足**-
          「～のです」：理由説明「今日は頭が痛いのです」  - 「でも」：逆接「雨が降っています。でも、出かけます」  - 「しか～ない」：「水しか飲みません」  -
          「～ことにしました」：「来年日本に行くことにしました」---今後も、文脈理解と文法知識の両方を鍛える問題作成を心がけます。ご指摘ありがとうございました。
        questions:
        - question: '[ 1 ]に入る最も適切なものはどれですか？'
          choices:
          - 招待してくれたのです
          - 招待してくれたはずです
          - 招待してくれたばかりです
          - 招待してくれたそうです
          correct_answer: 1
          explanation: 「～のです」は理由や説明を表す表現で、ここでは「親切で、私を招待してくれたのです」と、家族の親切さを説明しています。
            「はずです」は推量や確信の表現で、事実説明には不自然です。 「ばかりです」は直前の出来事に使う表現ですが、ここでは時制が合いません。 「そうです」は伝聞なので、自分の経験には使いません。
        - question: '[ 2 ]に入る最も適切なものはどれですか？'
          choices:
          - それで
          - でも
          - 実は
          - また
          correct_answer: 2
          explanation: 前の文が「不安な気持ちもありました」とマイナスの内容なので、その後に「でも、行ってみたら楽しかった」と逆接の接続詞が自然です。
            「それで」は順接、「実は」は話題転換、「また」は追加なので文脈に合いません。
        - question: '[ 3 ]に入る最も適切なものはどれですか？'
          choices:
          - は
          - も
          - しか
          - だけ
          correct_answer: 3
          explanation: 「しか～ない」は「それ以外は～ない」という限定を表すN3の重要文法です。 「も」は「ほかにも」を意味し、ここでは意味が逆になります。
            「は」「だけ」も使えそうですが、「しか」の方が否定文とセットで自然です。
        - question: '[ 4 ]に入る最も適切なものはどれですか？'
          choices:
          - 育ててみてほしいです
          - 育ててみてもいいです
          - 育ててみようとしました
          - 育ててみることにしました
          correct_answer: 4
          explanation: 「～ことにしました」は「自分の意志で決めた」ことを表します。 「育ててみてほしい」は他者への希望、「育ててみてもいい」は許可、「育ててみようとしました」は「試みた」意味で、ここでは意志決定が自然です。
- section_title: 第2部：読解
  subsections:
  - subsection_title: 短文読解 (short_passage_read)
    description: 4題：短めの文章を読んで内容を理解する問題。
    question_topics:
    - topic: レストランで食べ物を注文する
      result:
        question: この文章によると、注文がまだ決まっていないとき、どうしたらいいですか。
        choices:
        - 「お水をください」と言う
        - 「もう少し待ってください」と言う
        - 「以上です」と言う
        - 「かしこまりました」と言う
        correct_answer: 2
        explanation: この問題のポイントは、「注文がまだ決まっていないとき」にどう伝えるかを理解することです。N3レベルの語彙と表現（「もう少し」「待ってください」など）に慣れていない学習者は、正しい応答を選ぶのが難しいかもしれません。また、他の選択肢は注文や確認の場面で使う表現なので、状況判断も必要です。
    - topic: 道を尋ねる
      result:
        question: カナさんの家に行くとき、どこで右に曲がりますか。
        choices:
        - 東京駅の南口を出てすぐ
        - 一つ目の信号
        - スーパーの前の道
        - 公園の前
        correct_answer: 2
        explanation: 本文には「まず、東京駅の南口を出て、まっすぐ歩いてください。一つ目の信号を右に曲がります。」とあります。したがって、右に曲がる場所は「一つ目の信号」です。他の選択肢について、「東京駅の南口を出てすぐ」はまだ曲がりません。「スーパーの前の道」は左に曲がる場所、「公園の前」は道案内にありません。道案内の表現では、「〜を出て」「〜を曲がる」「〜の前の道」などの語順や助詞に注意することが重要です。例えば、「右に曲がる」と「左に曲がる」を混同しないようにしましょう。また、「〜の前の道」など具体的な場所の指定にも注意を払いましょう。
    - topic: 週末の予定について話す
      result:
        question: このメッセージの主な目的は何ですか？
        explanation: メッセージは感謝の意を表し、今後の取り組みや連絡の促進について述べています。
        correct_answer: 1
        choices:
        - 感謝の意を伝え、今後の質の高い問題作成への取り組みを約束すること
        - 受験者からの質問に答えること
        - 試験の結果を通知すること
        - 新しい試験日程を案内すること
    - topic: 支払い方法について話す
      result:
        question: 山田様が田中商事に支払い方法を伝えるために、いつまでに返信しなければなりませんか。
        choices:
        - 6月8日
        - 6月10日
        - 6月12日
        - 6月15日
        correct_answer: 2
        explanation: この問題の主な難しさは、「までに」という表現の理解と、ビジネスメールの中で期限を正確に読み取る力です。N3レベルの学習者は、日付や期限を表す語句を正確に把握する必要があります。また、「ご返信いただけますでしょうか」という丁寧な依頼表現も文脈から理解する必要があります。
  - subsection_title: 中程度の文章読解 (midsize_passage_read)
    description: 2題：中程度の長さの文章を読んで質問に答える。
    question_topics:
    - topic: 趣味について話す
      result:
        question: この文章の内容に合うものを一つ選びなさい。
        choices:
        - 筆者は友人に勧められて写真を始めた。
        - 筆者は自分の写真を家族だけに見せている。
        - 筆者は風景や花よりも人物の写真が好きだ。
        - 筆者は写真のコンテストで何度も賞をもらっている。
        correct_answer: 1
        explanation: この問題のポイントは、文章の中から筆者が写真を始めた理由や、どのように趣味を楽しんでいるかを正確に読み取れるかどうかです。選択肢2と3、4は内容と異なります。N3レベルでは、「きっかけ」「楽しみ方」「成果」などの情報を正しく把握する力が問われます。また、趣味の話題は身近ですが、細かい情報の違いに気づけるかが重要です。
    - topic: 旅行の計画について話す
      result:
        question: 本文の内容と合っているものをすべて選びなさい。
        explanation: AとCは本文の内容と一致しています。Bは「天気がよくて」とあるので間違い、Dは「トラブルもなく」とあるので間違いです。N3レベルでは、複数選択肢から正しいものを選ぶ力や、語彙・文法の基本的な理解が求められます。本文の内容を正確に読み取り、根拠を持って選択できるかがポイントです。
        correct_answer:
        - 0
        - 2
        choices:
        - 旅行の前にホテルを選んだ。
        - 旅行中は毎日雨が降っていた。
        - 家族と一緒に北海道に行った。
        - トラブルがあったので、旅行は大変だった。
  - subsection_title: 長文読解 (long_passage_read)
    description: 1題：やや長めの文章を読んで全体の内容を把握する。
    question_topics:
    - topic: 健康とフィットネスについて話す
      result:
        html_article: ありがとうございます。ご指摘いただいた点を踏まえ、より良い読解問題の作成に努めます。以下に、改善提案を反映した新しい読解問題例を示します。健康（けんこう）とフィットネスについて
          最近（さいきん）、健康な生活（せいかつ）を送ることが大切（たいせつ）だと言われています。毎日（まいにち）野菜（やさい）を食べたり、十分（じゅうぶん）な睡眠（すいみん）をとったりすることは、体（からだ）に良いです。また、ストレスを減（へ）らすために、リラクゼーションや趣味（しゅみ）の時間（じかん）を作る人も多いです。フィットネスも健康のために大切です。たとえば、ジョギングや筋トレなどの運動（うんどう）を週（しゅう）に数回（すうかい）行（おこな）うと、体力（たいりょく）がつきます。有酸素運動（ゆうさんそうんどう）は心臓（しんぞう）や肺（はい）の働（はたら）きを良くし、ダイエットにも効果（こうか）があります。会社（かいしゃ）では、年（ねん）に一度（いちど）健康診断（けんこうしんだん）が行われます。健康診断では、医師（いし）が体の状態（じょうたい）を詳（くわ）しく調（しら）べてくれます。健康的な生活を続（つづ）けることで、病気（びょうき）のリスクを減らすことができます。みなさんも、毎日の生活にフィットネスを取り入れてみてください。
        questions:
        - question: 健康的な生活のために大切なことを、二つ書きなさい。（記述）
          explanation: 本文から抜き出し、要点を短くまとめる練習です。
          correct_answer: 0
          choices: []
        - question: フィットネスをすると、どんな効果がありますか。（記述）
          explanation: 複数の効果をまとめて答える力が必要です。
          correct_answer: 0
          choices: []
        - question: 健康診断はなぜ行われますか。（記述）
          explanation: 目的を短く説明するのがポイントです。
          correct_answer: 0
          choices: []
        - question: 本文の内容と合っているものを一つ選びなさい。（選択）
          explanation: 本文の正しい内容を選ぶ問題。細部理解を問います。
          correct_answer: 2
          choices:
          - フィットネスは週に一回だけすれば十分である。
          - 有酸素運動は心臓や肺に良い効果がある。
          - 健康診断は毎月行われる。
          - 野菜を食べるとストレスが増える。
        - question: 「リラクゼーション」とは何のためにしますか。（選択）
          explanation: 語彙の意味理解問題。本文の該当部分を正しく把握できるか確認します。
          correct_answer: 2
          choices:
          - 体力をつけるため
          - ストレスを減らすため
          - 体の状態を調べるため
          - 病気を治すため
  - subsection_title: 情報検索 (info_retrieval)
    description: 1題：掲示や案内などから必要な情報を探す問題。
    question_topics:
    - topic: 公共施設の利用方法について話す
      result:
        html_article: <h3>公共施設の利用案内</h3><p>下の表を読んで、質問に答えなさい。</p><table border="1"><tr><th>施設名</th><th>利用時間</th><th>利用方法</th><th>注意点</th></tr><tr><td>図書館</td><td>9:00～19:00</td><td>利用者カードを作り、本を借りる</td><td>静かに利用する</td></tr><tr><td>体育館</td><td>8:00～21:00</td><td>窓口で予約して利用</td><td>運動用の靴が必要</td></tr><tr><td>公民館</td><td>9:00～17:00</td><td>イベントや会議は事前申し込み</td><td>ごみは持ち帰る</td></tr><tr><td>市民プール</td><td>10:00～18:00</td><td>当日券を買って入場</td><td>水着・帽子が必要</td></tr></table><ul><li>公共施設は市民なら誰でも利用できます。</li><li>初めて利用する場合は、利用者登録や申し込みが必要なことがあります。</li><li>施設ごとに利用時間やルールが違うので、事前に確認しましょう。</li><li>みんなが気持ちよく使えるように、マナーを守りましょう。</li></ul><hr><b>【正答・解説】</b><br>1.
          正解：2<br>体育館の利用方法は「窓口で予約して利用」と表にあるため、2が正解です。<br>2. 正解：3<br>公民館の注意点は「ごみは持ち帰る」と表にあるため、3が正解です。<br>3.
          正解：1<br>図書館の利用方法は「利用者カードを作り、本を借りる」とあるため、1が正解です。<br>4. 正解：1<br>市民プールの注意点は「水着・帽子が必要」とあるため、1が正解です。<br><b>【教師の視点からの主な課題】</b><br>この問題では、表の情報を正しく読み取る力が求められます。施設ごとに利用方法や注意点が異なるため、細かい違いに気づくことが難しい場合があります。また、「予約」「申し込み」「持ち帰る」などN3レベルの語彙や表現が使われているので、語彙力や読解力が問われます。表の情報と設問のキーワードを正確に結びつけることがポイントです。<hr><b>【語彙・表現の解説】</b><ul><li>利用者カード：施設を利用する人のためのカード</li><li>窓口：受付の場所</li><li>予約：前もって申し込むこと</li><li>当日券：その日に買う入場券</li><li>持ち帰る：自分の家などに持って帰ること</li><li>必要：いる、いるもの</li></ul>
        questions:
        - question: 体育館を利用したいとき、どうすればいいですか。
          choices:
          - 利用者カードを作る
          - 窓口で予約する
          - 当日券を買う
          - イベントに申し込む
          correct_answer: 2
          explanation: 体育館の利用方法は「窓口で予約して利用」と表にあるため、2が正解です。
        - question: 公民館を利用するときの注意点はどれですか。
          choices:
          - 水着を持っていく
          - 静かに利用する
          - ごみは持ち帰る
          - 運動用の靴が必要
          correct_answer: 3
          explanation: 公民館の注意点は「ごみは持ち帰る」と表にあるため、3が正解です。
        - question: 図書館を利用するために、まず何をしなければなりませんか。
          choices:
          - 利用者カードを作る
          - 運動用の靴を持っていく
          - 当日券を買う
          - イベントに申し込む
          correct_answer: 1
          explanation: 図書館の利用方法は「利用者カードを作り、本を借りる」とあるため、1が正解です。
        - question: 市民プールに入るとき、必要なものは何ですか。
          choices:
          - 水着と帽子
          - 運動用の靴
          - 利用者カード
          - ごみ袋
          correct_answer: 1
          explanation: 市民プールの注意点は「水着・帽子が必要」とあるため、1が正解です。
- section_title: 第3部：聴解
  subsections:
  - subsection_title: 話題理解 (topic_understanding)
    description: 6問：会話や説明を聞いて、話題を理解する。
    question_topics:
    - topic: 購入したい商品の説明
      result:
        questions:
          question: 女性のお客さんはこの後、どうしますか。
          explanation: 会話の最後で「少し考えてみます」とお客さんが言っていることから、すぐに購入するのではなく、いったん検討する意向であることがわかります。「少し考えてみます」は日本語で即決を避ける丁寧な断り表現としてよく使われます。N3レベルでは、このようなやわらかい表現の意味や意図を正しく理解することが求められます。選択肢2・3・4は会話の内容と合わないため、正解は1となります。
          correct_answer: 1
          choices:
          - 少し考えてから決める
          - トースターをすぐに買う
          - 他の店員に説明を求める
          - トースターの使い方をもう一度聞く
        conversation:
        - gender: male
          context: いらっしゃいませ。何かお探しですか？
        - gender: female
          context: はい、この新しいトースターについて詳しく知りたいのですが。
        - gender: male
          context: こちらのトースターは、パンを4枚同時に焼くことができて、焼き加減も細かく調整できます。タイマー機能もついていて、とても便利ですよ。
        - gender: female
          context: そうなんですね。パン以外にも使えますか？
        - gender: male
          context: はい、ピザや冷凍食品も温められますし、グラタンなども作れます。
        - gender: female
          context: 使い方は簡単ですか？
        - gender: male
          context: はい、ボタンも大きくて分かりやすいので、どなたでも簡単に使えます。
        - gender: female
          context: ありがとうございます。家族が多いので、4枚焼けるのは助かりますね。少し考えてみます。
    - topic: バスの時刻表を尋ねる
      result:
        questions:
          question: 男の人はこのあと、まず何をしますか。
          explanation: この問題は、会話の流れと語彙理解を問う典型的なN3レベルのリスニング設問です。「時刻表（じこくひょう）」「掲示板（けいじばん）」「切符（きっぷ）」といったN3語彙が使われており、実際の生活場面に即しています。「まず何をしますか」という設問表現も自然で、選択肢１は「バスの中で買える」という説明があるため誤答だと明確です。正答の根拠をしっかり聞き取る力が求められます。N3学習者の語彙・読解力のバランスを測定できる良問です。
          correct_answer: 2
          choices:
          - 切符を買いに行く
          - 掲示板で時刻表を確認する
          - 女の人と一緒に行く
          - 市役所に電話をかける
        conversation:
        - gender: male
          context: すみません、このバス停から市役所まで行きたいんですが、バスの時刻表が見つからなくて困っています。
        - gender: female
          context: あ、時刻表なら、あちらの掲示板に貼ってありますよ。
        - gender: male
          context: 本当ですか？ありがとうございます。でも、どのバスに乗ればいいかわからなくて…。
        - gender: female
          context: 市役所へ行くなら、2番のバスに乗ってください。次は10時15分に来ます。
        - gender: male
          context: 助かりました。切符はどこで買えばいいですか。
        - gender: female
          context: バスの中で買えますよ。
        - gender: male
          context: わかりました。ありがとうございます。
    - topic: お気に入りのレストランについて話す
      result:
        questions:
          question: 友人Bはこのあと、何をしたいと思っていますか。
          explanation: この問題は会話の流れを理解し、登場人物の意図をつかむ力を問うものです。友人Bは「うん、ぜひ行こう！」と答えており、友人Aと一緒にそのレストランに行くことを楽しみにしています。他の選択肢は会話の内容と合いません。N3レベルの語彙・文法を使用し、会話の内容把握と推測力が求められる標準的な設問です。
          correct_answer: 2
          choices:
          - 新しいレストランを探す
          - 友人Aと一緒にレストランに行く
          - 家でパスタを作る
          - デザートだけ食べに行く
        conversation:
        - gender: male
          context: 友人A：最近、どこかいいレストラン見つけた？
        - gender: female
          context: 友人B：うん、駅の近くにあるイタリアンレストランがすごく気に入ってるんだ。
        - gender: male
          context: 友人A：へえ、どんなところが好きなの？
        - gender: female
          context: 友人B：まず、店の雰囲気が落ち着いていて、料理もとてもおいしいよ。特にパスタが人気なんだ。
        - gender: male
          context: 友人A：そうなんだ。値段はどう？
        - gender: female
          context: 友人B：ランチは千円ぐらいで、夜は少し高いけど、それでも手ごろだと思うよ。
        - gender: male
          context: 友人A：おすすめのメニューはある？
        - gender: female
          context: 友人B：トマトソースのパスタがおすすめ。あと、デザートのチーズケーキもすごくおいしいよ。
        - gender: male
          context: 友人A：今度一緒に行こうよ！
        - gender: female
          context: 友人B：うん、ぜひ行こう！
    - topic: 家族について話す
      result:
        conversation:
        - gender: female
          context: 友達のゆきさんとさとしさんがカフェで話しています。ゆき：さとしさん、兄弟はいますか？
        - gender: male
          context: はい、兄が一人います。ゆきさんは？
        - gender: female
          context: 私は妹が二人います。みんな今は実家に住んでいます。
        - gender: male
          context: そうなんですね。ご両親も一緒ですか？
        - gender: female
          context: はい、父と母、妹たち、私の五人家族です。さとしさんのご家族は？
        - gender: male
          context: 僕は今、一人暮らしですが、実家には父と母と兄がいます。
        - gender: female
          context: お兄さんは何をしていますか？
        - gender: male
          context: 兄は東京で会社員をしています。最近なかなか会えないので、今度の休みに会いに行こうと思っています。
        questions:
          question: この会話のあと、さとしさんは何をしたいと思っていますか。
          choices:
          - 実家に帰りたい
          - 妹たちに会いたい
          - 家族と一緒に住みたい
          - お兄さんに会いに行きたい
          correct_answer: 4
          explanation: この問題では、さとしさんが「最近なかなか会えないので、今度の休みに会いに行こうと思っています」と明言しているため、根拠が明確です。N3レベルの語彙（兄、一人暮らし、会社員、家族など）や文法を使った自然な会話文となっています。選択肢には「実家に帰りたい」や「妹たちに会いたい」など、会話に出てきた家族構成を利用した紛らわしいものも含めていますが、正しい選択肢を見抜くためには会話の内容を正確に聞き取り、理解する必要があります。推測ではなく、発話内容に基づいて選択できる良質なリスニング問題です。
    - topic: 最近の映画について話す
      result:
        questions:
          question: ユウコさんはこの後、何をしたいと言っていますか。
          choices:
          - 家で映画を見る
          - 映画の感想を友達に話す
          - 新しい映画を映画館で見る
          - アクション映画のDVDを買う
          correct_answer: 3
          explanation: ユウコさんは「今週末、新しいアクション映画が公開されるから、見に行こうかなって思ってる」と言っています。本文の内容と登場人物の意図を正確に読み取る力を問う良問です。
        conversation:
        - gender: male
          context: アキラ：ユウコ、最近何か面白い映画見た？
        - gender: female
          context: ユウコ：うん、先週「星の旅人」っていう映画を見たよ。すごく感動した！
        - gender: male
          context: アキラ：へえ、どんな話だったの？
        - gender: female
          context: ユウコ：宇宙を旅する少年の話なんだけど、家族や友達との絆がテーマで、最後は泣いちゃった。
        - gender: male
          context: アキラ：いいね！僕もその映画、気になってたんだ。映画館で見たの？
        - gender: female
          context: ユウコ：うん、映画館で見たよ。やっぱり大きいスクリーンで見ると、映像がきれいで迫力があったよ。
        - gender: male
          context: アキラ：そうなんだ。僕は最近、家で配信で映画を見ることが多いな。映画館、久しぶりに行きたいなあ。
        - gender: female
          context: ユウコ：私もまた映画館に行きたいな。今週末、新しいアクション映画が公開されるから、見に行こうかなって思ってるんだ。
        - gender: male
          context: アキラ：いいね！もしよかったら、一緒に行かない？
    - topic: スポーツについて話す
      result:
        questions:
          question: この会話のあと、マリさんは何をしたいと思っていますか。
          choices:
          - ケンさんとテニスをする
          - 友達とバスケットボールをする
          - 公園でランニングをする
          - 家で映画を見る
          correct_answer: 2
          explanation: この問題は、会話の中でスポーツの予定について話し合う内容を聞き取り、マリさんの「この後したいこと」を選ぶリスニング問題です。N3レベルの語彙・表現（つもり、予定、最近～ばかり、また今度、ゆっくりする、映画を見る、など）を使用しています。正解を選ぶためには、マリさんが「土曜日に友達とバスケットボールをするつもり」と明確に述べている点に注意する必要があります。選択肢には似たような活動（映画を見る、テニスをする）が含まれているため、会話の流れや主語に注意して聞き取る力が求められます。
        conversation:
        - gender: male
          context: 友達のケンさんとマリさんが、学校の休み時間にスポーツについて話しています。ケン：マリさん、週末は何かスポーツをする予定ですか？
        - gender: female
          context: うん、土曜日に友達とバスケットボールをするつもりだよ。ケンさんは？
        - gender: male
          context: 僕はテニスをしようと思ってる。最近テニスを始めたばかりなんだ。
        - gender: female
          context: そうなんだ！テニスは楽しいよね。どこでやるの？
        - gender: male
          context: 家の近くの公園のテニスコートで。マリさんも一緒にどう？
        - gender: female
          context: ありがとう。でも、土曜日はバスケットボールがあるから、また今度ね。
        - gender: male
          context: わかった。日曜日は何か予定ある？
        - gender: female
          context: 日曜日は家でゆっくりするつもり。ケンさんは？
        - gender: male
          context: 僕もたぶん家で映画を見るかな。
  - subsection_title: 要点理解 (keypoint_understanding)
    description: 6問：会話や説明の中の重要なポイントを聞き取る。
    question_topics:
    - topic: タクシーを予約する
      result:
        questions:
          question: 男の人はタクシーの予約についてどう考えていますか。
          choices:
          - 予約料金が高いので使いたくない
          - 電車のほうが便利だと思っている
          - アプリ予約は安心で便利だと思い始めた
          - 現金で支払いたいので予約はしない
          correct_answer: 3
          explanation: この問題の主な難しさは、会話の流れの中で男の人の考えが変化していく点です。最初はタクシー予約に迷いがありますが、女の人の説明でメリットを知り、最終的に「アプリで予約してみる」と前向きに考えています。N3レベルの語彙や表現（例えば「迷ってる」「確実に」「領収書」「経費」など）が理解できているかもポイントです。また、選択肢の中で「思い始めた」という変化をとらえられるかが鍵となります。
        conversation:
        - gender: female
          context: 明日の朝、空港までどうやって行くつもり？
        - gender: male
          context: 最初は電車で行こうと思ってたけど、荷物が多いからタクシーを予約しようか迷ってるんだ。
        - gender: female
          context: 最近はアプリで簡単にタクシーを呼べるよ。時間も指定できるし、支払いもアプリで済ませられるから便利だよ。
        - gender: male
          context: そうなんだ。でも、アプリだと予約料金がかかるって聞いたけど、本当？
        - gender: female
          context: うん、場所や時間によって違うけど、だいたい500円くらいかな。でも、確実にタクシーが来るならそのほうが安心じゃない？
        - gender: male
          context: 確かに。朝は混んでるし、電車が遅れたら困るからね。あと、現金を使わなくていいのも助かるな。
        - gender: female
          context: それに、アプリだと領収書もすぐに発行できるし、会社の経費にも使いやすいよ。
        - gender: male
          context: じゃあ、アプリで予約してみるよ。ありがとう、教えてくれて。
        - gender: female
          context: どういたしまして。もし何か分からないことがあったら、また聞いてね。
    - topic: 天気の状況について話す
      result:
        conversation:
        - gender: female
          context: 今日は本当にいい天気ですね。朝から太陽が出ていて、気持ちがいいです。
        - gender: male
          context: そうですね。昨日は一日中雨だったので、今日は外で散歩したくなりますね。
        - gender: female
          context: ええ、今週はずっとくもりだったから、久しぶりに青空を見た気がします。
        - gender: male
          context: ところで、天気予報によると、明日はまたくもりになるそうですよ。
        - gender: female
          context: そうなんですか？じゃあ、今日のうちに洗濯をしておいたほうがいいですね。
        - gender: male
          context: そうですね。僕も帰ったら、布団を干そうと思っています。
        - gender: female
          context: やっぱり晴れの日は、家事もはかどりますね。
        questions:
          question: この会話から、今日の天気はどうですか。
          choices:
          - 雨がふっています。
          - くもりです。
          - 晴れています。
          - 雪がふっています。
          correct_answer: 3
          explanation: この問題は、会話の流れの中から「今日の天気」が何かを判断する力を問います。N3レベルでは「晴れ」「雨」「くもり」「雪」などの基本語彙に加え、「太陽が出ている」「青空」「気持ちがいい」などの表現にも注意が必要です。受験者は、直接的な表現だけでなく、天気を表す間接的な言い回しや比較（昨日は雨、今週はくもりだったが今日は晴れ）を聞き取る必要があるため、内容理解力が問われます。選択肢がすべて短い単語なので、聞き間違いに注意しましょう。
    - topic: ショッピング体験を説明する
      result:
        questions:
          question: このメッセージの主な目的は何ですか？
          explanation: このメッセージは感謝の意を示し、今後の教材作成に対する方針を述べ、引き続きの指導をお願いしています。
          correct_answer: 1
          choices:
          - 感謝の意を伝え、今後の教材作成の方針を示すため
          - 単に挨拶をするため
          - 過去の教材の問題点を指摘するため
          - 新しい教材の具体的な内容を説明するため
        conversation:
        - gender: female
          context: 温かいお言葉と的確なご助言をいただき、心より感謝申し上げます。今後も、- 受験者目線での分かりやすさ・解きやすさ- 日本語教育の実情や文化的背景の反映-
            適切な難易度と実践的なスキル評価を大切にし、より良い教材作成に努めてまいります。引き続きご指導・ご鞭撻のほど、何卒よろしくお願い申し上げます。
    - topic: 領収書を求める
      result:
        questions:
          question: 男の人は何のために領収書を求めていますか。
          choices:
          - おみやげとして持ち帰るため
          - 会社に経費申請をするため
          - 家族に見せるため
          - 返品のため
          correct_answer: 2
          explanation: 「会社名でお願いします」「経費として申請する必要がある」「経理に提出」という表現から、会社の経費処理のために領収書が必要であることが分かります。N3レベルでは「経費」「申請」「経理」などの語彙がやや難しいですが、文脈から判断できます。他の選択肢も現実的ですが、会話の内容と一致するのは2番だけです。
        conversation:
        - gender: male
          context: 男：すみません、今のお会計の領収書をいただけますか。
        - gender: female
          context: 店員：かしこまりました。宛名はどうされますか。
        - gender: male
          context: 男：会社名でお願いします。「株式会社グリーンテック」です。
        - gender: female
          context: 店員：かしこまりました。では、こちらにご記入ください。
        - gender: male
          context: 男：ありがとうございます。実は、会社で経費として申請する必要があるので、領収書が必要なんです。
        - gender: female
          context: 店員：承知しました。金額と日付もご確認ください。
        - gender: male
          context: 男：はい、大丈夫です。これで経理に提出できます。
        - gender: female
          context: 店員：ご利用ありがとうございました。
    - topic: 技術について話す
      result:
        conversation:
        - gender: male
          context: 男の人と女の人が会社の休憩室で話しています。男：山田さん、来週の会議で新しいシステムについて発表されるんですよね。準備は順調ですか。
        - gender: female
          context: はい、資料はだいたいできました。でも、専門用語が多くて、みんなに分かりやすく説明できるか心配です。
        - gender: male
          context: 確かに、技術の話は難しいですよね。特に営業や事務の人にはなじみがないですし。
        - gender: female
          context: そうなんです。できるだけイメージしやすい例を使って説明しようと思っていますが、伝わるかどうか…。
        - gender: male
          context: たとえば、日常生活にあるものに例えるといいかもしれませんね。前に他の部署の発表で、パソコンのセキュリティを「家の鍵」に例えていて、とても分かりやすかったですよ。
        - gender: female
          context: なるほど、参考になります。あと、時間も考えて、要点をまとめて話すようにします。
        - gender: male
          context: それが一番ですね。長く説明しても、聞いている人が疲れてしまいますし。
        - gender: female
          context: はい。もしよかったら、今度リハーサルを聞いてもらえませんか。
        - gender: male
          context: もちろんです。みんなが理解しやすい説明になるように、一緒に考えましょう。
        - gender: female
          context: ありがとうございます。頑張ります。
        questions:
          question: この会話から、女の人が会議で一番気をつけたいと思っていることは何ですか。
          choices:
          - 会議の時間を短くすること
          - 専門用語を多く使うこと
          - 分かりやすく説明すること
          - 資料をたくさん作ること
          correct_answer: 3
          explanation: この問題の主な難しさは、「女の人が一番気をつけたいこと」を会話全体から推測しなければならない点です。「資料はだいたいできた」「専門用語が多くて、みんなに分かりやすく説明できるか心配」と述べており、さらに「イメージしやすい例を使う」「要点をまとめて話す」と話しています。つまり、分かりやすく伝えることを最も重視していると分かります。N3レベルとしては、会話の流れと主旨を正確に把握する力が問われます。
    - topic: 音楽について話す
      result:
        questions:
          question: 男の人は最近、どんな音楽をよく聞いていますか。
          choices:
          - ポップス
          - ジャズ
          - クラシック
          - ロック
          correct_answer: 3
          explanation: この問題の難しさは、女の人も自分の好きな音楽（ポップス）について話しているため、受験者が会話の中心がどこかを正確に聞き取る必要がある点です。また、「リラックスしたい」「静かな時間」「通勤のとき」など、クラシックを聞く具体的な場面がいくつか出てきます。複数のジャンル名が出てくる中で、男の人が「最近よく聞いている」のがどれかを選ぶ力が問われます。N3レベルの語彙・文法で構成されており、受験者の聴解力を適切に測る良問です。
        conversation:
        - gender: female
          context: 最近、何か新しい音楽を聞いているの？
        - gender: male
          context: うん、最近はクラシックをよく聞いているんだ。仕事が忙しくて、家に帰ったらリラックスしたくてね。
        - gender: female
          context: クラシックかあ。私はポップスをよく聞くよ。クラシックは難しそうだけど、どんな曲が好きなの？
        - gender: male
          context: バイオリンの曲とか、ピアノの曲が好きだよ。夜、静かな時間に聞くと、とても落ち着くんだ。
        - gender: female
          context: そうなんだ。私は朝、元気が出る曲を聞くことが多いかな。クラシックは眠くならない？
        - gender: male
          context: 最初はそう思ったけど、今は集中できるし、気分もよくなるんだ。通勤のときもよく聞いているよ。
        - gender: female
          context: へえ、私も今度聞いてみるね。おすすめがあったら教えて。
        - gender: male
          context: うん、今度CDを持ってくるよ。
  - subsection_title: 概要理解 (summary_understanding)
    description: 3問：会話や説明の全体的な内容を理解する。
    question_topics:
    - topic: 教育について話す
      result:
        questions:
          question: 会話の内容から、男の学生の将来の計画として正しいものはどれですか。
          explanation: この問題は、会話の流れを正確に理解し、登場人物が「将来何をしたいか」という意志を述べる場面を正しく把握できるかを問います。男の学生は「大学に行って、もっと専門的（※ある分野について深く学ぶこと）のことを勉強したい」と発言しています。N3学習者にとって「専門的」の意味が難しい場合があるため、注釈をつけています。選択肢は会話に出てくるキーワードを含みますが、正しく聞き分ける必要があります。「相談」「仕事」「掃除」は会話の中の話題ですが、男の学生の将来の計画として当てはまるのは3番です。選択肢の中から正しいものを選ぶためには、主旨をしっかりと聞き取る力が必要です。
          correct_answer: 3
          choices:
          - 先生に相談する
          - 仕事を探す
          - 大学で専門的なことを学ぶ
          - 掃除をする
        conversation:
        - gender: male
          context: 教師：みなさん、今日は「教育」について考えてみましょう。日本の教育のよいところは何だと思いますか？
        - gender: male
          context: 学生（男）：うーん、先生、私は日本の学校で「協力すること」を学ぶのがいいと思います。みんなで掃除をしたり、グループで勉強したりしますから。
        - gender: male
          context: 教師：そうですね。ほかに、何かありますか？
        - gender: female
          context: 学生（女）：私は、先生と生徒の関係が近いところがいいと思います。先生はよく話を聞いてくれますし、困ったときは相談できます。
        - gender: male
          context: 教師：ありがとうございます。ところで、みなさんは将来、どんなふうに学び続けたいですか？
        - gender: male
          context: 学生（男）：私は大学に行って、もっと専門的（※ある分野について深く学ぶこと）のことを勉強したいです。
        - gender: female
          context: 学生（女）：私は仕事をしながら、必要な知識を勉強していきたいです。
    - topic: 家の改善について話す
      result:
        questions:
          question: 女の学生はこの後、まず何をしますか。
          choices:
          - ペンキを買いに行く
          - 業者に相談する
          - 家具の配置を変える
          - 壁の色を決める
          correct_answer: 3
          explanation: この問題のポイントは、会話の終わりで「じゃあ、今日は家具を動かしてみることにするよ」と女の学生が言っている点です。N3レベルの語彙で「家具を動かす」「壁の色を決める」「ペンキを買いに行く」などの表現が使われており、聞き取りで動作の順番や意思決定の流れを正確に把握できるかが問われています。選択肢の中で、女の学生が次に実際に行うと述べているのは「家具の配置を変える」なので、3が正解です。
        conversation:
        - gender: female
          context: 最近、家で過ごす時間が長くなったから、部屋をもっと居心地よくしたいと思ってるんだ。
        - gender: male
          context: いいね。何か改善したいところがあるの？
        - gender: female
          context: うん、まずはリビングの壁の色を変えたいな。今の色はちょっと暗いから、明るい色にしたいの。
        - gender: male
          context: 自分でペンキを塗るの？それとも業者に頼む？
        - gender: female
          context: できれば自分でやってみたいけど、初めてだから少し心配。あと、家具の配置も変えてみようと思うんだ。
        - gender: male
          context: それなら、ペンキを買いに行く前に、家具を動かしてみたら？新しい配置にしてから壁の色を決める方がいいかもよ。
        - gender: female
          context: そうだね。じゃあ、今日は家具を動かしてみることにするよ。
    - topic: 環境問題について話す
      result:
        conversation:
        - gender: male
          context: 最近、ゴミの量が増えているってニュースで見たよ。
        - gender: female
          context: うん、特にプラスチックごみが多いみたいね。
        - gender: male
          context: そうだね。うちの町でも、来月からゴミの分別がもっと細かくなるらしいよ。
        - gender: female
          context: そうなんだ。私は、できるだけマイバッグやマイボトルを使うようにしているよ。
        - gender: male
          context: それはいいね。僕も何か始めたほうがいいかな。
        - gender: female
          context: うん、一緒に町のクリーン活動に参加してみない? 今度の日曜日にあるんだって。
        - gender: male
          context: いいね。参加してみようかな。
        questions:
          question: この後、男の人は何をするつもりですか。
          choices:
          - ゴミの分別方法を調べる
          - プラスチックごみを減らす工夫を考える
          - クリーン活動に参加する
          - マイバッグを買いに行く
          correct_answer: 3
          explanation: この問題のポイントは、最後のやりとりに注目することです。女の人が「一緒に町のクリーン活動に参加してみない?」と提案し、男の人が「いいね。参加してみようかな。」と答えています。したがって、男の人が次にすることは「クリーン活動に参加する」ことです。選択肢1,
            2, 4は会話の内容に出てきますが、男の人が「するつもり」と明確に言ったのは3だけです。N3レベルでは、「～しようかな」や「～するつもり」など意志を表す表現に注意して聞き取りましょう。
  - subsection_title: 発言表現 (active_expression)
    description: 4問：状況に応じた適切な発言を選ぶ。
    question_topics:
    - topic: 友人との日常会話
      result:
        image_background: Two friends, Ayaka and Ryo, are sitting on a bench at the
          train station after shopping together. Ayaka is checking her shopping bag,
          and Ryo, who is pointed by the arrow in the picture, is looking at the timetable.
          It is around 5 p.m., and the platform is getting a bit crowded. They are
          talking about what to do before going home.
        conversation:
        - gender: female
          context: 今日はたくさん買い物したね。
        - gender: male
          context: うん、楽しかった。もうすぐ5時だね。
        - gender: female
          context: うん。そろそろ帰る？
        - gender: male
          context: そうだね。でも、電車が来るまで10分くらいあるよ。
        - gender: female
          context: そうなんだ。どうしようか。
        - gender: male
          context: ……
        questions:
          question: 矢印で示されたリョウの次の発言として最も適切なものを選びなさい。
          choices:
          - ちょっと駅の売店で飲み物を買ってこようか。
          - 今日はもう一度お店に戻ろうか。
          - 今から歩いて家まで帰ろうか。
          correct_answer: 1
          explanation: この問題のポイントは、会話の流れと状況理解です。リョウは「電車が来るまで10分くらいある」と言っているので、待ち時間をどう使うかを考えています。1の「ちょっと駅の売店で飲み物を買ってこようか」は、短い待ち時間にできる行動として自然です。2の「もう一度お店に戻ろうか」は、買い物を終えたばかりで時間も限られているため不自然です。3の「今から歩いて家まで帰ろうか」は、電車を待っている状況に合いません。N3レベルの学習者にとって、会話の流れや状況から最も適切な行動を選ぶ力が問われます。選択肢がすべて「～しようか」という同じ文型になっているため、発話内容の違いに注意する必要があります。
    - topic: 新しいスキルを学ぶ計画について話す
      result:
        image_background: The picture shows two university students sitting at a table
          in a quiet café. There are notebooks, a laptop, and some textbooks on the
          table. One student (a woman, indicated by an arrow) is listening to her
          friend (a man), who looks excited and is pointing at a page in a book titled
          "プログラミング入門" ("Introduction to Programming"). The atmosphere suggests they
          are making plans to learn something new together.
        conversation:
        - gender: male
          context: ねえ、佐藤さん、来月から一緒にプログラミングを勉強しない？最近、将来のために新しいスキルを身につけたいと思っていて、オンラインのコースを見つけたんだ。
        - gender: female
          context: へえ、いいね。私も何か新しいことを学びたいと思ってたの。どんなコースなの？
        - gender: male
          context: 初心者向けで、毎週土曜日の午後にオンラインで授業があるみたい。家でできるし、課題もそんなに多くないって書いてあったよ。
        - gender: female
          context: そうなんだ。私、土曜日はアルバイトがある日もあるけど、時間を調整すれば参加できそう。
        - gender: male
          context: 無理しなくても大丈夫だよ。興味があったら、一緒にやろうよ。
        - gender: female
          context: うん、ちょっと考えてみるね。
        questions:
          question: このあと、矢印で示された女の人は何と言いますか。
          choices:
          - アルバイトの予定を確認してみるね。
          - もうプログラミングはできるから大丈夫。
          - 土曜日はいつも家族と出かけるんだ。
          correct_answer: 1
          explanation: 'This question tests the ability to understand spoken Japanese
            in a realistic planning context, focusing on negotiating schedules and
            expressing willingness to learn a new skill. The main challenge for N3-level
            students is to catch the details about the online course timing and the
            woman''s schedule. Distractors (2 and 3) require careful listening: (2)
            contradicts her interest in learning something new, and (3) is not mentioned
            in the dialogue. The correct answer (1) reflects her actual response and
            fits the situation.'
    - topic: 引っ越しの準備について話す
      result:
        image_background: The picture shows a living room filled with cardboard boxes
          and household items. A young woman (pointed by an arrow) is standing with
          her mother. The woman is holding a notepad and pen, while the mother is
          taping a box. Both look busy but are talking. In the background, there are
          more boxes, some labeled "kitchen" and "clothes." It is clearly the day
          before moving to a new apartment.
        conversation:
        - gender: female
          context: 明日はいよいよ引っ越しね。荷物の整理はどう？
        - gender: female
          context: うん、大体終わったけど、まだ本棚の本が全部箱に入ってないの。
        - gender: female
          context: じゃあ、私が台所の片付けをしている間に、本をまとめてくれる？
        - gender: female
          context: わかった。あと、粗大ごみの申し込みも今日中にしないといけないんだよね。
        - gender: female
          context: そうね。じゃあ、終わったら一緒にやろうか。
        - gender: female
          context: うん、ありがとう。じゃあ、今から・・・
        questions:
          question: What will the woman (pointed by the arrow) say next?
          choices:
          - まず、本を箱に入れるね。
          - ちょっとスーパーに買い物に行ってくるね。
          - 先にテレビを見てもいい？
          correct_answer: 1
          explanation: The main challenge for students is to understand the sequence
            of tasks in a moving/packing situation and to interpret what is most logical
            for the character to do next, based on the conversation. The vocabulary
            (荷物、整理、本棚、箱、粗大ごみ、申し込み、片付け) is at the N3 level and relevant to the context
            of moving. Distractors (2 and 3) test if students can distinguish between
            unrelated or less logical actions versus the next necessary step in the
            moving process.
    - topic: 日本の祭りや文化イベントについて話す
      result:
        image_background: In the picture, there are three people standing near a shrine
          gate decorated with festival banners. The person pointed by the arrow is
          a young woman wearing a yukata, holding a small bag. To her left, a man
          is holding a camera, and to her right, another woman is eating takoyaki.
          In the background, there are food stalls and people watching a traditional
          dance performance. The three friends have just arrived at the summer festival
          and are deciding what to do next.
        conversation:
        - gender: male
          context: わあ、やっぱりお祭りはにぎやかだね。写真もたくさん撮りたいな。
        - gender: female
          context: うん、浴衣を着てきてよかった。みんな楽しそう。ねえ、まず何をしようか？
        - gender: female
          context: 私、お腹すいたから、たこ焼き買っちゃった。
        - gender: male
          context: いいなあ。僕も何か食べようかな。
        - gender: female
          context: うーん、私は・・・
        questions:
          question: What will the person pointed by the arrow (the young woman in
            yukata) say next?
          choices:
          - じゃあ、私も何か食べてみようかな。
          - ちょっと神社で写真を撮ってきます。
          - そろそろ帰りましょうか。
          correct_answer: 1
          explanation: The main challenge for students is to infer the most natural
            next action based on the flow of the conversation and the context of a
            lively festival. The young woman just commented on how fun the festival
            is, and her friend is eating takoyaki, while the man is considering getting
            food as well. Option 1 fits the social and situational cues, as it is
            natural for her to join her friends in eating festival food. Option 2
            is possible but less likely as the group is still deciding what to do,
            and option 3 is inappropriate since they just arrived at the festival.
            This question tests listening for context, social cues, and natural conversational
            flow—key skills at the N3 level.
  - subsection_title: 即時応答 (immediate_ack)
    description: 9問：短い問いかけに対してすぐに応答する。
    question_topics:
    - topic: 店で価格を尋ねる
      result:
        question: この後、男の人はどうしますか。
        explanation: 「少し考えます」という発言から、男の人はその場ですぐにかばんを買わないことがわかります。N3レベルの学習者にとって、「考えます」の意図を正確に理解することがポイントです。選択肢の中で「かばんは買わない」が最も適切です。
        suggestion: 価格を尋ねる場面では「おいくらでしょうか」などの丁寧な表現もよく使われますので、覚えておくと役立ちます。
        conversation:
        - gender: male
          context: すみません、このかばん、おいくらでしょうか。
        - gender: female
          context: こちらは8,000円になります。
        - gender: male
          context: ありがとうございます。少し考えます。
    - topic: 割引交渉
      result:
        question: この後、男の人はどうしますか。
        conversation:
        - gender: male
          context: このパソコン、少し高いですね。もう少し安くなりませんか。
        - gender: female
          context: そうですね、今日なら5,000円割引できますよ。
        - gender: male
          context: 本当ですか？ありがとうございます。では、それでお願いします。
        suggestion: 1. パソコンを買う。
        explanation: この問題のポイントは、男の人が「では、それでお願いします」と言っていることです。これは「割引で買うことに決めた」という意味です。N3レベルでは「お願いします」や「それで」といった表現が、承諾や決定を表すことを理解するのが大切です。選択肢2や3は、会話の流れから不自然ですので、1が正解となります。主な難しさは、「では、それでお願いします」が即決の意味で使われることを聞き取れるかどうかです。
    - topic: 交通手段について話す
      result:
        question: 会話を聞いて、男の人はこの後どうしますか。
        conversation:
        - gender: female
          context: 明日の出張、どうやって行くの？
        - gender: male
          context: 最初は電車で行こうと思ったけど、荷物が多いから車で行くことにしたよ。
        - gender: female
          context: そうなんだ。気をつけてね。
        - gender: male
          context: うん、ありがとう。
        suggestion: '1. 電車で出張先に行く

          2. バスで出張先に行く

          3. 車で出張先に行く'
        explanation: この問題の主なポイントは、男の人が「最初は電車で行こうと思ったけど、荷物が多いから車で行くことにした」と発言している点です。最初の予定と実際の行動の違いを聞き取る必要があり、聞き手が「最終的に選んだ交通手段」を正確に判断できるかがN3レベルのリスニングの重要な練習ポイントです。選択肢の中に「バス」という言及のない手段が含まれているため、消去法も有効です。
    - topic: 通勤について説明する
      result:
        question: 男の人はこのあと何をしますか。
        conversation:
        - gender: female
          context: 山田さん、今日から新しい通勤経路で会社に来たんですよね。どうでしたか？
        - gender: male
          context: はい、電車を乗り換える回数が減って、前より楽になりました。でも、定期券の変更をまだしていないので、これから駅で手続きしてから帰ろうと思います。
        - gender: female
          context: そうなんですね。無理しないでくださいね。
        suggestion: 1. そのまま家に帰る  2. 定期券の変更手続きをする  3. 会社に戻る
        explanation: 男の人は「これから駅で手続きしてから帰ろうと思います」と言っています。「これから」という言葉から、これから定期券の変更手続きをすることが分かります。選択肢1の「そのまま家に帰る」は、まだ帰らないことから不正解です。選択肢3の「会社に戻る」も、会話の流れからその予定がないため不正解です。
    - topic: おすすめを尋ねる
      result:
        question: 女の人はこれからどうしますか。
        conversation:
        - gender: female
          context: この近くで昼ごはんを食べたいんですが、どこかおすすめのお店はありますか。
        - gender: male
          context: そうですね、駅の前に新しいカフェができましたよ。ランチメニューもいろいろあって、おいしいですよ。
        - gender: female
          context: そうなんですね。ありがとうございます。行ってみます。
        suggestion: 1. 駅の前のカフェに行く。
        explanation: この問題のポイントは、「おすすめのお店」を尋ねるやりとりの中で、女の人が「行ってみます」と答えている点です。N3レベルの文法・語彙（「おすすめ」「カフェ」「ランチメニュー」「行ってみます」など）を理解し、会話の流れから次の行動を推測する必要があります。選択肢2や3は会話の内容と一致しません。会話を正確に聞き取り、意図をつかむことが主なチャレンジです。
    - topic: 本について話す
      result:
        question: 男の人がこれからすることとして最もよいものを一つ選びなさい。
        conversation:
        - gender: female
          context: 田中さん、最近何か面白い本を読みましたか？
        - gender: male
          context: はい、先週「星の王子さま」を読みました。とても感動しました。
        - gender: female
          context: そうなんですね。その本について、今度の読書会で話してもらえませんか？
        - gender: male
          context: はい、ぜひ話したいです。今日はそのために、もう一度本を読んでおこうと思います。
        suggestion: 本をもう一度読む
        explanation: 男の人は「今日はそのために、もう一度本を読んでおこうと思います」と言っています。「これからすること」を聞いているので、選択肢2が正解です。発表するのは「今度の読書会」であり、友達に貸す話は出ていません。ポイントは「〜ておく」「〜と思います」などの意志表現の理解です。
    - topic: ペットについて話す
      result:
        question: 会話を聞いて、次に男の人が何をするか、最もよいものを一つ選びなさい。
        conversation:
        - gender: female
          context: 田中さん、犬を飼っているって聞いたんですけど、今日の夕方はどうしますか？
        - gender: male
          context: はい、仕事が終わったらすぐに家に帰って、犬の散歩に行くつもりです。
        - gender: female
          context: そうなんですね。毎日散歩しているんですか？
        - gender: male
          context: はい。犬も楽しみにしているので、できるだけ行くようにしています。
        suggestion: 3. 家に帰って犬の散歩に行く
        explanation: この問題のポイントは、「仕事が終わったらすぐに家に帰って、犬の散歩に行くつもりです。」という男の人の発言内容を正確に聞き取れるかどうかです。N3レベルの語彙や文法を使っていますが、「つもりです」や「できるだけ」など、意志や頻度を表す表現が理解できているかが問われます。また、「犬の散歩に行く」という日常的な行動を正しく選択肢から選ぶことも重要です。
    - topic: ファッションとスタイルについて話す
      result:
        question: この後、女の人は何をするつもりですか。
        conversation:
        - gender: female
          context: 最近、ファッションとスタイルの違いについてよく考えているんだ。
        - gender: male
          context: どうして？
        - gender: female
          context: 友達が「ファッションは流行だけど、スタイルはその人らしさだよ」って言ってて、私も自分のスタイルを大切にしたいなと思って。
        - gender: male
          context: それで、これから何をするつもり？
        suggestion: 2. 自分に合ったスタイルを探してみる
        explanation: 女の人は「自分のスタイルを大切にしたい」と話しているので、「自分に合ったスタイルを探してみる」が最も会話の流れに合っています。選択肢1は流行を追う行動、選択肢3は相談する行動ですが、会話の内容からは「自分に合ったスタイルを見つけたい」という意志が読み取れるため、2が正解です。この問題の主なポイントは、「ファッション」と「スタイル」の違いを理解し、意志表現（～したい、～つもり）を正しく聞き取ることです。また、選択肢の違いを正確に判断する力も問われます。
    - topic: 地域のイベントについて話す
      result:
        question: このあと、女性は何をしますか。
        explanation: この問題の主なポイントは、会話の最後で女性が「九時半ごろに駅で待ち合わせしましょうか」と提案している点です。N3レベルでは、会話の内容を正確に理解し、登場人物の次の行動を判断する力が求められます。選択肢2や3は会話からの誤解を誘うものですが、正解は「九時半に駅に行く」です。細かい時間や場所の情報を正確に聞き取ることが、学習者にとっての主なチャレンジとなります。
        suggestion: この問題は会話の内容理解と登場人物の行動予測を問う良問です。今後もこうした形式で、時間や場所の情報を正確に把握する練習問題を増やすと良いでしょう。
        conversation:
        - gender: male
          context: 今度の土曜日、駅前の広場で地域のイベントがあるそうですよ。行ってみませんか。
        - gender: female
          context: 面白そうですね。どんなイベントなんですか。
        - gender: male
          context: 地元の野菜やお菓子の販売があって、子ども向けのゲームコーナーもあるらしいです。
        - gender: female
          context: へえ、家族で楽しめそうですね。何時から始まるんですか。
        - gender: male
          context: 午前10時からだそうです。早めに行かないと混むかもしれませんね。
        - gender: female
          context: では、九時半ごろに駅で待ち合わせしましょうか。


In [ ]:
output_data

{'sections': [{'section_title': '第1部：語彙・文法',
   'subsections': [{'subsection_title': '漢字の読み (kanji_reading)',
     'description': '8問：漢字の読み方をひらがなで答える問題。',
     'question_topics': [{'topic': '悲劇',
       'result': {'question': '悲劇 の漢字の読み方を選びなさい。',
        'choices': ['ひげき', 'かんげき', 'ひげつ', 'かなしいげき'],
        'correct_answer': 1,
        'explanation': '「悲劇」はN3レベルの語彙として重要です。「悲」は「ひ」と音読みし、「劇」は「げき」と音読みします。選択肢の「かんげき」は「感激」という別の語の読みであり、「ひげつ」は実際には「劇」の読みではありません（「げつ」は「月」の音読みです）。また、「かなしいげき」は訓読みと音読みを混ぜた不正確な読み方です。音読みの区別が重要ですので、正しい読みを覚えましょう。'}},
      {'topic': '収める',
       'result': {'question': '書類を棚に**収めた**。',
        'choices': ['おさめた', 'いれた', 'ながめた', 'おさえた'],
        'correct_answer': 1,
        'explanation': '「収める」は「中にきちんと入れる」「収納する」という意味で使われ、読み方は「おさめる」です。例文の「書類を棚に収めた」は、「書類を棚の中にきちんと入れた」という意味になり、最も自然な使い方です。他の選択肢について：- 2「いれた」は意味は近いですが、漢字「収めた」の読みではありません。- 3「ながめた」は「眺めた」（見る）で意味が異なります。- 4「おさえた」は「押さえた」（押す・抑える）で意味も読みも違います。使い分けポイント：- 収める（おさめる）：物を中に入れる、成果を得る（例：成果を収める）- 納める（おさめる）：税金や会費などを支払う、物を納入する（例：税金を納

In [ ]:
def render_to_html(sections):
    html = '<html><body>\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for qtopic in subsection.get("question_topics", []):
                html += f'<h3>{qtopic.get("topic", "")}</h3>\n'
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "question" in result:
                        html += f'<p><strong>Q: {result["question"]}</strong></p>\n'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in result:
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [ ]:
html_output = render_to_html(output_data['sections'])
display(Markdown(html_output))

<html><body>
<h1>第1部：語彙・文法</h1>
<h2>漢字の読み (kanji_reading)</h2>
<p>8問：漢字の読み方をひらがなで答える問題。</p>
<h3>悲劇</h3>
<p><strong>Q: 悲劇 の漢字の読み方を選びなさい。</strong></p>
<ul>
<li><b>ひげき</b> <span style='color:green;'>(correct)</span></li>
<li>かんげき</li>
<li>ひげつ</li>
<li>かなしいげき</li>
</ul>
<p><em>Explanation: 「悲劇」はN3レベルの語彙として重要です。「悲」は「ひ」と音読みし、「劇」は「げき」と音読みします。選択肢の「かんげき」は「感激」という別の語の読みであり、「ひげつ」は実際には「劇」の読みではありません（「げつ」は「月」の音読みです）。また、「かなしいげき」は訓読みと音読みを混ぜた不正確な読み方です。音読みの区別が重要ですので、正しい読みを覚えましょう。</em></p>
<h3>収める</h3>
<p><strong>Q: 書類を棚に**収めた**。</strong></p>
<ul>
<li><b>おさめた</b> <span style='color:green;'>(correct)</span></li>
<li>いれた</li>
<li>ながめた</li>
<li>おさえた</li>
</ul>
<p><em>Explanation: 「収める」は「中にきちんと入れる」「収納する」という意味で使われ、読み方は「おさめる」です。例文の「書類を棚に収めた」は、「書類を棚の中にきちんと入れた」という意味になり、最も自然な使い方です。他の選択肢について：- 2「いれた」は意味は近いですが、漢字「収めた」の読みではありません。- 3「ながめた」は「眺めた」（見る）で意味が異なります。- 4「おさえた」は「押さえた」（押す・抑える）で意味も読みも違います。使い分けポイント：- 収める（おさめる）：物を中に入れる、成果を得る（例：成果を収める）- 納める（おさめる）：税金や会費などを支払う、物を納入する（例：税金を納める）- 治める（おさめる）：国や集団を支配・統治する（例：国を治める）- 修める（おさめる）：学問や技術を身につける（例：学業を修める）このように、同じ「おさめる」でも使う漢字によって意味が異なるため、文脈をよく読み取ることが大切です。</em></p>
<h3>間隔</h3>
<p><strong>Q: 次の文の（　）に入る漢字の正しい読み方を選びなさい。この列車は10分の（　）で出発します。</strong></p>
<ul>
<li>かんけつ</li>
<li><b>かんかく</b> <span style='color:green;'>(correct)</span></li>
<li>けんかく</li>
<li>かんがく</li>
</ul>
<p><em>Explanation: 「間隔」は「かんかく」と読みます。似た語に「感覚（かんかく）」がありますが、意味も使い方も異なります。また、「間欠（かんけつ）」や「間隙（かんげき）」など、同じ「間」の漢字を使う語も多いので、正しい読みをしっかり覚えておきましょう。「間隔」は物や時間のあいだ、距離を表すときによく使われます。</em></p>
<h3>女優</h3>
<p><strong>Q: 彼女は有名な **女優** です。 1 じょゆう 2 じょうゆ 3 にょゆう 4 じょうゆう</strong></p>
<ul>
<li><b>じょゆう</b> <span style='color:green;'>(correct)</span></li>
<li>じょうゆ</li>
<li>にょゆう</li>
<li>じょうゆう</li>
</ul>
<p><em>Explanation: 「女優」は「じょゆう」と読みます。N3レベルでは「女（じょ）」と「優（ゆう）」の音読みを組み合わせて読む熟語が多く出題されますが、「優」を「ゆう」と読むことに慣れていない学習者も多いです。また、「女」を「じょ」と読む熟語は「女性」「女神」など他にもあるので、音読みのパターンをしっかり覚えておく必要があります。他の選択肢は実際には使われない読み方なので、消去法も有効です。</em></p>
<h3>頼る</h3>
<p><strong>Q: 今の私は友人に **頼る** しかありません。  1 たよる  2 たのる  3 たるる  4 たゆる</strong></p>
<ul>
<li><b>たよる</b> <span style='color:green;'>(correct)</span></li>
<li>たのる</li>
<li>たるる</li>
<li>たゆる</li>
</ul>
<p><em>Explanation: 「頼る」はN3レベルで頻出の動詞で、「他の人や物に助けを求める」「依存する」という意味です。読み方は「たよる」です。「たのる」と間違えやすいですが、「頼む（たのむ）」とは別の動詞なので注意が必要です。「たゆる」は「弛む」と書き意味も異なります。漢字の読みと意味の違いを正確に区別できるかがポイントです。</em></p>
<h3>休憩</h3>
<p><strong>Q: 工場では、昼の**休憩**時間にみんなが食堂に集まります。</strong></p>
<ul>
<li><b>きゅうけい</b> <span style='color:green;'>(correct)</span></li>
<li>きゅうけ</li>
<li>きゅうけん</li>
<li>きゅうけいん</li>
</ul>
<p><em>Explanation: 「休憩」はN3レベルの重要語彙で、正しい読みは「きゅうけい」です。選択肢2「きゅうけ」や3「きゅうけん」、4「きゅうけいん」は実際には使われない読み方ですが、漢字の訓読みや音読みの混同、また「憩」の字に慣れていない受験者が間違えやすいポイントです。特に「休」は「やすむ」とも読むため、訓読みと音読みの区別が問われます。</em></p>
<h3>記事</h3>
<p><strong>Q: 次の漢字語の読みとして最もよいものを、1・2・3・4から一つ選びなさい。今朝の新聞に、サッカー日本代表についての記事が載っていました。</strong></p>
<ul>
<li><b>きじ</b> <span style='color:green;'>(correct)</span></li>
<li>きし</li>
<li>ぎじ</li>
<li>ぎし</li>
</ul>
<p><em>Explanation: 「記事（きじ）」は、新聞や雑誌、ウェブサイトなどでニュースや出来事について書かれた文章を指します。JLPT N3レベルでは、読解問題や語彙問題で頻繁に登場する重要な単語です。「記」は「き」と読むことが多く、「記事」は「きじ」と読むのが正解です。似たような読み方の「議事（ぎじ）」や「技師（ぎし）」などと混同しやすいので注意しましょう。また、「記事」は実生活でもよく使われる表現であり、ニュースや出来事を伝える際に欠かせない語彙です。意味や使い方も合わせて覚えると、読解力や会話力の向上に役立ちます。</em></p>
<h3>趣味について話す</h3>
<p><strong>Q: 友人と一緒に旅行へ行き、たくさんの写真を撮りました。次の中で「写真」の正しい読み方はどれですか？</strong></p>
<ul>
<li><b>しゃしん</b> <span style='color:green;'>(correct)</span></li>
<li>しんしゃ</li>
<li>しゃじん</li>
<li>しんじゃ</li>
</ul>
<p><em>Explanation: 「写真」はJLPT N3公式出題範囲（新完全マスターN3漢字・N3語彙など）にも含まれており、日常会話や趣味の話題で頻出する重要語彙です。この問題では「旅行」「友人」「撮る」などの文脈から、自然に「写真＝しゃしん」と読めるかどうかを確認します。他の選択肢は実際の日本語では使われない読みや、意味が異なる語（例：「しんじゃ」は「信者」）を含め、注意深く選ぶ力を養います。また、写真撮影は日本でも人気の趣味であり、面接や自己紹介の話題にもよく登場します。普段から例文を音読したり、写真に関する語彙（撮影・写す・思い出など）もあわせて覚えると、語彙力がさらに強化されます。</em></p>
<h2>漢字の書き取り (write_chinese)</h2>
<p>6問：ひらがなで書かれている言葉を漢字で書く問題。</p>
<h3>天気の状況について話す</h3>
<p><strong>Q: 今日は朝から空が（　）ので、気持ちがいいです。</strong></p>
<ul>
<li><b>晴れる</b> <span style='color:green;'>(correct)</span></li>
<li>清める</li>
<li>静まる</li>
<li>濡れる</li>
</ul>
<p><em>Explanation: 「はれる」は天気がよくなる、雲がなくなって青空になることを表し、正しい漢字は「晴れる」です。2.「清める（きよめる）」は「きれいにする」という意味で、天気には使いません。3.「静まる（しずまる）」は「うるさいものがおさまる」時などに使う言葉です。4.「濡れる（ぬれる）」は「水や液体でぬれる」ことを表し、天気が良くなる意味ではありません。似た読みや意味に惑わされず、文脈と意味から正しい漢字を選ぶことが大切です。</em></p>
<h3>ぼうけん（冒険）</h3>
<p><strong>Q: 次の文の（　）に入る「ぼうけん」の正しい漢字表記を1～4から一つ選びなさい。子どもたちは森で（　）を始めることにしました。</strong></p>
<ul>
<li><b>冒険</b> <span style='color:green;'>(correct)</span></li>
<li>冒険者</li>
<li>冒険心</li>
<li>冒険談</li>
</ul>
<p><em>Explanation: この問題のポイントは、「ぼうけん」という言葉が様々な熟語に使われることですが、文脈上「森で冒険を始める」は「冒険（adventure）」が最も適切です。2の「冒険者」は「冒険する人」、3の「冒険心」は「冒険したい気持ち」、4の「冒険談」は「冒険についての話」を意味します。選択肢の違いを理解し、正しい語彙を文脈に合わせて選ぶ力が求められます。</em></p>
<h3>きじ（記事）</h3>
<p><strong>Q: この きじ はとてもおもしろかったです。</strong></p>
<ul>
<li>生地</li>
<li><b>記事</b> <span style='color:green;'>(correct)</span></li>
<li>木地</li>
<li>基地</li>
</ul>
<p><em>Explanation: 「きじ」には複数の漢字表記があり、意味によって使い分けられます。生地（きじ）は布地や素材、木地（きじ）は木の表面、基地（きち）は軍事や活動の拠点を指します。一方、新聞や雑誌で書かれた文章やニュースを意味する「きじ」は「記事」と書きます。N3レベルでは同音異義語の区別が重要なポイントであり、文脈から正しい漢字を選ぶ力が問われます。</em></p>
<h3>たよる（頼る）</h3>
<p><strong>Q: （　　　）に入る最も適切な漢字を、1～4の中から一つ選びなさい。困ったときは友人に（　　　）ことが多いです。</strong></p>
<ul>
<li><b>頼る</b> <span style='color:green;'>(correct)</span></li>
<li>頼む</li>
<li>頼み</li>
<li>頼れ</li>
</ul>
<p><em>Explanation: この問題は、動詞「たよる」の正しい漢字表記とその活用形を問うものです。「頼る」は「他人や何かに助けを求めて依存する」という意味で、文脈に最も合っています。2の「頼む」は「お願いする」という意味で、文法的にも意味的にも合いません。3の「頼み」は名詞形（例：頼みがある）で、動詞としては使えません。4の「頼れ」は可能形（例：頼れる人）ですが、「ことが多い」という表現には使えません。N3学習者には、動詞の原形と活用形、意味の違いを正確に理解することが重要なポイントとなります。</em></p>
<h3>じょゆう（女優）</h3>
<p><strong>Q: 彼女は有名な じょゆう です。</strong></p>
<ul>
<li>女性</li>
<li><b>女優</b> <span style='color:green;'>(correct)</span></li>
<li>女役</li>
<li>女星</li>
</ul>
<p><em>Explanation: この問題のポイントは、「じょゆう（女優）」という言葉の正しい漢字表記を選ぶことです。「女優」は「女性の俳優」という意味で、映画やドラマなどで演技をする女性を指します。1の「女性」は「じょせい」と読み、女性一般を指します。3の「女役」は「おんなやく」と読み、役の種類を表しますが、職業名としては使いません。4の「女星」は中国語圏でよく使われる表現で、日本語では一般的ではありません。N3レベルでは、似たような意味や漢字が並ぶことで混乱しやすいですが、「女優」という正しい熟語を覚えておくことが大切です。</em></p>
<h3>きょうちょう（協調）</h3>
<p><strong>Q: チームで仕事をするときは、みんなで きょうちょう することが大切です。</strong></p>
<ul>
<li>協定</li>
<li><b>協調</b> <span style='color:green;'>(correct)</span></li>
<li>協力</li>
<li>強調</li>
</ul>
<p><em>Explanation: この問題の主な難しさは、似た意味や読み方を持つ漢字語が選択肢に並んでいる点です。「協調（きょうちょう）」は、意見や立場の異なる人が協力し合い、調和を図ることを意味します。1の「協定」は「きょうてい」と読み、取り決めや合意の意味です。3の「協力（きょうりょく）」は一緒に力を合わせる意味で、「協調」と似ていますが、文脈的に「協調」が最も適切です。4の「強調（きょうちょう）」は「力を入れて目立たせる」という意味で、読みは同じですが意味が異なります。このように、読みや意味が似ている語の区別がN3学習者にとってのポイントです。</em></p>
<h2>語彙の意味選択 (word_meaning)</h2>
<p>11問：与えられた単語の意味に最も近いものを選ぶ。</p>
<h3>協調</h3>
<p><strong>Q: 新しいプロジェクトを始めるには、みんなの「きょうちょう」が必要です。</strong></p>
<ul>
<li><b>協調</b> <span style='color:green;'>(correct)</span></li>
<li>協調性</li>
<li>協定</li>
<li>協力</li>
</ul>
<p><em>Explanation: 「きょうちょう」は協調と書き、意見や立場の違う人たちが調和をはかりながら協力することを意味します。選択肢2「協調性」は、性格や能力を表す語で、この文脈では不適切です。選択肢3「協定」は、国や団体などが結ぶ約束事です。選択肢4「協力」は、単に力を合わせることですが、調和というニュアンスが足りません。N3レベルでは「協調」と「協力」の違いが特に混同されやすいので注意しましょう。</em></p>
<h3>睡眠</h3>
<p><strong>Q: 最近、夜遅くまで仕事をしていたので、すいみんが不足しています。</strong></p>
<ul>
<li><b>睡眠</b> <span style='color:green;'>(correct)</span></li>
<li>水面</li>
<li>推進</li>
<li>随分</li>
</ul>
<p><em>Explanation: この問題のポイントは、「すいみん」という読みの正しい漢字表記を選ぶことです。・「睡眠（すいみん）」は「ねむること、ねむり」の意味で、N3レベルの重要語彙です。・「水面（すいめん）」は「みずのうえ」という意味で、読みは似ていますが意味が異なります。・「推進（すいしん）」は「物事をすすめること」という意味で、漢字や音が似ているため混同しやすいですが、意味が異なります。・「随分（ずいぶん）」は「かなり、非常に」という意味で、読みが似ているためのひっかけです。このように、選択肢には実際に使われる熟語を入れ、受験者が漢字の意味・読み・使い方を総合的に判断できるようにしています。JLPT N3レベルでは、似た音や漢字の語彙を区別する力が求められます。</em></p>
<h3>子供の教育について話す</h3>
<p><strong>Q: 子供の教育について話すとき、「きょういく」という言葉がよく使われます。次の文の（　）に入る正しい漢字を選びなさい。最近、親同士で子供の（きょういく）について話す機会が増えました。</strong></p>
<ul>
<li><b>教育</b> <span style='color:green;'>(correct)</span></li>
<li>教域</li>
<li>教欲</li>
<li>教浴</li>
</ul>
<p><em>Explanation: 「きょういく」の正しい漢字は「教育」です。N3レベルではよく出る重要な語彙ですが、似た音の漢字（域＝いき、欲＝よく、浴＝あびる）と混同しやすいため、正確に覚える必要があります。特に「育」は「育てる」と関連づけて覚えると良いでしょう。</em></p>
<h3>体温</h3>
<p><strong>Q: 風邪をひいたので、今朝は（たいおん）をはかりました。</strong></p>
<ul>
<li>体音</li>
<li><b>体温</b> <span style='color:green;'>(correct)</span></li>
<li>耐温</li>
<li>体恩</li>
</ul>
<p><em>Explanation: この問題の難しさは、「たいおん」という読みの漢字表記を正しく選ぶことにあります。「体音」や「耐温」など、見た目が似ている漢字を使った選択肢が紛らわしいですが、「たいおん（体温）」は「体（からだ）」と「温（あたたかい）」の組み合わせで、「身体の温度」という意味になります。他の選択肢は意味や読みが異なるので注意が必要です。N3レベルでは、日常生活や健康に関する基本的な語彙として「体温」は押さえておきたい単語です。</em></p>
<h3>販売</h3>
<p><strong>Q: 新しいスマートフォンの（はんばい）が、全国の家電量販店で始まりました。</strong></p>
<ul>
<li><b>販売</b> <span style='color:green;'>(correct)</span></li>
<li>発売</li>
<li>販売</li>
<li>飯売</li>
</ul>
<p><em>Explanation: 「はんばい」は「販売」と書き、商品やサービスを売ることを意味します。N3レベルでは「販売」と「発売」の違いも問われやすいです。2の「発売」は「売り出しを始めること」で、「販売」は「実際に売る活動全般」を指します。3の「販売」は「販」の後ろに「賣（旧字体）」を使っており、現代日本語では使いません。4の「飯売」は意味が異なります（「飯」は「ご飯」）。特に「発売」と「販売」は混同しやすいので、文脈から「売り始める」ではなく「売ること全般」を指す「販売」が正解となります。</em></p>
<h3>様子</h3>
<p><strong>Q: 最近、田中さんのようすが少し変だと感じます。</strong></p>
<ul>
<li><b>様子</b> <span style='color:green;'>(correct)</span></li>
<li>洋子</li>
<li>様式</li>
<li>状子</li>
</ul>
<p><em>Explanation: 「ようす」は「様子」と書きます。「様子」は人や物事の状態・ありさまを表すN3レベルの重要語彙です。選択肢2「洋子」は人名でよく使われますが、意味が異なります。3「様式」は「スタイル」「形式」を表し、「ようしき」と読みます。4「状子」は実際には使われない組み合わせで、誤答としてよく出されるパターンです。この問題の主な難しさは、「様」「子」「式」「状」など、形や意味が似ている漢字を区別できるかを問う点です。漢字の正しい組み合わせを覚えているかどうかが重要です。</em></p>
<h3>改善</h3>
<p><strong>Q: 最近、会社では仕事の流れをもっとよくするために（　　　）活動が行われている。</strong></p>
<ul>
<li><b>改善</b> <span style='color:green;'>(correct)</span></li>
<li>改選</li>
<li>改造</li>
<li>改築</li>
</ul>
<p><em>Explanation: 「改善（かいぜん）」は「悪いところを直して、より良くすること」という意味で、ビジネスシーンや日常生活でもよく使われる言葉です。例：「生活を改善する」「業務改善」。他の選択肢はすべて「改」の字を含みますが、意味が異なります。- 「改選（かいせん）」は、役員などを選び直すこと。- 「改造（かいぞう）」は、物や建物などを作り変えること。- 「改築（かいちく）」は、建物を建て直すこと。読みや漢字が似ているため混同しやすいですが、「善」は「よい、よくする」という意味を持つので、「改善」で「より良くする」という意味になります。使用例：・生活習慣を改善したいです。・会社の業務を改善するために会議を開きました。教師の視点からのポイント：N3レベルでは、同じ「改」を使った熟語の意味や使い分けが課題となります。選択肢の漢字はどれもよく使われるため、正しい意味と使い方をしっかり覚えることが大切です。</em></p>
<h3>材料</h3>
<p><strong>Q: 次の文の（　）に入る正しい漢字を、1～4の中から一つ選びなさい。新しい機械を作るために、いろいろな（　）を集めています。</strong></p>
<ul>
<li><b>材料</b> <span style='color:green;'>(correct)</span></li>
<li>材質</li>
<li>素材</li>
<li>原料</li>
</ul>
<p><em>Explanation: 「材料（ざいりょう）」は、物を作るためのもとになるものを表します。例えば、家具を作る時の木や金属、料理を作る時の肉や野菜などが「材料」です。- 「材質（ざいしつ）」は、そのものの性質や品質を指します。例：「この机の材質は木です。」- 「素材（そざい）」は、加工されていない自然のままのものを指します。例：「この服の素材はコットンです。」- 「原料（げんりょう）」は、主に工業や化学、食品などで最初に使うもとの物質を指します。例：「チョコレートの原料はカカオ豆です。」このように、似ている言葉ですが使い分けが必要です。文脈によって最もふさわしい漢字を選びましょう。ポイントの強調例：- 材料：物を作るためのもとになるもの（正解）- 材質：ものの性質・品質- 素材：加工前のもとのもの- 原料：製品のもとになる物質（主に工業・食品など）このように、指示文の追加や解説の視覚的な強調を加えることで、受験者がより理解しやすくなります。今後もこの形式を参考に、複数の語彙で問題を作成することをおすすめします。</em></p>
<h3>議会</h3>
<p><strong>Q: 新しい法律を決めるために、国の（ぎかい）が開かれました。</strong></p>
<ul>
<li><b>議会</b> <span style='color:green;'>(correct)</span></li>
<li>義会</li>
<li>宜会</li>
<li>儀会</li>
</ul>
<p><em>Explanation: この問題のポイントは「ぎかい」という読みの正しい漢字表記を選ぶことです。N3レベルの学習者にとって、「議」（議論・議事などでよく出る漢字）と「会」（会社・会議などでよく出る漢字）の組み合わせで「議会」となります。他の選択肢は音や意味が似ている漢字を使っていますが、意味が異なります。「義」は「正義」など、「宜」は「適宜」など、「儀」は「儀式」などで使われるため、混同しやすいですが、正しい組み合わせを選ぶ必要があります。</em></p>
<h3>天気の状況について話す</h3>
<p><strong>Q: 次の文章の（　）に入る正しい漢字を、1・2・3・4の中から一つ選びなさい。昨日は一日中「くもり」だったので、洗濯物がなかなか乾きませんでした。（　）</strong></p>
<ul>
<li><b>曇</b> <span style='color:green;'>(correct)</span></li>
<li>雲</li>
<li>霧</li>
<li>雨</li>
</ul>
<p><em>Explanation: 「くもり」は天気の状態を表す言葉で、正しい漢字は「曇」です。選択肢2の「雲」は空に浮かぶ「くも」、3の「霧」は「きり」、4の「雨」は「あめ」と、それぞれ異なる天気や現象を表します。JLPT N3レベルでは、「くもり（曇）」と「くも（雲）」の区別が混同しやすいので注意が必要です。また、例文の「乾きませんでした」のように、天気と日常生活（洗濯物が乾かないこと）を結びつける表現は、日本語の実用的な運用力を測る良い出題となります。教師からのポイント：この問題の主な難しさは、「曇」と「雲」の漢字の意味の違いを理解し、文脈から適切な語を選ぶ点です。天気に関する表現は会話やニュースなどでよく使われるため、正しい漢字表記を身につけておくことが大切です。</em></p>
<h3>流れ</h3>
<p><strong>Q: 次の文の（　）に入る正しい漢字を選びなさい。川の（ながれ）が速くて、泳ぐのは危険です。</strong></p>
<ul>
<li>流木（りゅうぼく）</li>
<li>流星（りゅうせい）</li>
<li><b>流れ（ながれ）</b> <span style='color:green;'>(correct)</span></li>
<li>流行（りゅうこう）</li>
</ul>
<p><em>Explanation: この問題は「ながれ」の正しい漢字表記を問うものです。選択肢にはすべて「流」の漢字が含まれており、どれも実在する日本語の単語です。しかし、「川の流れが速い」という文脈に合うのは「流れ」だけです。他の選択肢は意味が異なります（流木：川などに流れている木、流星：流れ星、流行：はやり）。N3レベルでは、意味と漢字表記の正しい組み合わせを理解しているかが問われます。似ている言葉が並んでいるため、意味の違いをしっかり区別できるかがポイントです。</em></p>
<h2>同意語置き換え (synonym_substitution)</h2>
<p>5問：文中の語を同じ意味の語に置き換える問題。</p>
<h3>意外</h3>
<p><strong>Q: 昨日のニュースで、彼が会社を辞めると聞いて、このニュースはとても意外（いがい）でした。</strong></p>
<ul>
<li>普通（ふつう）</li>
<li><b>予想外（よそうがい）</b> <span style='color:green;'>(correct)</span></li>
<li>安全（あんぜん）</li>
<li>重要（じゅうよう）</li>
</ul>
<p><em>Explanation: ・「意外（いがい）」は、「思っていたことと違っていること」や「予期しなかったこと」という意味です。・「予想外（よそうがい）」は、「予想していなかったこと」「思いがけないこと」という意味で、「意外」と最も近い意味を持っています。・他の選択肢について：　- 「普通（ふつう）」は「一般的」「特別ではない」という意味です。　- 「安全（あんぜん）」は「危険がないこと」を表します。　- 「重要（じゅうよう）」は「大切なこと」を意味します。・この問題のポイントは、「意外」と「予想外」の意味の近さを理解し、文脈から正しい語を選べるかどうかです。・「意外」と「予想外」はどちらも「unexpected」という意味ですが、日常会話では「意外」の方が使われやすいです。・選択肢の意味をしっかり区別し、文全体の意味を正確に理解する力が求められます。</em></p>
<h3>占める</h3>
<p><strong>Q: この会社の商品は、国内市場の50％を占める。適切な意味の言葉を選びなさい。</strong></p>
<ul>
<li><b>占有する</b> <span style='color:green;'>(correct)</span></li>
<li>示す</li>
<li>埋める</li>
<li>支配する</li>
</ul>
<p><em>Explanation: 「占める」は「ある範囲や割合、地位などを自分のものとする」「全体の中で一定の割合を持つ」という意味です。選択肢1の「占有する」は「場所や地位などを自分のものとして持つ」という意味で、「占める」と最も近い意味になります。選択肢2の「示す」は「見せる、表す」という意味で、「占める」とは異なります。選択肢3の「埋める」は「空いているところをふさぐ」という意味で、割合や地位を取る意味では使いません。選択肢4の「支配する」は「コントロールする、管理する」という意味で、直接的に割合や位置を取る意味ではありません。この問題の難しさは、「占める」の持つ「割合を取る」という意味を正しく理解し、それに最も近い語を選ぶ必要がある点です。選択肢の中には意味が似ている語もあるため、細かい意味の違いを理解しているかどうかが問われます。</em></p>
<h3>問い</h3>
<p><strong>Q: 次の文の「問い」と最も近い意味の言葉を選びなさい。先生は授業の終わりに新しい問いを出しました。</strong></p>
<ul>
<li><b>問題</b> <span style='color:green;'>(correct)</span></li>
<li>答え</li>
<li>説明</li>
<li>注意</li>
</ul>
<p><em>Explanation: 「問い」は「質問」「設問」「問題」などと近い意味を持ちますが、文脈によっては「質問」とも似ています。しかし、授業で「出す」のは「問題」や「設問」であることが多く、答えや説明、注意とは意味が異なります。N3レベルでは「問い」と「問題」の使い分けが難しく、特に「質問」との違いに迷う受験生もいるため、設問の意図を読み取る力が問われます。</em></p>
<h3>泊める</h3>
<p><strong>Q: 次の文の意味に最も近いものを、1～4の中から一つ選びなさい。友人が遠くから来たので、私の家に泊めることにした。</strong></p>
<ul>
<li><b>受け入れる</b> <span style='color:green;'>(correct)</span></li>
<li>止める</li>
<li>集める</li>
<li>送る</li>
</ul>
<p><em>Explanation: 「泊める」は「他人を自分の家などに宿泊させる」という意味です。選択肢1の「受け入れる」は、人を迎えて家の中に入れてあげる、世話をするという意味があり、「泊める」と近い意味になります。一方、「止める」は動いているものを止める、「集める」はたくさんのものを一か所に集める、「送る」はどこかへ行かせる、という意味であり、文脈に合いません。また、「とめる」と読む漢字には「止める（動きをやめる）」「留める（位置を保つ・残す）」「停める（車などを止める）」などがあり、意味が異なります。「泊める」は「宿泊させる」という意味で使われるので、漢字の違いにも注意しましょう。この問題のポイントは、似た動詞の意味の違いをしっかり理解しているかどうかです。誤答選択肢も日常でよく使う動詞なので、文脈から最も適切な語を選ぶ力が求められます。</em></p>
<h3>海外旅行の体験を話す</h3>
<p><strong>Q: 海外旅行の体験を話すときは、できるだけ具体的に説明すると、聞いている人に伝わりやすくなります。この文の「説明する」と意味が近い言葉を選びなさい。</strong></p>
<ul>
<li><b>伝える</b> <span style='color:green;'>(correct)</span></li>
<li>遊ぶ</li>
<li>笑う</li>
<li>集める</li>
</ul>
<p><em>Explanation: 「説明する」は「物事を分かりやすく話す、伝える」という意味です。選択肢2「遊ぶ」、3「笑う」、4「集める」は意味が異なります。1「伝える」が最も近い意味を持つ同義語です。N3レベルでは「説明」と「伝える」の違いを文脈で判断する力が求められます。「説明」は詳細に話すこと、「伝える」は情報や気持ちを相手に届けることですが、ここでは「体験を話す」＝「伝える」と自然に置き換えられるため、紛らわしさがあります。distractor（誤答肢）も日常的な動詞なので注意が必要です。</em></p>
<h2>語彙の使い方 (word_usage)</h2>
<p>5問：語彙の使い方が正しい文を選ぶ。</p>
<h3>手術</h3>
<p><strong>Q: 次の文の中で、「手術」の使い方として最も適切なものを一つ選びなさい。</strong></p>
<ul>
<li>祖母は病気が進んでしまったので、来週病院で手術を受けることになりました。</li>
<li>彼は新しいパソコンを買ったので、古いパソコンの手術をしてデータを移しました。</li>
<li>昨日、料理中に手を切ってしまい、少し手術をしました。</li>
<li>旅行の前に部屋の手術をして、きれいに片付けました。</li>
</ul>
<p><em>Explanation: 「手術」とは、医師がメスや特別な道具を使って、病気やけがを治すために体を切ったり治療したりする医療行為です。Aは医療の場面で正しい使い方です。BやDのように、パソコンや部屋など人や体以外のものには使いません。Cはけがの手当てをした場合、「手術」ではなく「消毒」や「ばんそうこうを貼った」などと言います。この問題のポイントは、「手術」が医療の専門的な行為であり、日常の作業や簡単なけがの手当てには使わないことを理解できるかどうかです。N3レベルの学習者にとって、語彙の意味と使い方の正確な区別が重要です。</em></p>
<h3>哀れ</h3>
<p><strong>Q: 次の文の中で、「哀れ」の使い方として最も適切なものを一つ選びなさい。</strong></p>
<ul>
<li><b>この映画の主人公は、家族を失って一人で生きる哀れな少年です。</b> <span style='color:green;'>(correct)</span></li>
<li>長い間手入れされず、哀れな景色が広がっていた。</li>
<li>昨日の天気は哀れで、みんな外に出られませんでした。</li>
<li>この料理は哀れな味がして、あまりおいしくありません。</li>
</ul>
<p><em>Explanation: 「哀れ」は「かわいそう」「気の毒」といった意味で、主に人や動物など、感情や同情の対象となるものに使います。1番の文は、家族を失った少年について「哀れな」と表現しており、正しい使い方です。2番の「哀れな景色」は、景色に対して「哀れ」を使うのはやや不自然で、通常は「寂しい」「荒れた」などを使います。3番の「天気が哀れ」、4番の「哀れな味」も誤用です。天気や味など、物や抽象的なものには「哀れ」は使いません。類義語には「かわいそう」「気の毒」「不憫」などがあり、対義語には「立派」「堂々」などがあります。「哀れ」は人や動物など、同情や悲しみを感じる対象に使うことを意識しましょう。</em></p>
<h3>動詞</h3>
<p><strong>Q: 次の文の（　　）に入る最も適切な動詞を1つ選びなさい。Aさんは毎朝6時に（　　　）、コーヒーを飲んでから会社に行きます。</strong></p>
<ul>
<li><b>起きます</b> <span style='color:green;'>(correct)</span></li>
<li>泳ぎます</li>
<li>住みます</li>
<li>使います</li>
</ul>
<p><em>Explanation: この問題は、日常生活の流れと動詞の意味・使い方を理解しているかどうかを問うものです。「毎朝6時に（　　　）、コーヒーを飲んでから会社に行きます。」という文脈では、朝の行動として最も自然なのは「起きます（おきます）」＝「to get up」です。他の選択肢「泳ぎます（およぎます）」＝「to swim」、「住みます（すみます）」＝「to live」、「使います（つかいます）」＝「to use」は、この文脈では不自然です。動詞の基本的な意味と日常的な使い方、そして文脈に合うかどうかを判断する力が問われる点が学習者にとっての主なチャレンジです。</em></p>
<h3>好み</h3>
<p><strong>Q: 次の文の中から、最も自然で適切な文を一つ選びなさい。</strong></p>
<ul>
<li>彼は私の好みのタイプです。</li>
<li><b>この店は、好みの服がたくさん売っています。</b> <span style='color:green;'>(correct)</span></li>
<li>好みを言えば、もう少し甘いほうが好きです。</li>
<li>好みの問題だから、どちらを選んでもいいですよ。</li>
</ul>
<p><em>Explanation: 「好み」は「好きなものの傾向」や「自分が良いと思うもの」という意味で使われます。特に「好みの＋名詞」の形（例：好みの服）は、N3レベルでよく出題される典型的なコロケーションです。2番の文は「自分の好きなタイプの服」という意味になり、最も自然な使い方です。他の選択肢について、1番も自然な表現ですが、2番の方が「好みの＋名詞」としてより基本的です。3番の「好みを言えば」は「もし自分の好みを伝えるなら」という意味で使われ、やや抽象的な表現です。4番の「好みの問題」は「個人の好き嫌いによる問題」という慣用的表現ですが、2番と比べると、N3レベルの基本的な使い方としてはやや難易度が高いです。参考例文：「好みの問題と言われればそれまでですが、私はこのレストランの内装があまり好きではありません。」「好みを言えば、もう少し辛いほうがいいです。」主なポイント・難しさ：「好み」には複数の意味や使い方があり、文脈によって使い分ける必要があります。「好みの＋名詞」や「好みを言えば」など、決まったコロケーション・表現があるため、正しく使えるかどうかがポイントです。</em></p>
<h3>キャリア目標について話す</h3>
<p><strong>Q: 次の語句の使い方として最も適切な文を一つ選びなさい。内容: 1. 修理のため、エアコンの内容を一度取り出します。 2. 鍋の中にカレーの内容を入れて、1時間くらい煮てください。 3. 古い財布から新しい財布へ内容を移しました。 4. この手紙の内容は、ほかの人には秘密にしてください。</strong></p>
<ul>
<li>修理のため、エアコンの内容を一度取り出します。</li>
<li>鍋の中にカレーの内容を入れて、1時間くらい煮てください。</li>
<li>古い財布から新しい財布へ内容を移しました。</li>
<li><b>この手紙の内容は、ほかの人には秘密にしてください。</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: この問題は、語彙の意味だけでなく、自然な使い方（コロケーション）を問うものです。 「内容」は「手紙の内容」など、情報や中身を表すときに使います。</em></p>
<h2>文法穴埋め (sentence_grammar)</h2>
<p>13問：文中の空欄に当てはまる文法表現を選ぶ。</p>
<h3>年齢</h3>
<p><strong>Q: （自己紹介の場面）A：「はじめまして。田中と申します。Bさんはおいくつですか。」B：「私は今年、二十歳（　　　）なります。」</strong></p>
<ul>
<li>が</li>
<li>を</li>
<li><b>に</b> <span style='color:green;'>(correct)</span></li>
<li>で</li>
</ul>
<p><em>Explanation: 「〜に なる」は、年齢や状態が変化するときに使われる表現です。たとえば、「学生になります」「先生になります」「二十歳になります」などのように、変化の到達点を表すときは「に」を使います。他の助詞「が」「を」「で」はこの文脈では使えません。N3レベルの学習者は母語の影響で誤った助詞を選びやすいため、特に注意が必要です。</em></p>
<h3>高まる</h3>
<p><strong>Q: 次の文の（　　　）に入る最も適切なものを、1～4の中から一つ選びなさい。最近、世界的に環境問題への関心が（　　　）、多くの国が協力して対策を進めています。</strong></p>
<ul>
<li><b>高まっている</b> <span style='color:green;'>(correct)</span></li>
<li>高めている</li>
<li>高められている</li>
<li>高まらせている</li>
</ul>
<p><em>Explanation: 「関心が高まる」は「関心の程度が強くなる」という意味で、自動詞「高まる」を使います。「高まっている」は、その状態が続いていることを表します。他の選択肢は他動詞や受け身・使役形であり、この文には合いません。例文：環境問題に対する関心は年々高まっています。</em></p>
<h3>仲間</h3>
<p><strong>Q: （サークル活動にて）A：「来月のイベントの準備、大変だね。」B：「うん。でも、（　）と一緒なら頑張れる気がするよ。」</strong></p>
<ul>
<li><b>仲間</b> <span style='color:green;'>(correct)</span></li>
<li>友達</li>
<li>家族</li>
<li>先生</li>
</ul>
<p><em>Explanation: この問題のポイントは、「一緒に何かをする人」や「共通の目的に向かって協力する人」を指す最も適切な語彙を選ぶことです。「仲間」はN3レベルでよく使われる語彙であり、サークル活動やイベントの準備など、共通の目的を持つ集団にぴったりの言葉です。「友達」も親しい人を指しますが、目的を共有して協力するニュアンスが弱くなります。「家族」「先生」は文脈に合いません。学習者は「仲間」と「友達」の違いを文脈で判断するのが難しい場合が多いので、意味の違いに注意しましょう。</em></p>
<h3>冷ます</h3>
<p><strong>Q: （食堂にて）A：「このスープ、まだ熱いね。」B：「うん、少し（　　　　）から飲もう。」※（　　　　）に入る最も適切な動詞のて形を答えなさい。</strong></p>
<ul>
<li>冷まして</li>
</ul>
<p><em>Explanation: 「冷ます」は他動詞で、「熱いものの温度を下げる」という意味です。この場面では、スープを飲む前に「冷まして」から飲む、という流れになります。「冷める」は自動詞で、自然に温度が下がる場合に使いますが、ここでは人が意図的に行う動作なので「冷まして」が正解です。N3レベルでは自動詞と他動詞の区別がよく問われるため、注意が必要です。</em></p>
<h3>節約</h3>
<p><strong>Q: （友人との会話） A：「最近、生活費が高くなってきたね。何か（　）方法を考えている？」 B：「うん、家計簿をつけたり、外食を減らしたりしているよ。」 （　）に入る最も適切な語句を選びなさい。</strong></p>
<ul>
<li>節約する</li>
<li>節約した</li>
<li>節約している</li>
<li><b>節約できる</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「方法」という名詞の前には、その方法がどのようなものかを説明する修飾語が入ります。「節約する方法」も文法的には正しいですが、一般的には「節約できる方法」と言うことで、「節約することができる方法」、つまり“可能性”や“実現性”を強調する自然な表現になります。「節約する方法」は、単に「節約するための方法」という意味ですが、「節約できる方法」は「実際に自分が節約することが可能な方法」というニュアンスを持ち、より実用的で会話でもよく使われます。このように、「～できる＋名詞」はJLPT N3レベルでよく問われる文法ポイントで、可能性や能力を表現する際に使われます。【追加例文】・最近は、簡単に作れる料理のレシピや、安く買い物ができるアプリなど、いろいろな（　）サービスが増えています。→　節約できる・新しいアパートに引っ越したいけど、家賃を（　）場所を探しています。→　節約できる</em></p>
<h3>身長</h3>
<p><strong>Q: ご丁寧なフィードバックをいただき、ありがとうございます。内容の正確性・問題の質・形式・文化的適切性など、多角的にご評価いただき感謝いたします。ご指摘いただいた改善点をふまえ、より良い問題例を以下にご提案いたします。---【健康診断の場面での会話】先生：「はい、次の方、こちらに来てください。まず、（　　　）を測りますので、靴を脱いでください。体重は後で測ります。」1. 体重 2. 年齢 3. 身長 4. 気分---【正解】3. 身長【解説】- 健康診断の場面で「まず、（　　　）を測ります」という指示は、身長測定が一般的です。- 「体重は後で測ります」と加えることで、「体重」と「身長」の語彙の使い分けがより明確になります。- 「身長」「体重」「年齢」「気分」はN3レベルの基本語彙です。- 「年齢」「気分」は測定の対象として不自然であるため、文脈理解力が問われます。---ご指摘いただいた点を反映し、今後もより分かりやすく、実用的な問題作成を心がけます。貴重なご意見、ありがとうございました。</strong></p>
<ul>
<li>体重</li>
<li>年齢</li>
<li><b>身長</b> <span style='color:green;'>(correct)</span></li>
<li>気分</li>
</ul>
<p><em>Explanation: 健康診断の場面で「まず、（　　　）を測ります」という指示は、身長測定が一般的です。「体重は後で測ります」と加えることで、「体重」と「身長」の語彙の使い分けがより明確になります。「身長」「体重」「年齢」「気分」はN3レベルの基本語彙です。「年齢」「気分」は測定の対象として不自然であるため、文脈理解力が問われます。</em></p>
<h3>適切</h3>
<p><strong>Q: この問題に最も（　　　）な答えを選びなさい。</strong></p>
<ul>
<li><b>適切</b> <span style='color:green;'>(correct)</span></li>
<li>適当</li>
<li>適性</li>
<li>適格</li>
</ul>
<p><em>Explanation: この文は「この問題に最も（　　　）な答えを選びなさい。」という指示文です。「適切」は、状況や目的にぴったり合っている、ふさわしいという意味で使われます。「適当」も似た意味ですが、「いいかげん」という意味もあり、試験や正式な場面では「適切」の方が正確で望ましい表現となります。「適性」は「向いている性質」、「適格」は「資格・条件に合っている」という意味なので文脈に合いません。教師からみた主な難しさ：「適切」と「適当」はN3レベルでも混同しやすい語彙です。どちらも「ふさわしい」という意味があるため、文脈によって正しい語を選ぶ必要があります。特に「適切」はフォーマルな場面や試験など、正確さが求められる状況で使われることが多いので、その使い分けを理解しているかがポイントとなります。</em></p>
<h3>契機</h3>
<p><strong>Q: 部長：「今回の新しいシステム導入を（　）、業務の効率化を図りたいと考えています。」</strong></p>
<ul>
<li><b>契機に</b> <span style='color:green;'>(correct)</span></li>
<li>契機で</li>
<li>契機を</li>
<li>契機から</li>
</ul>
<p><em>Explanation: この文では「～を契機に」という表現を使います。「契機に」は「何かをきっかけとして」「何かを機会として」という意味で、物事の変化や新しいことを始める直接的な原因やきっかけを表します。他の選択肢は文法的に自然ではありません。「契機に」はN3レベルでもよく使われる表現ですが、漢語的でややフォーマルな響きがあるため、「きっかけに」との使い分けが難しい点が主な課題です。特に「契機で」「契機を」「契機から」など、助詞の誤用が多く見られるため、正しいコロケーションを覚えることが重要です。</em></p>
<h3>技術について話す</h3>
<p><strong>Q: 次の会話を読んで、（　）に入る最も適切な語を選びなさい。（会議室にて）A：「今回のプロジェクトでは、新しい技術（　）話す必要があります。」B：「はい、分かりました。分かりやすく説明します。」</strong></p>
<ul>
<li>を</li>
<li><b>について</b> <span style='color:green;'>(correct)</span></li>
<li>で</li>
<li>が</li>
</ul>
<p><em>Explanation: 「〜について話す」は「〜の内容を話す、〜に関して話す」という意味で使われるN3レベルの重要表現です。「技術を話す」は日本語では不自然な表現になります。この問題の主な難しさは、「を話す」と「について話す」の違いを理解することです。「について」を使うことで、「新しい技術に関して話す」という正しい意味になります。N3レベルの学習者にとって、「について」の使い方は頻出で大切なポイントですので、しっかり身につけておきましょう。</em></p>
<h3>正午</h3>
<p><strong>Q: 次の会話を読んで、空欄に入る最も適切な語句を1つ選びなさい。（昼の時間帯、時計を見ながら）A：「今、何時ですか。」B：「もうすぐ（　　　）ですよ。」</strong></p>
<ul>
<li>午前</li>
<li>午後</li>
<li><b>正午</b> <span style='color:green;'>(correct)</span></li>
<li>深夜</li>
</ul>
<p><em>Explanation: この問題は、日常会話でよく使われる時間に関する語彙の理解を問うものです。「もうすぐ（　　　）ですよ。」という発言は、昼の12時が近い状況を想定しています。「正午」は「昼の12時」を意味し、時間帯を表す「午前（夜中～昼前）」「午後（昼～夜）」「深夜（夜中）」とは異なります。N3レベルでは「正午」と「午後」の区別が難しいことが多いですが、「正午」は特定の時刻（12時ちょうど）を指し、「午後」は12時以降の時間帯全体を指します。したがって、ここでは「正午」が最も適切です。主なチャレンジポイントは、「正午」と「午後」「午前」の意味の違いを正確に理解できているかどうか、会話の状況（昼の時間帯、12時が近いこと）を文脈から推測できるかどうかです。</em></p>
<h3>信用</h3>
<p><strong>Q: （会社の会議にて）部長：「新しい取引先と契約する前に、その会社の（　）をしっかり調べておく必要があります。」（　）に入る最も適切な語を1〜4の中から選びなさい。</strong></p>
<ul>
<li><b>信用</b> <span style='color:green;'>(correct)</span></li>
<li>信号</li>
<li>信号機</li>
<li>信念</li>
</ul>
<p><em>Explanation: この文では、「その会社の（　）をしっかり調べておく必要があります。」とあります。「信用（しんよう）」は、「他人や会社などを信じて受け入れること、または信頼できると判断すること」という意味です。取引や契約の前に、相手の信頼性や過去の実績などを調べる場合によく使います。「信念（しんねん）」は「自分が正しいと信じている考えや価値観」のことで、会社の信頼性を調べる文脈では使いません。「信号（しんごう）」や「信号機（しんごうき）」は、どちらも交通に関する言葉であり、この文脈には合いません。N3レベルでは、「信用」と「信念」など、意味が異なるが漢字や音が似ている語彙の区別が難しいです。また、「信号」など全く違う意味の語も選択肢にあるため、文脈をしっかり読む力が必要です。「信用」はビジネスや日常生活でもよく使う重要語彙なので、使い方を正確に理解しましょう。</em></p>
<h3>個人的な成果について話す</h3>
<p><strong>Q: （会社の面談にて）上司：「最近のプロジェクトで、何か個人的に（　）ことはありましたか。」</strong></p>
<ul>
<li><b>成し遂げた</b> <span style='color:green;'>(correct)</span></li>
<li>成し遂げる</li>
<li>成し遂げて</li>
<li>成し遂げます</li>
</ul>
<p><em>Explanation: この文は、過去の経験や成果について尋ねる表現です。「成し遂げたことはありましたか」は「過去に何かをやり遂げた経験がありましたか」という意味になります。N3レベルでは「〜たことがある」「〜たことはありますか」など経験を表す文型がよく出題されますが、「〜たことはありましたか」はやや丁寧な言い方で、過去の特定の期間や状況に焦点を当てる場合に使われます。「成し遂げた（過去形）」以外の活用形（辞書形、て形、ます形）はこの文脈では不自然です。主なチャレンジポイントは、「〜たことはありましたか」と「〜たことがありますか」のニュアンスの違い（前者は過去のある期間や状況に焦点、後者は一般的な経験）、適切な動詞の活用（過去形）が使えるかどうか、ビジネスシーンでよく使われる語彙や表現に慣れているか、です。</em></p>
<h3>音楽について話す</h3>
<p><strong>Q: （　）に入る適切な言葉を選びなさい。友人との会話 A：「最近、どんな音楽をよく（　）いますか。」 B：「最近はジャズをよく聴いています。」</strong></p>
<ul>
<li><b>聞いて</b> <span style='color:green;'>(correct)</span></li>
<li>聞き</li>
<li>聞いてる</li>
<li>聞かれて</li>
</ul>
<p><em>Explanation: この文では「～ていますか」という現在進行形・現在習慣の形を作る必要があります。「聞いていますか」が正しい形です。選択肢2「聞き」は未然形で文法的に不正確です。選択肢3「聞いてる」は話し言葉の縮約形で、日常会話ではよく使われますが、書き言葉や正式な場面では「聞いています」が望ましいです。選択肢4「聞かれて」は受け身形で意味が変わってしまいます。N3レベルでは「～ています」の使い方、動詞のて形＋いますの活用に注意しましょう。</em></p>
<h2>文の並び替え (sentence_sort)</h2>
<p>5問：バラバラになった文を正しい順序に並び替える。</p>
<h3>異なる</h3>
<p><strong>Q: 次の文の__★_ に入る最も適切な語順を、下の1～4の中から一つ選びなさい。文化が__★_ 生活習慣も違います。1.　国によって 2.　異なるため 3.　人々の 4.　それぞれ</strong></p>
<ul>
<li>4 → 1 → 2 → 3</li>
<li>1 → 4 → 2 → 3</li>
<li><b>1 → 2 → 4 → 3</b> <span style='color:green;'>(correct)</span></li>
<li>4 → 2 → 1 → 3</li>
</ul>
<p><em>Explanation: この問題の主な難しさは、「異なるため」の使い方と、その前後に来る語句の関係を正しく理解することです。「国によって異なるため、それぞれ人々の生活習慣も違います。」という流れが自然ですが、「それぞれ」と「人々の」の位置関係も間違えやすいポイントです。「異なる」はN3レベルでよく出る語で、理由を表す「ため」と接続する形が頻出です。語順を間違えると意味が通じなくなるため、正しい語順を選ぶ力が問われます。</em></p>
<h3>大会</h3>
<p><strong>Q: 山川大学では、__★__ 新入生にアンケート調査を行っている。次の語句を正しい順序に並べ替えなさい。1. 大学生活 2. に対して 3. 持っている 4. イメージ</strong></p>
<ul>
<li>1→2→3→4</li>
<li>1→4→2→3</li>
<li>2→1→3→4</li>
<li>1→3→2→4</li>
</ul>
<p><em>Explanation: この並べ替え問題の主なポイントは、「に対して」の接続先、「持っている」の修飾関係、「イメージ」の位置です。「大学生活に対して持っているイメージ」で一つの名詞句を作り、それを調査する、という流れになります。N3レベルでは「に対して」の使い方や、名詞修飾節の語順が難しく感じられることが多いので、正しい語順を身につける良い練習になります。また、「新入生にアンケート調査を行っている」という表現で、調査の対象が明確になっています。</em></p>
<h3>要素</h3>
<p><strong>Q: 次の文の__★_ に入る最も適切な語順を、下の1〜4の中から一つ選びなさい。プロジェクトを成功させるためには、いくつかの__★_ が必要です。1. 要素 2. 重要な 3. が 4. あります 1. 2 → 1 → 3 → 4 2. 1 → 2 → 3 → 4 3. 2 → 1 → 4 → 3 4. 1 → 3 → 2 → 4</strong></p>
<ul>
<li><b>2 → 1 → 3 → 4</b> <span style='color:green;'>(correct)</span></li>
<li>1 → 2 → 3 → 4</li>
<li>2 → 1 → 4 → 3</li>
<li>1 → 3 → 2 → 4</li>
</ul>
<p><em>Explanation: この問題では、「重要な要素がある（あります）」という自然な日本語の語順を作る力が問われます。「重要な」は「要素」を修飾し、「重要な要素」で「大切な部分・成分」という意味の名詞句になります。その名詞句が「が」によって主語となり、「あります（存在する）」で文が完結します。N3レベルの学習者にとって、「形容詞＋名詞」の修飾関係と、「が」を使った主語の作り方、そして「あります」の使い方を総合的に理解することがポイントです。また、「要素」は「物事を成り立たせる基本的な部分」という意味で、プロジェクトや計画の説明によく使われます。</em></p>
<h3>捜査</h3>
<p><strong>Q: 次の文の（　★　）に入るように、下の1～4の文を正しい順番に並べたものを一つ選びなさい。警察は、事件の真相を明らかにするために__★__行った。1. 証拠を 2. 集める 3. 捜査を 4. ために</strong></p>
<ul>
<li><b>3 → 1 → 2 → 4</b> <span style='color:green;'>(correct)</span></li>
<li>1 → 2 → 3 → 4</li>
<li>3 → 2 → 1 → 4</li>
<li>1 → 3 → 2 → 4</li>
</ul>
<p><em>Explanation: この問題の難しさは、動詞「捜査を行う」の目的語となる「証拠を集めるために」という副詞句を正しく並べることです。まず「捜査を」が来て、その目的「証拠を集める」が続き、最後に「ために」で理由や目的を表現します。N3レベルの語彙と文法（目的の「ために」）を理解しているかが問われます。</em></p>
<h3>問う</h3>
<p><strong>Q: 山川大学では、__★_ アンケート調査を行っている。次の語句を正しい順番に並べ替えて、自然な文を完成させなさい。 【並べ替え語句】 1. 大学生活 2. についての 3. イメージ 4. 新入生が</strong></p>
<ul>
<li><b>4 → 1 → 2 → 3</b> <span style='color:green;'>(correct)</span></li>
<li>4 → 3 → 1 → 2</li>
<li>4 → 2 → 1 → 3</li>
<li>4 → 3 → 2 → 1</li>
</ul>
<p><em>Explanation: この問題の主なポイントは、「についての」という連語と「イメージ」のつながりを正しく理解し、「大学生活についてのイメージ」という自然な名詞句を作ることです。JLPT N3レベルでは、「についての」の使い方や、名詞修飾の順序（どの名詞がどの名詞を修飾するか）がよく問われます。また、「新入生が」という主語を先頭に置き、「大学生活についてのイメージ」というまとまりを作った上で、「アンケート調査を行う」という動詞句につなげる必要があります。語句の区切りや助詞の扱いを意識しないと、「イメージ大学生活について」など不自然な語順になりやすいので、文全体の意味のつながりを考えながら並べ替えることが大切です。</em></p>
<h2>文法構造選択 (sentence_structure)</h2>
<p>5問：文法的に正しい文を選ぶ。</p>
<h3>大戦</h3>
# 大戦について 20世紀には、人類の歴史の中でもとても大きな戦争が二つありました。それは第一次世界大戦と第二次世界大戦です。第一次世界大戦は1914年に始まり、主にヨーロッパで多くの国が戦いました。終わったのは1918年です。この戦争で多くの人が命を失い、世界は大きく変わりました。 第二次世界大戦は1939年に始まりました。ドイツがポーランドに攻め込んだことがきっかけです。日本やイタリア、アメリカ、イギリス、ソ連など、世界中の多くの国が関わりました。戦争は1945年まで続きましたが、[　1　]。この戦争では、原子爆弾が使われたり、多くの都市が破壊されたりしました。 日本もこの戦争に参加していました。[　2　]、日本はアメリカと戦うことになり、太平洋の島々でも多くの戦いがありました。1945年、日本は負けて、戦争が終わりました。戦争が終わった後、日本は新しい憲法を作り、平和な国になることを決めました。 今でも、[　3　]。戦争の悲しさや平和の大切さについて学ぶことができます。大戦について知ることは、これからの世界を考える上でとても大切です。
<p><strong>Q: 1 の答えはどれですか？</strong></p>
<ul>
<li><b>世界はまた平和になりました</b> <span style='color:green;'>(correct)</span></li>
<li>世界はまだ戦いを続けました</li>
<li>世界は分かれてしまいました</li>
<li>世界は小さくなりました</li>
</ul>
<p><em>Explanation: 文脈から、戦争が終わった後の世界の様子を説明しています。「また平和になりました」が最も自然な文の流れです。他の選択肢は意味が合いません。</em></p>
<p><strong>Q: 2 の答えはどれですか？</strong></p>
<ul>
<li><b>そのため</b> <span style='color:green;'>(correct)</span></li>
<li>たとえば</li>
<li>しかし</li>
<li>それから</li>
</ul>
<p><em>Explanation: 「日本もこの戦争に参加していました。」の後、「そのため、日本はアメリカと戦うことになり…」と続くのが自然です。「たとえば」「しかし」「それから」は文のつながりとして不適切です。</em></p>
<p><strong>Q: 3 の答えはどれですか？</strong></p>
<ul>
<li><b>学校で大戦について教えられています</b> <span style='color:green;'>(correct)</span></li>
<li>学校で大戦について教えられていません</li>
<li>学校で大戦について知られていません</li>
<li>学校で大戦について終わられています</li>
</ul>
<p><em>Explanation: 現代の日本や多くの国では、大戦について学校で学びます。他の選択肢は意味が通りません。</em></p>
<h3>感動</h3>
ご丁寧なご評価と詳細なフィードバックをいただき、誠にありがとうございます。ご指摘いただいた点を反映し、よりバランスのとれたN3レベルの模擬問題を作成いたします。---## 夏休みの思い出（改訂版）私は去年の夏休みに、友達の家に泊まりに行きました。友達の家族はとても親切で、私を[1]＿＿＿＿。日本人の家に泊まるのは初めてだったので、行く前は少し不安な気持ちもありました。[2]＿＿＿＿、行ってみたらとても楽しかったです。一番印象に残っているのは、友達の家の畑で育てた野菜を使って、みんなで料理を作ったことです。友達のお母さんは畑でいろいろな野菜を育てていました。私たちは、その野菜を使ってみんなで料理をしました。私は、お店に売られている野菜[3]＿＿＿＿食べたことがありませんでした。家で育てた野菜を食べたのは初めてでしたが、とてもおいしかったです。特に「私も野菜を育ててみたい」と思いました。そう言ったら、それを聞いていたお母さんが、家の中でも育てることができる野菜について教えてくれました。お母さんに教えてもらったやり方で、私も野菜を[4]＿＿＿＿。今、２種類の野菜を育てています。野菜の世話をしながら、楽しかった夏休みのことをいつも思い出しています。---### 設問#### 文法・語彙選択問題1. [1]に入る最も適切な文法はどれですか。1. 招待してくれたのです2. 招待してくれたはずです3. 招待してくれたばかりです4. 招待してくれたそうです2. [2]に入る最も適切な文法はどれですか。1. それで2. でも3. 実は4. また3. [3]に入る最も適切な文法はどれですか。1. は2. などを3. しか4. だけ4. [4]に入る最も適切な文法はどれですか。1. 育ててみてほしいです2. 育ててみてもいいです3. 育ててみようとしました4. 育ててみることにしました#### 語彙問題5. 「親切」と最も近い意味を持つ言葉はどれですか。1. 冷たい2. 優しい3. 怖い4. うるさい#### 読解問題6. 友達の家で育てた野菜の特徴として正しいものはどれですか。1. お店で買ったものよりおいしくなかった。2. 初めて食べてとてもおいしかった。3. 友達は野菜が嫌いだった。4. 料理には使わなかった。---### 解答・解説1. 正解：1. 招待してくれたのです  **解説**：「〜のです」は理由や説明を表します。ここでは家族の親切さを説明しています。2. 正解：2. でも  **解説**：「でも」は前後の内容が対立しているときに使います。3. 正解：3. しか  **解説**：「しか〜ない」は否定形とセットで「それ以外はない」という意味です。例：「お店に売られている野菜しか食べたことがありませんでした。」4. 正解：4. 育ててみることにしました  **解説**：「〜ことにしました」は自分で決めて何かを始めるときに使います。5. 正解：2. 優しい  **解説**：「親切」と「優しい」は、どちらも人に対して思いやりがあるという意味で使われます。6. 正解：2. 初めて食べてとてもおいしかった。  **解説**：本文中に「家で育てた野菜を食べたのは初めてでしたが、とてもおいしかった」と書かれています。---### 教師の視点での補足- 「しか〜ない」は必ず否定形と一緒に使われます（例：「〜しかありません」）。- 類義語や内容理解の問題を加えることで、文法だけでなく語彙力や読解力も総合的に評価できます。- 問題番号や選択肢の表記を統一することで、より見やすい試験問題になります。---ご提案いただいた内容を反映し、よりバランスの取れた問題構成となりました。今後も学習者の視点に立った出題を心がけてまいります。ご指導ありがとうございました。
<p><strong>Q: [1]に入る最も適切な文法はどれですか。</strong></p>
<ul>
<li><b>招待してくれたのです</b> <span style='color:green;'>(correct)</span></li>
<li>招待してくれたはずです</li>
<li>招待してくれたばかりです</li>
<li>招待してくれたそうです</li>
</ul>
<p><em>Explanation: 「〜のです」は理由や説明を表します。ここでは家族の親切さを説明しています。</em></p>
<p><strong>Q: [2]に入る最も適切な文法はどれですか。</strong></p>
<ul>
<li>それで</li>
<li><b>でも</b> <span style='color:green;'>(correct)</span></li>
<li>実は</li>
<li>また</li>
</ul>
<p><em>Explanation: 「でも」は前後の内容が対立しているときに使います。</em></p>
<p><strong>Q: [3]に入る最も適切な文法はどれですか。</strong></p>
<ul>
<li>は</li>
<li>などを</li>
<li><b>しか</b> <span style='color:green;'>(correct)</span></li>
<li>だけ</li>
</ul>
<p><em>Explanation: 「しか〜ない」は否定形とセットで「それ以外はない」という意味です。例：「お店に売られている野菜しか食べたことがありませんでした。」</em></p>
<p><strong>Q: [4]に入る最も適切な文法はどれですか。</strong></p>
<ul>
<li>育ててみてほしいです</li>
<li>育ててみてもいいです</li>
<li>育ててみようとしました</li>
<li><b>育ててみることにしました</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「〜ことにしました」は自分で決めて何かを始めるときに使います。</em></p>
<p><strong>Q: 「親切」と最も近い意味を持つ言葉はどれですか。</strong></p>
<ul>
<li>冷たい</li>
<li><b>優しい</b> <span style='color:green;'>(correct)</span></li>
<li>怖い</li>
<li>うるさい</li>
</ul>
<p><em>Explanation: 「親切」と「優しい」は、どちらも人に対して思いやりがあるという意味で使われます。</em></p>
<p><strong>Q: 友達の家で育てた野菜の特徴として正しいものはどれですか。</strong></p>
<ul>
<li>お店で買ったものよりおいしくなかった。</li>
<li><b>初めて食べてとてもおいしかった。</b> <span style='color:green;'>(correct)</span></li>
<li>友達は野菜が嫌いだった。</li>
<li>料理には使わなかった。</li>
</ul>
<p><em>Explanation: 本文中に「家で育てた野菜を食べたのは初めてでしたが、とてもおいしかった」と書かれています。</em></p>
<h3>限界</h3>
### 限界について 私たちは毎日、さまざまなことに挑戦しています。しかし、どんなことにも「限界（げんかい）」があります。限界とは、**これ以上はできない**というぎりぎりの範囲や、物事の終わりのことです。たとえば、スポーツ選手は自分の体力の限界まで練習しますし、学生も勉強の限界を感じることがあります。 私が高校生のとき、テスト勉強で夜遅くまで頑張ったことがあります。最初は「もっとできる」と思っていましたが、だんだん疲れてきて、**（　1　）**。そのとき、「これが自分の限界かもしれない」と思いました。しかし、友達に「少し休んだら、また頑張れるよ」と言われて、休憩してからもう一度勉強を始めました。 限界を感じたとき、**（　2　）**。無理をしすぎると、体や心に悪い影響が出ることもあります。だから、時々自分の限界を知って、休むことも大切です。 最近は、「限界オタク」や「限界飯」など、いろいろな言い方もあります。これは、**（　3　）**。自分なりの限界を知っているからこそ、毎日を楽しく過ごせるのかもしれません。
<p><strong>Q: **（　1　）** に入る最もよいものを選びなさい。</strong></p>
<ul>
<li><b>眠くなってきました</b> <span style='color:green;'>(correct)</span></li>
<li>眠くなってきたそうです</li>
<li>眠くなってきたはずです</li>
<li>眠くなってきたらしいです</li>
</ul>
<p><em>Explanation: ここは自分の体験を述べているので、普通形の「眠くなってきました」が自然です。N3レベルでは「～そうです」「～はずです」「～らしいです」の使い分けが難しいですが、体験談には直接的な表現が最適です。</em></p>
<p><strong>Q: **（　2　）** に入る最もよいものを選びなさい。</strong></p>
<ul>
<li>すぐにあきらめるべきです</li>
<li><b>少し休むことも大切です</b> <span style='color:green;'>(correct)</span></li>
<li>もっとがんばるしかありません</li>
<li>何もしないほうがいいです</li>
</ul>
<p><em>Explanation: 限界を感じたときのアドバイスとして最も自然です。他の選択肢は極端すぎて文脈に合いません。</em></p>
<p><strong>Q: **（　3　）** に入る最もよいものを選びなさい。</strong></p>
<ul>
<li><b>限界という言葉がいろいろな場面で使われている例です</b> <span style='color:green;'>(correct)</span></li>
<li>限界を知らない人が使う言葉です</li>
<li>限界をこえるための言葉です</li>
<li>限界は昔から使われていない言葉です</li>
</ul>
<p><em>Explanation: 「限界オタク」「限界飯」などの説明に最も合っています。</em></p>
<p><strong>Q: 下線部「限界」とは、どんな意味ですか。</strong></p>
<ul>
<li>何もできないこと</li>
<li><b>これ以上できないぎりぎりの範囲</b> <span style='color:green;'>(correct)</span></li>
<li>とても楽しいこと</li>
<li>何度も挑戦すること</li>
</ul>
<p><em>Explanation: 記事の冒頭で説明しています。</em></p>
<h3>黙る</h3>
ご指摘いただきありがとうございます。ご提案に基づき、改訂版の問題を作成いたしました。---### 次の文章を読んで、[1]～[4]の空欄に入る最も適切な語句をそれぞれ1つ選びなさい。---#### 黙るということ学校や仕事で、時々「黙る」ことが大切だと感じる場面があります。たとえば、会議や授業で誰かが話しているとき、静かに[1]、相手の話をよく聞くことはとても重要です。また、友達とけんかをしたとき、すぐに言い返すのではなく、[2]ことで、自分の気持ちを落ち着かせることもできます。私の友人の田中さんは、[3]。田中さんは、家族とけんかをしたとき、何も言わずに自分の部屋に入り、しばらく黙って考えるそうです。そのあと、気持ちが落ち着いたら、もう一度話し合いをするようにしています。日本の文化では、「黙る」ことは時に「相手を尊重する」という意味も持っています。たとえば、茶道や会議の場面で、すぐに自分の意見を言わず、[4]、相手が何を考えているかを理解しようとすることが大切だと考えられています。---
<p><strong>Q: [1]に入る最も適切な語句を選びなさい。</strong></p>
<ul>
<li><b>黙って</b> <span style='color:green;'>(correct)</span></li>
<li>黙らせて</li>
<li>黙るように</li>
<li>黙ろうと</li>
</ul>
<p><em>Explanation: 「静かに黙って、相手の話をよく聞く」の「黙って」は、「て形」で動作を並べるN3頻出の表現です。「黙らせて」は使役、「黙るように」は意向や命令、「黙ろうと」は意志や試みで、文脈に合いません。</em></p>
<p><strong>Q: [2]に入る最も適切な語句を選びなさい。</strong></p>
<ul>
<li><b>黙ってみる</b> <span style='color:green;'>(correct)</span></li>
<li>黙らせる</li>
<li>黙ること</li>
<li>黙っている</li>
</ul>
<p><em>Explanation: 「黙ってみることで」は「試しに黙る」という意味。「～てみる」はN3でよく出る「試す」表現です。他の選択肢は意味がずれます。</em></p>
<p><strong>Q: [3]に入る最も適切な語句を選びなさい。</strong></p>
<ul>
<li>よくしゃべる人です</li>
<li>あまり話さない人です</li>
<li>口数が少ない人です</li>
<li><b>よく黙る人です</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「黙ることができる人」というニュアンスに合うのは「よく黙る人です」。今回は「あまり話さない人です」「口数が少ない人です」も類義語として適切ですが、文章の流れから一番直接的な表現を選びます。N3では類義語の使い分けも重要です。</em></p>
<p><strong>Q: [4]に入る最も適切な語句を選びなさい。</strong></p>
<ul>
<li><b>しばらく黙って</b> <span style='color:green;'>(correct)</span></li>
<li>しばらく黙らせて</li>
<li>しばらく黙るように</li>
<li>しばらく黙ろうと</li>
</ul>
<p><em>Explanation: 「しばらく黙って、相手が何を考えているかを理解しようとする」は、「て形」で動作の連続を表します。N3頻出の接続です。</em></p>
<h3>持ち上げる</h3>
ご丁寧なご指摘、誠にありがとうございます。ご提案いただいた改善点を反映し、より完成度の高いJLPT N3向け模擬試験問題を以下に再作成いたします。---### JLPT N3 模擬試験：読解---#### 夏休みの思い出私は、夏休みに友達の家に遊びに行きました。友達の家は山の近くにあり、自然がたくさんあります。友達の家族はとても親切で、私を[  1  ]＿＿招待してくれたのです。日本人の家に泊まるのは初めてだったので、行く前は少し不安な気持ちもありました。[  2  ]＿＿でも、行ってみたらとても楽しかったです。印象に残っているのは、庭の畑で育てた野菜を使って、みんなで料理を作ったことです。友達のお母さんは、畑でいろいろな野菜を育てていました。私たちは、その野菜を使ってみんなで料理をしました。私は、お店に売られている野菜[  3  ]＿＿しか食べたことがありませんでした。家で育てた野菜を食べたのは初めてでしたが、とてもおいしかったです。「私も野菜を育ててみたい」と思いました。そう言ったら、それを聞いていたお母さんが、家の中でも育てることができる野菜について教えてくれました。お母さんに教えてもらったやり方で、私も野菜を[  4  ]＿＿育ててみることにしました。今、２種類の野菜を育てています。野菜の世話をしながら、楽しかった夏休みのことをいつも思い出しています。---#### 問題次の[  1  ]～[  4  ]に入る最も適切なものを一つ選びなさい。**[ 1 ]**1. 招待してくれたのです  2. 招待してくれたはずです  3. 招待してくれたばかりです  4. 招待してくれたそうです  **[ 2 ]**1. それで  2. でも  3. 実は  4. また  **[ 3 ]**1. は  2. も  3. しか  4. だけ  **[ 4 ]**1. 育ててみてほしいです  2. 育ててみてもいいです  3. 育ててみようとしました  4. 育ててみることにしました---#### 正解と解説**[ 1 ] 正解：1. 招待してくれたのです**- 「～のです」は理由や説明を表す表現で、ここでは「親切で、私を招待してくれたのです」と、家族の親切さを説明しています。- 「はずです」は推量や確信の表現で、事実説明には不自然です。- 「ばかりです」は直前の出来事に使う表現ですが、ここでは時制が合いません。- 「そうです」は伝聞なので、自分の経験には使いません。**[ 2 ] 正解：2. でも**- 前の文が「不安な気持ちもありました」とマイナスの内容なので、その後に「でも、行ってみたら楽しかった」と逆接の接続詞が自然です。- 「それで」は順接、「実は」は話題転換、「また」は追加なので文脈に合いません。**[ 3 ] 正解：3. しか**- 「しか～ない」は「それ以外は～ない」という限定を表すN3の重要文法です。- 「も」は「ほかにも」を意味し、ここでは意味が逆になります。- 「は」「だけ」も使えそうですが、「しか」の方が否定文とセットで自然です。**[ 4 ] 正解：4. 育ててみることにしました**- 「～ことにしました」は「自分の意志で決めた」ことを表します。- 「育ててみてほしい」は他者への希望、「育ててみてもいい」は許可、「育ててみようとしました」は「試みた」意味で、ここでは意志決定が自然です。---#### 教師からのポイントこの問題では、- 理由や説明を表す「～のです」  - 逆接の接続詞「でも」  - 限定の「しか～ない」  - 意志決定の「～ことにする」 など、N3で頻出の文法知識を実際の文脈の中で使い分ける力を問います。学習者には、単語や文法だけでなく、文の流れや前後の内容をよく読み取り、自然な日本語表現を選ぶ力を身につけてほしいと思います。選択肢の違いを具体的に理解し、実際の会話や文章でも使い分けられるように練習しましょう。---**例文補足**- 「～のです」：理由説明「今日は頭が痛いのです」  - 「でも」：逆接「雨が降っています。でも、出かけます」  - 「しか～ない」：「水しか飲みません」  - 「～ことにしました」：「来年日本に行くことにしました」---今後も、文脈理解と文法知識の両方を鍛える問題作成を心がけます。ご指摘ありがとうございました。
<p><strong>Q: [ 1 ]に入る最も適切なものはどれですか？</strong></p>
<ul>
<li><b>招待してくれたのです</b> <span style='color:green;'>(correct)</span></li>
<li>招待してくれたはずです</li>
<li>招待してくれたばかりです</li>
<li>招待してくれたそうです</li>
</ul>
<p><em>Explanation: 「～のです」は理由や説明を表す表現で、ここでは「親切で、私を招待してくれたのです」と、家族の親切さを説明しています。 「はずです」は推量や確信の表現で、事実説明には不自然です。 「ばかりです」は直前の出来事に使う表現ですが、ここでは時制が合いません。 「そうです」は伝聞なので、自分の経験には使いません。</em></p>
<p><strong>Q: [ 2 ]に入る最も適切なものはどれですか？</strong></p>
<ul>
<li>それで</li>
<li><b>でも</b> <span style='color:green;'>(correct)</span></li>
<li>実は</li>
<li>また</li>
</ul>
<p><em>Explanation: 前の文が「不安な気持ちもありました」とマイナスの内容なので、その後に「でも、行ってみたら楽しかった」と逆接の接続詞が自然です。 「それで」は順接、「実は」は話題転換、「また」は追加なので文脈に合いません。</em></p>
<p><strong>Q: [ 3 ]に入る最も適切なものはどれですか？</strong></p>
<ul>
<li>は</li>
<li>も</li>
<li><b>しか</b> <span style='color:green;'>(correct)</span></li>
<li>だけ</li>
</ul>
<p><em>Explanation: 「しか～ない」は「それ以外は～ない」という限定を表すN3の重要文法です。 「も」は「ほかにも」を意味し、ここでは意味が逆になります。 「は」「だけ」も使えそうですが、「しか」の方が否定文とセットで自然です。</em></p>
<p><strong>Q: [ 4 ]に入る最も適切なものはどれですか？</strong></p>
<ul>
<li>育ててみてほしいです</li>
<li>育ててみてもいいです</li>
<li>育ててみようとしました</li>
<li><b>育ててみることにしました</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「～ことにしました」は「自分の意志で決めた」ことを表します。 「育ててみてほしい」は他者への希望、「育ててみてもいい」は許可、「育ててみようとしました」は「試みた」意味で、ここでは意志決定が自然です。</em></p>
<h1>第2部：読解</h1>
<h2>短文読解 (short_passage_read)</h2>
<p>4題：短めの文章を読んで内容を理解する問題。</p>
<h3>レストランで食べ物を注文する</h3>
<p><strong>Q: この文章によると、注文がまだ決まっていないとき、どうしたらいいですか。</strong></p>
<ul>
<li>「お水をください」と言う</li>
<li><b>「もう少し待ってください」と言う</b> <span style='color:green;'>(correct)</span></li>
<li>「以上です」と言う</li>
<li>「かしこまりました」と言う</li>
</ul>
<p><em>Explanation: この問題のポイントは、「注文がまだ決まっていないとき」にどう伝えるかを理解することです。N3レベルの語彙と表現（「もう少し」「待ってください」など）に慣れていない学習者は、正しい応答を選ぶのが難しいかもしれません。また、他の選択肢は注文や確認の場面で使う表現なので、状況判断も必要です。</em></p>
<h3>道を尋ねる</h3>
<p><strong>Q: カナさんの家に行くとき、どこで右に曲がりますか。</strong></p>
<ul>
<li>東京駅の南口を出てすぐ</li>
<li><b>一つ目の信号</b> <span style='color:green;'>(correct)</span></li>
<li>スーパーの前の道</li>
<li>公園の前</li>
</ul>
<p><em>Explanation: 本文には「まず、東京駅の南口を出て、まっすぐ歩いてください。一つ目の信号を右に曲がります。」とあります。したがって、右に曲がる場所は「一つ目の信号」です。他の選択肢について、「東京駅の南口を出てすぐ」はまだ曲がりません。「スーパーの前の道」は左に曲がる場所、「公園の前」は道案内にありません。道案内の表現では、「〜を出て」「〜を曲がる」「〜の前の道」などの語順や助詞に注意することが重要です。例えば、「右に曲がる」と「左に曲がる」を混同しないようにしましょう。また、「〜の前の道」など具体的な場所の指定にも注意を払いましょう。</em></p>
<h3>週末の予定について話す</h3>
<p><strong>Q: このメッセージの主な目的は何ですか？</strong></p>
<ul>
<li><b>感謝の意を伝え、今後の質の高い問題作成への取り組みを約束すること</b> <span style='color:green;'>(correct)</span></li>
<li>受験者からの質問に答えること</li>
<li>試験の結果を通知すること</li>
<li>新しい試験日程を案内すること</li>
</ul>
<p><em>Explanation: メッセージは感謝の意を表し、今後の取り組みや連絡の促進について述べています。</em></p>
<h3>支払い方法について話す</h3>
<p><strong>Q: 山田様が田中商事に支払い方法を伝えるために、いつまでに返信しなければなりませんか。</strong></p>
<ul>
<li>6月8日</li>
<li><b>6月10日</b> <span style='color:green;'>(correct)</span></li>
<li>6月12日</li>
<li>6月15日</li>
</ul>
<p><em>Explanation: この問題の主な難しさは、「までに」という表現の理解と、ビジネスメールの中で期限を正確に読み取る力です。N3レベルの学習者は、日付や期限を表す語句を正確に把握する必要があります。また、「ご返信いただけますでしょうか」という丁寧な依頼表現も文脈から理解する必要があります。</em></p>
<h2>中程度の文章読解 (midsize_passage_read)</h2>
<p>2題：中程度の長さの文章を読んで質問に答える。</p>
<h3>趣味について話す</h3>
<p><strong>Q: この文章の内容に合うものを一つ選びなさい。</strong></p>
<ul>
<li><b>筆者は友人に勧められて写真を始めた。</b> <span style='color:green;'>(correct)</span></li>
<li>筆者は自分の写真を家族だけに見せている。</li>
<li>筆者は風景や花よりも人物の写真が好きだ。</li>
<li>筆者は写真のコンテストで何度も賞をもらっている。</li>
</ul>
<p><em>Explanation: この問題のポイントは、文章の中から筆者が写真を始めた理由や、どのように趣味を楽しんでいるかを正確に読み取れるかどうかです。選択肢2と3、4は内容と異なります。N3レベルでは、「きっかけ」「楽しみ方」「成果」などの情報を正しく把握する力が問われます。また、趣味の話題は身近ですが、細かい情報の違いに気づけるかが重要です。</em></p>
<h3>旅行の計画について話す</h3>
<p><strong>Q: 本文の内容と合っているものをすべて選びなさい。</strong></p>
<ul>
<li>旅行の前にホテルを選んだ。</li>
<li>旅行中は毎日雨が降っていた。</li>
<li>家族と一緒に北海道に行った。</li>
<li>トラブルがあったので、旅行は大変だった。</li>
</ul>
<p><em>Explanation: AとCは本文の内容と一致しています。Bは「天気がよくて」とあるので間違い、Dは「トラブルもなく」とあるので間違いです。N3レベルでは、複数選択肢から正しいものを選ぶ力や、語彙・文法の基本的な理解が求められます。本文の内容を正確に読み取り、根拠を持って選択できるかがポイントです。</em></p>
<h2>長文読解 (long_passage_read)</h2>
<p>1題：やや長めの文章を読んで全体の内容を把握する。</p>
<h3>健康とフィットネスについて話す</h3>
ありがとうございます。ご指摘いただいた点を踏まえ、より良い読解問題の作成に努めます。以下に、改善提案を反映した新しい読解問題例を示します。健康（けんこう）とフィットネスについて 最近（さいきん）、健康な生活（せいかつ）を送ることが大切（たいせつ）だと言われています。毎日（まいにち）野菜（やさい）を食べたり、十分（じゅうぶん）な睡眠（すいみん）をとったりすることは、体（からだ）に良いです。また、ストレスを減（へ）らすために、リラクゼーションや趣味（しゅみ）の時間（じかん）を作る人も多いです。フィットネスも健康のために大切です。たとえば、ジョギングや筋トレなどの運動（うんどう）を週（しゅう）に数回（すうかい）行（おこな）うと、体力（たいりょく）がつきます。有酸素運動（ゆうさんそうんどう）は心臓（しんぞう）や肺（はい）の働（はたら）きを良くし、ダイエットにも効果（こうか）があります。会社（かいしゃ）では、年（ねん）に一度（いちど）健康診断（けんこうしんだん）が行われます。健康診断では、医師（いし）が体の状態（じょうたい）を詳（くわ）しく調（しら）べてくれます。健康的な生活を続（つづ）けることで、病気（びょうき）のリスクを減らすことができます。みなさんも、毎日の生活にフィットネスを取り入れてみてください。
<p><strong>Q: 健康的な生活のために大切なことを、二つ書きなさい。（記述）</strong></p>
<ul>
</ul>
<p><em>Explanation: 本文から抜き出し、要点を短くまとめる練習です。</em></p>
<p><strong>Q: フィットネスをすると、どんな効果がありますか。（記述）</strong></p>
<ul>
</ul>
<p><em>Explanation: 複数の効果をまとめて答える力が必要です。</em></p>
<p><strong>Q: 健康診断はなぜ行われますか。（記述）</strong></p>
<ul>
</ul>
<p><em>Explanation: 目的を短く説明するのがポイントです。</em></p>
<p><strong>Q: 本文の内容と合っているものを一つ選びなさい。（選択）</strong></p>
<ul>
<li>フィットネスは週に一回だけすれば十分である。</li>
<li><b>有酸素運動は心臓や肺に良い効果がある。</b> <span style='color:green;'>(correct)</span></li>
<li>健康診断は毎月行われる。</li>
<li>野菜を食べるとストレスが増える。</li>
</ul>
<p><em>Explanation: 本文の正しい内容を選ぶ問題。細部理解を問います。</em></p>
<p><strong>Q: 「リラクゼーション」とは何のためにしますか。（選択）</strong></p>
<ul>
<li>体力をつけるため</li>
<li><b>ストレスを減らすため</b> <span style='color:green;'>(correct)</span></li>
<li>体の状態を調べるため</li>
<li>病気を治すため</li>
</ul>
<p><em>Explanation: 語彙の意味理解問題。本文の該当部分を正しく把握できるか確認します。</em></p>
<h2>情報検索 (info_retrieval)</h2>
<p>1題：掲示や案内などから必要な情報を探す問題。</p>
<h3>公共施設の利用方法について話す</h3>
<h3>公共施設の利用案内</h3><p>下の表を読んで、質問に答えなさい。</p><table border="1"><tr><th>施設名</th><th>利用時間</th><th>利用方法</th><th>注意点</th></tr><tr><td>図書館</td><td>9:00～19:00</td><td>利用者カードを作り、本を借りる</td><td>静かに利用する</td></tr><tr><td>体育館</td><td>8:00～21:00</td><td>窓口で予約して利用</td><td>運動用の靴が必要</td></tr><tr><td>公民館</td><td>9:00～17:00</td><td>イベントや会議は事前申し込み</td><td>ごみは持ち帰る</td></tr><tr><td>市民プール</td><td>10:00～18:00</td><td>当日券を買って入場</td><td>水着・帽子が必要</td></tr></table><ul><li>公共施設は市民なら誰でも利用できます。</li><li>初めて利用する場合は、利用者登録や申し込みが必要なことがあります。</li><li>施設ごとに利用時間やルールが違うので、事前に確認しましょう。</li><li>みんなが気持ちよく使えるように、マナーを守りましょう。</li></ul><hr><b>【正答・解説】</b><br>1. 正解：2<br>体育館の利用方法は「窓口で予約して利用」と表にあるため、2が正解です。<br>2. 正解：3<br>公民館の注意点は「ごみは持ち帰る」と表にあるため、3が正解です。<br>3. 正解：1<br>図書館の利用方法は「利用者カードを作り、本を借りる」とあるため、1が正解です。<br>4. 正解：1<br>市民プールの注意点は「水着・帽子が必要」とあるため、1が正解です。<br><b>【教師の視点からの主な課題】</b><br>この問題では、表の情報を正しく読み取る力が求められます。施設ごとに利用方法や注意点が異なるため、細かい違いに気づくことが難しい場合があります。また、「予約」「申し込み」「持ち帰る」などN3レベルの語彙や表現が使われているので、語彙力や読解力が問われます。表の情報と設問のキーワードを正確に結びつけることがポイントです。<hr><b>【語彙・表現の解説】</b><ul><li>利用者カード：施設を利用する人のためのカード</li><li>窓口：受付の場所</li><li>予約：前もって申し込むこと</li><li>当日券：その日に買う入場券</li><li>持ち帰る：自分の家などに持って帰ること</li><li>必要：いる、いるもの</li></ul>
<p><strong>Q: 体育館を利用したいとき、どうすればいいですか。</strong></p>
<ul>
<li>利用者カードを作る</li>
<li><b>窓口で予約する</b> <span style='color:green;'>(correct)</span></li>
<li>当日券を買う</li>
<li>イベントに申し込む</li>
</ul>
<p><em>Explanation: 体育館の利用方法は「窓口で予約して利用」と表にあるため、2が正解です。</em></p>
<p><strong>Q: 公民館を利用するときの注意点はどれですか。</strong></p>
<ul>
<li>水着を持っていく</li>
<li>静かに利用する</li>
<li><b>ごみは持ち帰る</b> <span style='color:green;'>(correct)</span></li>
<li>運動用の靴が必要</li>
</ul>
<p><em>Explanation: 公民館の注意点は「ごみは持ち帰る」と表にあるため、3が正解です。</em></p>
<p><strong>Q: 図書館を利用するために、まず何をしなければなりませんか。</strong></p>
<ul>
<li><b>利用者カードを作る</b> <span style='color:green;'>(correct)</span></li>
<li>運動用の靴を持っていく</li>
<li>当日券を買う</li>
<li>イベントに申し込む</li>
</ul>
<p><em>Explanation: 図書館の利用方法は「利用者カードを作り、本を借りる」とあるため、1が正解です。</em></p>
<p><strong>Q: 市民プールに入るとき、必要なものは何ですか。</strong></p>
<ul>
<li><b>水着と帽子</b> <span style='color:green;'>(correct)</span></li>
<li>運動用の靴</li>
<li>利用者カード</li>
<li>ごみ袋</li>
</ul>
<p><em>Explanation: 市民プールの注意点は「水着・帽子が必要」とあるため、1が正解です。</em></p>
<h1>第3部：聴解</h1>
<h2>話題理解 (topic_understanding)</h2>
<p>6問：会話や説明を聞いて、話題を理解する。</p>
<h3>購入したい商品の説明</h3>
<p><strong>Q: 女性のお客さんはこの後、どうしますか。</strong></p>
<ul>
<li><b>少し考えてから決める</b> <span style='color:green;'>(correct)</span></li>
<li>トースターをすぐに買う</li>
<li>他の店員に説明を求める</li>
<li>トースターの使い方をもう一度聞く</li>
</ul>
<p><em>Explanation: 会話の最後で「少し考えてみます」とお客さんが言っていることから、すぐに購入するのではなく、いったん検討する意向であることがわかります。「少し考えてみます」は日本語で即決を避ける丁寧な断り表現としてよく使われます。N3レベルでは、このようなやわらかい表現の意味や意図を正しく理解することが求められます。選択肢2・3・4は会話の内容と合わないため、正解は1となります。</em></p>
<h3>バスの時刻表を尋ねる</h3>
<p><strong>Q: 男の人はこのあと、まず何をしますか。</strong></p>
<ul>
<li>切符を買いに行く</li>
<li><b>掲示板で時刻表を確認する</b> <span style='color:green;'>(correct)</span></li>
<li>女の人と一緒に行く</li>
<li>市役所に電話をかける</li>
</ul>
<p><em>Explanation: この問題は、会話の流れと語彙理解を問う典型的なN3レベルのリスニング設問です。「時刻表（じこくひょう）」「掲示板（けいじばん）」「切符（きっぷ）」といったN3語彙が使われており、実際の生活場面に即しています。「まず何をしますか」という設問表現も自然で、選択肢１は「バスの中で買える」という説明があるため誤答だと明確です。正答の根拠をしっかり聞き取る力が求められます。N3学習者の語彙・読解力のバランスを測定できる良問です。</em></p>
<h3>お気に入りのレストランについて話す</h3>
<p><strong>Q: 友人Bはこのあと、何をしたいと思っていますか。</strong></p>
<ul>
<li>新しいレストランを探す</li>
<li><b>友人Aと一緒にレストランに行く</b> <span style='color:green;'>(correct)</span></li>
<li>家でパスタを作る</li>
<li>デザートだけ食べに行く</li>
</ul>
<p><em>Explanation: この問題は会話の流れを理解し、登場人物の意図をつかむ力を問うものです。友人Bは「うん、ぜひ行こう！」と答えており、友人Aと一緒にそのレストランに行くことを楽しみにしています。他の選択肢は会話の内容と合いません。N3レベルの語彙・文法を使用し、会話の内容把握と推測力が求められる標準的な設問です。</em></p>
<h3>家族について話す</h3>
<p><strong>Q: この会話のあと、さとしさんは何をしたいと思っていますか。</strong></p>
<ul>
<li>実家に帰りたい</li>
<li>妹たちに会いたい</li>
<li>家族と一緒に住みたい</li>
<li><b>お兄さんに会いに行きたい</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: この問題では、さとしさんが「最近なかなか会えないので、今度の休みに会いに行こうと思っています」と明言しているため、根拠が明確です。N3レベルの語彙（兄、一人暮らし、会社員、家族など）や文法を使った自然な会話文となっています。選択肢には「実家に帰りたい」や「妹たちに会いたい」など、会話に出てきた家族構成を利用した紛らわしいものも含めていますが、正しい選択肢を見抜くためには会話の内容を正確に聞き取り、理解する必要があります。推測ではなく、発話内容に基づいて選択できる良質なリスニング問題です。</em></p>
<h3>最近の映画について話す</h3>
<p><strong>Q: ユウコさんはこの後、何をしたいと言っていますか。</strong></p>
<ul>
<li>家で映画を見る</li>
<li>映画の感想を友達に話す</li>
<li><b>新しい映画を映画館で見る</b> <span style='color:green;'>(correct)</span></li>
<li>アクション映画のDVDを買う</li>
</ul>
<p><em>Explanation: ユウコさんは「今週末、新しいアクション映画が公開されるから、見に行こうかなって思ってる」と言っています。本文の内容と登場人物の意図を正確に読み取る力を問う良問です。</em></p>
<h3>スポーツについて話す</h3>
<p><strong>Q: この会話のあと、マリさんは何をしたいと思っていますか。</strong></p>
<ul>
<li>ケンさんとテニスをする</li>
<li><b>友達とバスケットボールをする</b> <span style='color:green;'>(correct)</span></li>
<li>公園でランニングをする</li>
<li>家で映画を見る</li>
</ul>
<p><em>Explanation: この問題は、会話の中でスポーツの予定について話し合う内容を聞き取り、マリさんの「この後したいこと」を選ぶリスニング問題です。N3レベルの語彙・表現（つもり、予定、最近～ばかり、また今度、ゆっくりする、映画を見る、など）を使用しています。正解を選ぶためには、マリさんが「土曜日に友達とバスケットボールをするつもり」と明確に述べている点に注意する必要があります。選択肢には似たような活動（映画を見る、テニスをする）が含まれているため、会話の流れや主語に注意して聞き取る力が求められます。</em></p>
<h2>要点理解 (keypoint_understanding)</h2>
<p>6問：会話や説明の中の重要なポイントを聞き取る。</p>
<h3>タクシーを予約する</h3>
<p><strong>Q: 男の人はタクシーの予約についてどう考えていますか。</strong></p>
<ul>
<li>予約料金が高いので使いたくない</li>
<li>電車のほうが便利だと思っている</li>
<li><b>アプリ予約は安心で便利だと思い始めた</b> <span style='color:green;'>(correct)</span></li>
<li>現金で支払いたいので予約はしない</li>
</ul>
<p><em>Explanation: この問題の主な難しさは、会話の流れの中で男の人の考えが変化していく点です。最初はタクシー予約に迷いがありますが、女の人の説明でメリットを知り、最終的に「アプリで予約してみる」と前向きに考えています。N3レベルの語彙や表現（例えば「迷ってる」「確実に」「領収書」「経費」など）が理解できているかもポイントです。また、選択肢の中で「思い始めた」という変化をとらえられるかが鍵となります。</em></p>
<h3>天気の状況について話す</h3>
<p><strong>Q: この会話から、今日の天気はどうですか。</strong></p>
<ul>
<li>雨がふっています。</li>
<li>くもりです。</li>
<li><b>晴れています。</b> <span style='color:green;'>(correct)</span></li>
<li>雪がふっています。</li>
</ul>
<p><em>Explanation: この問題は、会話の流れの中から「今日の天気」が何かを判断する力を問います。N3レベルでは「晴れ」「雨」「くもり」「雪」などの基本語彙に加え、「太陽が出ている」「青空」「気持ちがいい」などの表現にも注意が必要です。受験者は、直接的な表現だけでなく、天気を表す間接的な言い回しや比較（昨日は雨、今週はくもりだったが今日は晴れ）を聞き取る必要があるため、内容理解力が問われます。選択肢がすべて短い単語なので、聞き間違いに注意しましょう。</em></p>
<h3>ショッピング体験を説明する</h3>
<p><strong>Q: このメッセージの主な目的は何ですか？</strong></p>
<ul>
<li><b>感謝の意を伝え、今後の教材作成の方針を示すため</b> <span style='color:green;'>(correct)</span></li>
<li>単に挨拶をするため</li>
<li>過去の教材の問題点を指摘するため</li>
<li>新しい教材の具体的な内容を説明するため</li>
</ul>
<p><em>Explanation: このメッセージは感謝の意を示し、今後の教材作成に対する方針を述べ、引き続きの指導をお願いしています。</em></p>
<h3>領収書を求める</h3>
<p><strong>Q: 男の人は何のために領収書を求めていますか。</strong></p>
<ul>
<li>おみやげとして持ち帰るため</li>
<li><b>会社に経費申請をするため</b> <span style='color:green;'>(correct)</span></li>
<li>家族に見せるため</li>
<li>返品のため</li>
</ul>
<p><em>Explanation: 「会社名でお願いします」「経費として申請する必要がある」「経理に提出」という表現から、会社の経費処理のために領収書が必要であることが分かります。N3レベルでは「経費」「申請」「経理」などの語彙がやや難しいですが、文脈から判断できます。他の選択肢も現実的ですが、会話の内容と一致するのは2番だけです。</em></p>
<h3>技術について話す</h3>
<p><strong>Q: この会話から、女の人が会議で一番気をつけたいと思っていることは何ですか。</strong></p>
<ul>
<li>会議の時間を短くすること</li>
<li>専門用語を多く使うこと</li>
<li><b>分かりやすく説明すること</b> <span style='color:green;'>(correct)</span></li>
<li>資料をたくさん作ること</li>
</ul>
<p><em>Explanation: この問題の主な難しさは、「女の人が一番気をつけたいこと」を会話全体から推測しなければならない点です。「資料はだいたいできた」「専門用語が多くて、みんなに分かりやすく説明できるか心配」と述べており、さらに「イメージしやすい例を使う」「要点をまとめて話す」と話しています。つまり、分かりやすく伝えることを最も重視していると分かります。N3レベルとしては、会話の流れと主旨を正確に把握する力が問われます。</em></p>
<h3>音楽について話す</h3>
<p><strong>Q: 男の人は最近、どんな音楽をよく聞いていますか。</strong></p>
<ul>
<li>ポップス</li>
<li>ジャズ</li>
<li><b>クラシック</b> <span style='color:green;'>(correct)</span></li>
<li>ロック</li>
</ul>
<p><em>Explanation: この問題の難しさは、女の人も自分の好きな音楽（ポップス）について話しているため、受験者が会話の中心がどこかを正確に聞き取る必要がある点です。また、「リラックスしたい」「静かな時間」「通勤のとき」など、クラシックを聞く具体的な場面がいくつか出てきます。複数のジャンル名が出てくる中で、男の人が「最近よく聞いている」のがどれかを選ぶ力が問われます。N3レベルの語彙・文法で構成されており、受験者の聴解力を適切に測る良問です。</em></p>
<h2>概要理解 (summary_understanding)</h2>
<p>3問：会話や説明の全体的な内容を理解する。</p>
<h3>教育について話す</h3>
<p><strong>Q: 会話の内容から、男の学生の将来の計画として正しいものはどれですか。</strong></p>
<ul>
<li>先生に相談する</li>
<li>仕事を探す</li>
<li><b>大学で専門的なことを学ぶ</b> <span style='color:green;'>(correct)</span></li>
<li>掃除をする</li>
</ul>
<p><em>Explanation: この問題は、会話の流れを正確に理解し、登場人物が「将来何をしたいか」という意志を述べる場面を正しく把握できるかを問います。男の学生は「大学に行って、もっと専門的（※ある分野について深く学ぶこと）のことを勉強したい」と発言しています。N3学習者にとって「専門的」の意味が難しい場合があるため、注釈をつけています。選択肢は会話に出てくるキーワードを含みますが、正しく聞き分ける必要があります。「相談」「仕事」「掃除」は会話の中の話題ですが、男の学生の将来の計画として当てはまるのは3番です。選択肢の中から正しいものを選ぶためには、主旨をしっかりと聞き取る力が必要です。</em></p>
<h3>家の改善について話す</h3>
<p><strong>Q: 女の学生はこの後、まず何をしますか。</strong></p>
<ul>
<li>ペンキを買いに行く</li>
<li>業者に相談する</li>
<li><b>家具の配置を変える</b> <span style='color:green;'>(correct)</span></li>
<li>壁の色を決める</li>
</ul>
<p><em>Explanation: この問題のポイントは、会話の終わりで「じゃあ、今日は家具を動かしてみることにするよ」と女の学生が言っている点です。N3レベルの語彙で「家具を動かす」「壁の色を決める」「ペンキを買いに行く」などの表現が使われており、聞き取りで動作の順番や意思決定の流れを正確に把握できるかが問われています。選択肢の中で、女の学生が次に実際に行うと述べているのは「家具の配置を変える」なので、3が正解です。</em></p>
<h3>環境問題について話す</h3>
<p><strong>Q: この後、男の人は何をするつもりですか。</strong></p>
<ul>
<li>ゴミの分別方法を調べる</li>
<li>プラスチックごみを減らす工夫を考える</li>
<li><b>クリーン活動に参加する</b> <span style='color:green;'>(correct)</span></li>
<li>マイバッグを買いに行く</li>
</ul>
<p><em>Explanation: この問題のポイントは、最後のやりとりに注目することです。女の人が「一緒に町のクリーン活動に参加してみない?」と提案し、男の人が「いいね。参加してみようかな。」と答えています。したがって、男の人が次にすることは「クリーン活動に参加する」ことです。選択肢1, 2, 4は会話の内容に出てきますが、男の人が「するつもり」と明確に言ったのは3だけです。N3レベルでは、「～しようかな」や「～するつもり」など意志を表す表現に注意して聞き取りましょう。</em></p>
<h2>発言表現 (active_expression)</h2>
<p>4問：状況に応じた適切な発言を選ぶ。</p>
<h3>友人との日常会話</h3>
<p><strong>Q: 矢印で示されたリョウの次の発言として最も適切なものを選びなさい。</strong></p>
<ul>
<li><b>ちょっと駅の売店で飲み物を買ってこようか。</b> <span style='color:green;'>(correct)</span></li>
<li>今日はもう一度お店に戻ろうか。</li>
<li>今から歩いて家まで帰ろうか。</li>
</ul>
<p><em>Explanation: この問題のポイントは、会話の流れと状況理解です。リョウは「電車が来るまで10分くらいある」と言っているので、待ち時間をどう使うかを考えています。1の「ちょっと駅の売店で飲み物を買ってこようか」は、短い待ち時間にできる行動として自然です。2の「もう一度お店に戻ろうか」は、買い物を終えたばかりで時間も限られているため不自然です。3の「今から歩いて家まで帰ろうか」は、電車を待っている状況に合いません。N3レベルの学習者にとって、会話の流れや状況から最も適切な行動を選ぶ力が問われます。選択肢がすべて「～しようか」という同じ文型になっているため、発話内容の違いに注意する必要があります。</em></p>
<h3>新しいスキルを学ぶ計画について話す</h3>
<p><strong>Q: このあと、矢印で示された女の人は何と言いますか。</strong></p>
<ul>
<li><b>アルバイトの予定を確認してみるね。</b> <span style='color:green;'>(correct)</span></li>
<li>もうプログラミングはできるから大丈夫。</li>
<li>土曜日はいつも家族と出かけるんだ。</li>
</ul>
<p><em>Explanation: This question tests the ability to understand spoken Japanese in a realistic planning context, focusing on negotiating schedules and expressing willingness to learn a new skill. The main challenge for N3-level students is to catch the details about the online course timing and the woman's schedule. Distractors (2 and 3) require careful listening: (2) contradicts her interest in learning something new, and (3) is not mentioned in the dialogue. The correct answer (1) reflects her actual response and fits the situation.</em></p>
<h3>引っ越しの準備について話す</h3>
<p><strong>Q: What will the woman (pointed by the arrow) say next?</strong></p>
<ul>
<li><b>まず、本を箱に入れるね。</b> <span style='color:green;'>(correct)</span></li>
<li>ちょっとスーパーに買い物に行ってくるね。</li>
<li>先にテレビを見てもいい？</li>
</ul>
<p><em>Explanation: The main challenge for students is to understand the sequence of tasks in a moving/packing situation and to interpret what is most logical for the character to do next, based on the conversation. The vocabulary (荷物、整理、本棚、箱、粗大ごみ、申し込み、片付け) is at the N3 level and relevant to the context of moving. Distractors (2 and 3) test if students can distinguish between unrelated or less logical actions versus the next necessary step in the moving process.</em></p>
<h3>日本の祭りや文化イベントについて話す</h3>
<p><strong>Q: What will the person pointed by the arrow (the young woman in yukata) say next?</strong></p>
<ul>
<li><b>じゃあ、私も何か食べてみようかな。</b> <span style='color:green;'>(correct)</span></li>
<li>ちょっと神社で写真を撮ってきます。</li>
<li>そろそろ帰りましょうか。</li>
</ul>
<p><em>Explanation: The main challenge for students is to infer the most natural next action based on the flow of the conversation and the context of a lively festival. The young woman just commented on how fun the festival is, and her friend is eating takoyaki, while the man is considering getting food as well. Option 1 fits the social and situational cues, as it is natural for her to join her friends in eating festival food. Option 2 is possible but less likely as the group is still deciding what to do, and option 3 is inappropriate since they just arrived at the festival. This question tests listening for context, social cues, and natural conversational flow—key skills at the N3 level.</em></p>
<h2>即時応答 (immediate_ack)</h2>
<p>9問：短い問いかけに対してすぐに応答する。</p>
<h3>店で価格を尋ねる</h3>
<p><strong>Q: この後、男の人はどうしますか。</strong></p>
<p><em>Explanation: 「少し考えます」という発言から、男の人はその場ですぐにかばんを買わないことがわかります。N3レベルの学習者にとって、「考えます」の意図を正確に理解することがポイントです。選択肢の中で「かばんは買わない」が最も適切です。</em></p>
<h3>割引交渉</h3>
<p><strong>Q: この後、男の人はどうしますか。</strong></p>
<p><em>Explanation: この問題のポイントは、男の人が「では、それでお願いします」と言っていることです。これは「割引で買うことに決めた」という意味です。N3レベルでは「お願いします」や「それで」といった表現が、承諾や決定を表すことを理解するのが大切です。選択肢2や3は、会話の流れから不自然ですので、1が正解となります。主な難しさは、「では、それでお願いします」が即決の意味で使われることを聞き取れるかどうかです。</em></p>
<h3>交通手段について話す</h3>
<p><strong>Q: 会話を聞いて、男の人はこの後どうしますか。</strong></p>
<p><em>Explanation: この問題の主なポイントは、男の人が「最初は電車で行こうと思ったけど、荷物が多いから車で行くことにした」と発言している点です。最初の予定と実際の行動の違いを聞き取る必要があり、聞き手が「最終的に選んだ交通手段」を正確に判断できるかがN3レベルのリスニングの重要な練習ポイントです。選択肢の中に「バス」という言及のない手段が含まれているため、消去法も有効です。</em></p>
<h3>通勤について説明する</h3>
<p><strong>Q: 男の人はこのあと何をしますか。</strong></p>
<p><em>Explanation: 男の人は「これから駅で手続きしてから帰ろうと思います」と言っています。「これから」という言葉から、これから定期券の変更手続きをすることが分かります。選択肢1の「そのまま家に帰る」は、まだ帰らないことから不正解です。選択肢3の「会社に戻る」も、会話の流れからその予定がないため不正解です。</em></p>
<h3>おすすめを尋ねる</h3>
<p><strong>Q: 女の人はこれからどうしますか。</strong></p>
<p><em>Explanation: この問題のポイントは、「おすすめのお店」を尋ねるやりとりの中で、女の人が「行ってみます」と答えている点です。N3レベルの文法・語彙（「おすすめ」「カフェ」「ランチメニュー」「行ってみます」など）を理解し、会話の流れから次の行動を推測する必要があります。選択肢2や3は会話の内容と一致しません。会話を正確に聞き取り、意図をつかむことが主なチャレンジです。</em></p>
<h3>本について話す</h3>
<p><strong>Q: 男の人がこれからすることとして最もよいものを一つ選びなさい。</strong></p>
<p><em>Explanation: 男の人は「今日はそのために、もう一度本を読んでおこうと思います」と言っています。「これからすること」を聞いているので、選択肢2が正解です。発表するのは「今度の読書会」であり、友達に貸す話は出ていません。ポイントは「〜ておく」「〜と思います」などの意志表現の理解です。</em></p>
<h3>ペットについて話す</h3>
<p><strong>Q: 会話を聞いて、次に男の人が何をするか、最もよいものを一つ選びなさい。</strong></p>
<p><em>Explanation: この問題のポイントは、「仕事が終わったらすぐに家に帰って、犬の散歩に行くつもりです。」という男の人の発言内容を正確に聞き取れるかどうかです。N3レベルの語彙や文法を使っていますが、「つもりです」や「できるだけ」など、意志や頻度を表す表現が理解できているかが問われます。また、「犬の散歩に行く」という日常的な行動を正しく選択肢から選ぶことも重要です。</em></p>
<h3>ファッションとスタイルについて話す</h3>
<p><strong>Q: この後、女の人は何をするつもりですか。</strong></p>
<p><em>Explanation: 女の人は「自分のスタイルを大切にしたい」と話しているので、「自分に合ったスタイルを探してみる」が最も会話の流れに合っています。選択肢1は流行を追う行動、選択肢3は相談する行動ですが、会話の内容からは「自分に合ったスタイルを見つけたい」という意志が読み取れるため、2が正解です。この問題の主なポイントは、「ファッション」と「スタイル」の違いを理解し、意志表現（～したい、～つもり）を正しく聞き取ることです。また、選択肢の違いを正確に判断する力も問われます。</em></p>
<h3>地域のイベントについて話す</h3>
<p><strong>Q: このあと、女性は何をしますか。</strong></p>
<p><em>Explanation: この問題の主なポイントは、会話の最後で女性が「九時半ごろに駅で待ち合わせしましょうか」と提案している点です。N3レベルでは、会話の内容を正確に理解し、登場人物の次の行動を判断する力が求められます。選択肢2や3は会話からの誤解を誘うものですが、正解は「九時半に駅に行く」です。細かい時間や場所の情報を正確に聞き取ることが、学習者にとっての主なチャレンジとなります。</em></p>
</body></html>

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"/content/jlpt_simulator/JLPT_{timestamp}.html"

with open(filename, "w", encoding="utf-8") as file:
    file.write(html_output)